In [1]:
# !/usr/bin/env python
import os.path
import math
from functools import reduce
import h5py
import numpy as np
import scipy
from scipy import ndimage
from scipy.optimize import curve_fit
import peakutils
from peakutils.plot import plot as pplot
#import ipympl
#%matplotlib ipympl
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import time

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm

import pandas as pd

from ipywidgets import interact, interactive, fixed, interact_manual, Button, VBox, HBox, interactive, interactive_output
import ipywidgets as widgets
import traitlets
from IPython.display import display, clear_output
import collections
try:
	import LightPipes
except ImportError:
	print("LightPipes not present")
	exit()

from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
#rc('font',**{'family':'sans-serif','sans-serif':['DejaVu Sans']})

## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
#rc('text', usetex=False)

import cv2
import scipy.io as sio

In [2]:
from scipy.signal import convolve2d as conv2

from skimage import color, data, restoration

from scipy import fftpack

In [3]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:8000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [4]:
%matplotlib inline
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%config InlineBackend.close_figures=True


In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [6]:
data_dir = '/asap3/flash/gpfs/fl24/2017/data/11004861/raw/hdf/wodzinski/'
scratch_dir = '/asap3/flash/gpfs/fl24/2017/data/11004861/scratch_cc/'
processed_dir = '/asap3/flash/gpfs/fl24/2017/data/11004861/processed/'
useful_dir = scratch_dir + 'useful/'
#useful_dir = 'D:/useful/'
print(useful_dir)

/asap3/flash/gpfs/fl24/2017/data/11004861/scratch_cc/useful/


In [7]:
%cd $data_dir
%ls -l --block-size=G $data_dir 
filelist=!ls $data_dir

/asap3/flash/gpfs/fl24/2017/data/11004861/raw/hdf/wodzinski
total 687G
-r--r----- 1 fsdata fsdata 121G Dec  3  2017 FLASH2_USER1-2017-11-24T1309.h5
-r--r----- 1 fsdata fsdata  18G Dec  3  2017 FLASH2_USER1-2017-11-24T1907.h5
-r--r----- 1 fsdata fsdata  71G Dec  3  2017 FLASH2_USER1-2017-11-24T2020.h5
-r--r----- 1 fsdata fsdata  49G Dec  3  2017 FLASH2_USER1-2017-11-25T0141.h5
-r--r----- 1 fsdata fsdata   7G Dec  3  2017 FLASH2_USER1-2017-11-25T0441.h5
-r--r----- 1 fsdata fsdata  73G Dec  3  2017 FLASH2_USER1-2017-11-26T1916.h5
-r--r----- 1 fsdata fsdata  67G Dec  3  2017 FLASH2_USER1-2017-11-26T2300.h5
-r--r----- 1 fsdata fsdata  20G Dec  3  2017 FLASH2_USER1-2017-11-27T0241.h5
-r--r----- 1 fsdata fsdata  48G Dec  3  2017 FLASH2_USER1-2017-11-27T1101.h5
-r--r----- 1 fsdata fsdata  14G Dec  4  2017 FLASH2_USER1-2017-11-27T1402.h5
-r--r----- 1 fsdata fsdata  35G Dec  4  2017 FLASH2_USER1-2017-11-27T1529.h5
-r--r----- 1 fsdata fsdata  44G Dec  4  2017 FLASH2_USER1-2017-11-28T1915.h5
-r--r

## List all groups inside the hd5file

In [8]:
def printname(name):
    print(name)
#hdf5_file = h5py.File(data_dir + hdf5_file_name, 'r') # 'r' means that hdf5 file is open in read-only mode
#hdf5_file.visit(printname)

## Dataset definitions for images and backgrounds


In [9]:
dataset_bg_args_d={
    'FLASH2_USER1-2017-11-24T1309.h5':[
        #wavelength_set = 13.5
        
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        (88400,90000,'bg1cd',0.6,1,7.0,50.0,'none'), #    15:36:00    15:39:00    20395    background 1c to 1d           
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    background 2c to 2d            70600    72400
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    background 3c to 3d        88000    89800
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        (123000,126200,'bg4cd',0.6,1,7.0,50.0,'none') # 17:05:00    17:11:00    20395    4d    1335    horizontal 141400    145000 
        
    ],
    'FLASH2_USER1-2017-11-24T2020.h5':[
        #wavelength_set = 13.5
        (27000,27600,'bg2bc',0.6,1,7.0,50.0,'none'), #    20:56:00    21:01:00    20428    background 2b to 2c            21400    24400
        #(27600,28000,'background 2b to 2c (with closed shutter)',0.6,1,7.0,50.0,'none'), #    21:03:00    21:05:00    20428    background with closed shutter         25600    26800
        #(28000,30000,'background 2b to 2c (with FEL)',0.6,1,7.0,50.0,'none'), #    21:05:00    21:11:00    20428    background with FEL            26800    30400
        #(31000,32800,'background 2b to 2c (with Zr200)',0.6,0), #    21:12:00    21:15:00    20428    background with Zr200            31000    32800
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none') # 23:41:00    23:46:00    20428    background 5a 5b            120400    123400
    ],
    'FLASH2_USER1-2017-11-25T0141.h5':[
        #wavelength_set = 18.0
        (6500,9500,'bg2ab',0.6,1,7.0,50.0,'none'), #	1:52:00	01:57:00	20441	background 2a 2b YAG+FEL			6500	9500
        (91100,92300,'bg5cd',0.6,1,7.0,50.0,'none') #	4:13:00	04:15:00	20441	background 5c 5d			91100	92300
    ],
    'FLASH2_USER1-2017-11-26T1916.h5':[
        #wavelength_set = 18.0
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        (129500,131500,'bg2ab',0.6,1,7.0,50.0,'none'), #	22:52:00	22:55:00	20461	background 2a 2b			129500	131300
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	background 3a 3b			124700	127700
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	background 4c 4d	74900	76100
        (116300,118800,'bg5cd',0.6,1,7.0,50.0,'none') #	22:29:00	22:34:00	20461	5c	445	horizontal	115700	118700
    ],
    'FLASH2_USER1-2017-11-26T2300.h5':[
        #wavelength_set = 18.0
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	background 1b to 1a			50900	53900
        (7200,10400,'bg2ab',0.6,1,7.0,50.0,'none'), #	26.11.17 23:12	23:17:00	20462	background 2a to2b			7100	10100
        (67450,70900,'bg3ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:52	00:58:00	20462	background 3a 3b			67100	70700
        (88200,91100,'bg4ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:27	01:32:00	20462	background 4a to 4b			88100	91100
        (106600,110700,'bg5ab',0.6,1,7.0,1.5,'none') #	27.11.17 1:57	02:04:00	20462	background 5a to 5b			106100	110300
    ],
    'FLASH2_USER1-2017-11-27T1101.h5':[
        #wavelength_set = 8.0
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
#       (31900,32000,'background 2a to 2b Nb405',0.6,0), #	Nb 405nm 11:55:00	11:56:00	20469	background			32100	32700 RAUS, keine Daten, da zu wenig Intensität
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
        (91100,93300,'bg5cd',0.6,1,7.0,50.0,'none') #	no filter 13:33:21	13:37:01	20469	background			90900	93900
    ],
    'FLASH2_USER1-2017-11-27T1529.h5':[
        #wavelength_set = 8.0
        (30300,31500,'bg1cd',0.6,1,7.0,50.0,'none'), #	16:20:00	16:22:00		background 1d 1c			30300	31500
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        (59200,60400,'bg5ab',0.6,1,7.0,50.0,'none') #	17:08:00	17:10:00		Backgound 5a 5b			59100	60300
    ],
    'FLASH2_USER1-2017-11-29T1007.h5':[
        #wavelength_set = 13.5
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		background 1d 1c	17700	20100 KAOS=1.5mm
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700 KAOS=PMMA
        (101700,103600,'bg2ab#2',0.6,1,8.8,3.0,'none'), #	12:57	13:01		backgroung			101700	104100 KAOS=1.5mm
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700 KAOS=PMMA
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none') #	12:08	12:11		background			72300	74100 KAOS=PMMA
        
    ]
}

dataset_bg_args = collections.OrderedDict(sorted(dataset_bg_args_d.items(), key=lambda t: t[0]))

dataset_image_args_d={
    'FLASH2_USER1-2017-11-24T1309.h5':[
        #wavelength_set = 13.5
        (105800,107000,'1a#1',0.6,1,7.0,50.0,'none'), #     20395    1a    50    vertical 
        (107500,112000,'1a#2',0.6,1,7.0,50.0,'none'), #    1a 50
        (177300,178500,'1a#3',0.6,1,7.0,50.0,'none'), # 1a 50
        (91500,94400,'1c',0.6,1,7.0,50.0,'none'), # 15:41:00    15:46:00    20395    1c    50    horizontal 91000    94000
        (56000,59000,'2a',0.6,1,7.0,50.0,'none'), # 14:41:00    14:48:00    20395    2a    107    vertical    55000 59200
        (67200,70500,'2c',0.6,1,7.0,50.0,'none'), # 15:00:00    15:07:00    20395    2c    107    horizontal 66400    70600
        (0,5000,'3a#1',0.6,1,7.0,50.0,'none'),     # 13:09:20    13:17:00    20395    3a    215    vertical    0 4600
        (46400,48400,'3a#2',0.6,1,7.0,50.0,'none'), # 14:26:00    14:30:00    20395    3a    215    vertical    46000 48400
        (28000,30500,'3c',0.6,1,7.0,50.0,'none'), # 13:54:00    13:59:00    20395    3c    215    horizontal 26800    29800
        (115500,118000,'4a',0.6,1,7.0,50.0,'none'), # 4a 322 vertical
        (132500,136500,'4c#1',0.6,1,7.0,50.0,'none'), # 4c 322 horizontal
        (139700,140400,'4c#2',0.6,1,7.0,50.0,'none'), # 20395    4c    322    vertical
        (184000,187000,'5a',0.6,1,7.0,50.0,'none'), # 5a 445
        (153000,156000,'5c#1',0.6,1,7.0,50.0,'none'), # 5c 445
        (197500,201500,'5c#2',0.6,1,7.0,50.0,'none'), # 18:39:00    18:44:00    20395    5c    445    horizontal 197800    200800
        (99300,103900,'1b',0.6,1,7.0,50.0,'none'), # 16:02:00    16:15:00    20395    1b    707    vertical   
        (83000,87400,'1d',0.6,1,7.0,50.0,'none'), # 15:29:00    15:35:00    20395    1d    707    horizontal 83800    87400
        (60400,65300,'2b',0.6,1,7.0,50.0,'none'), # 14:50:00    15:57:00    20395    2b    890    vertical    60400 10060
        (74500,79200,'2d',0.6,1,7.0,50.0,'none'), # 15:13:00    15:21:00    20395    2d    890    horizontal 74200    79000
        (6600,10000,'3b#1',0.6,1,7.0,50.0,'none'), # 13:20:00    13:25:00    20395    3b    1047    vertical    6400 9400
        (41500,44800,'3b#2',0.6,1,7.0,50.0,'none'), # 14:18:00    14:23:00    20395    3b    1047    vertical 41200    44200
        (36500,40000,'3d',0.6,1,7.0,50.0,'none'), # 14:10:00    14:15:00    20395    3d    1047    horizontal 36400    39400
        (127300,130000,'4b',0.6,1,7.0,50.0,'none'), # 4b 1335 vertical
        
        
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        (88400,90000,'bg1cd',0.6,1,7.0,50.0,'none'), #    15:36:00    15:39:00    20395    background 1c to 1d           
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    background 2c to 2d            70600    72400
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    background 3c to 3d        88000    89800
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        (123000,126200,'bg4cd',0.6,1,7.0,50.0,'none') # 17:05:00    17:11:00    20395    4d    1335    horizontal 141400    145000 
        
    ],
    'FLASH2_USER1-2017-11-24T2020.h5':[
        #wavelength_set = 13.5
        (60000,63000,'1a',0.6,1,7.0,50.0,'none'), # 21:59:00 22:05:00    20428    1a    50        59200    62800
        (55500,58000,'1c',0.6,1,7.0,50.0,'none'), # 21:52:00 21:57:00    20428    1c    50        55000    58000
        (35200,38800,'2a',0.6,1,7.0,50.0,'none'), # 21:19:00 21:25:00    20428    2a    107    vertical    35200 38800
        (41600,44800,'2c',0.6,1,7.0,50.0,'none'), # 21:29:00 21:35:00    20428    2c    107        41200    44800
        (71200,74200,'3a',0.6,1,7.0,50.0,'none'), # 22:18:00 22:24:00    20428    3a    215    vertical    70600 74200
        (79600,82000,'3c',0.6,1,7.0,50.0,'none'), # 22:32:00 22:37:00    20428    3c    215    horizontal    79000 82000
        (88000,91600,'4a',0.6,1,7.0,50.0,'none'), # 22:47:00 22:53:00    20428    4a    322        88000    91600
        (106600,110500,'4c',0.6,1,7.0,50.0,'none'), # 23:18:00 23:24:00  20428    4c    322    horizontal    106600 110200
        (116800,120000,'5a',0.6,1,7.0,50.0,'none'), # 23:35:00 23:40:00  20428    5a    445    vertical    116800 119800
        (126800,129200,'5c',0.6,1,7.0,50.0,'none'), # 23:51:00 23:55:00  20428    5c    445    horizontal    126400 128800
        (63700,67000,'1b',0.6,1,7.0,50.0,'none'), # 22:06:00 22:12:00    20428    1b    707        63400    67000
        (52500,54400,'1d',0.6,1,7.0,50.0,'none'), # 21:45:00 21:51:00    20428    1d    707    horizontal    50800 54400
        (1000,2500,'2b#1',0.6,1,7.0,50.0,'none'), #20:20:20    20:24:00 20428    2b    890        0    2200
        (22000,25000,'2b#2',0.6,1,7.0,50.0,'none'), # 20:45:00 20:47:00    20428    2b    890        14800    16000
        (51300,52300,'1d#2',0.6,1,7.0,50.0,'none'), # 21:37:00 21:43:00    20428    2d    890        46000    49600
        (74800,77800,'3b',0.6,1,7.0,50.0,'none'), # 22:25:00 22:30:00    20428    3b    1047    vertical    74800 77800
        (83200,85600,'3d',0.6,1,7.0,50.0,'none'), # 22:38:00 22:43:00    20428    3d    1047        82600    85600
        (102800,105500,'4b',0.6,1,7.0,50.0,'none'), # 23:11:00 23:16:00    20428    4b    1335    vertical    102400 105400
        (111500,115500,'4d',0.6,1,7.0,50.0,'none'), # 23:25:00 23:32:00    20428    4d    1335    horizontal 110800    115000
        (124000,125400,'5b',0.8,1,7.0,50.0,'none'), # 23:47:00 23:49:00    20428    5b    1570    vertical    124000 125200
        (129700,130800,'5d',0.8,1,7.0,50.0,'none'), # 23:56:00 23:59:00    20428    5d    1570    horizontal    129400 131200
        
        
        (27000,27600,'bg2bc',0.6,1,7.0,50.0,'none'), #    20:56:00    21:01:00    20428    background 2b to 2c            21400    24400
        #(27600,28000,'background 2b to 2c (with closed shutter)',0.6,1,7.0,50.0,'none'), #    21:03:00    21:05:00    20428    background with closed shutter         25600    26800
        #(28000,30000,'background 2b to 2c (with FEL)',0.6,1,7.0,50.0,'none'), #    21:05:00    21:11:00    20428    background with FEL            26800    30400
        #(31000,32800,'background 2b to 2c (with Zr200)',0.6,0), #    21:12:00    21:15:00    20428    background with Zr200            31000    32800
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none') # 23:41:00    23:46:00    20428    background 5a 5b            120400    123400
    
    ],
    'FLASH2_USER1-2017-11-25T0141.h5':[
        #wavelength_set = 18.0
        (33500,37000,'1a',0.6,1,7.0,50.0,'none'), #	2:37:00	02:42:00	20441	1a	50		33500	36500
        (26400,29500,'1c',0.6,1,7.0,50.0,'none'), #	2:25:00	02:30:00	20441	1c	50		26300	29300
        (10200,13500,'2a',0.6,1,7.0,50.0,'none'), #	1:57:00	02:04:00	20441	2a	107	vertical	9500	13700
        (14500,17500,'2c',0.6,1,7.0,50.0,'none'), #	2:05:00	02:10:00	20441	2c	107		14300	17300
        (39100,42300,'3a#1',0.6,1,7.0,50.0,'none'), #	2:46:00	02:52:00	20441	3a	215	vertical	58100	61100
        (46200,48600,'3a#2',0.6,1,7.0,50.0,'none'), #	2:57:00	03:02:00	20441	3a	215	vertical	58100	61100
        (58700,61000,'3a#3',0.6,1,7.0,50.0,'none'), #	3:18:00	03:23:00	20441	3a	215	vertical	58100	61100
        (65700,68700,'4a',0.6,1,7.0,50.0,'none'), #	3:30:00	03:35:00	20441	4a	322		65300	68300
        (73100,76200,'4c',0.6,1,7.0,50.0,'none'), #	3:43:00	03:48:00	20441	4c	322		73100	76100
        (83800,86300,'5a',0.6,1,7.0,50.0,'none'), #	4:00:00	04:05:00	20441	5a	445		83300	86300
        (87700,90500,'5c',0.6,1,7.0,50.0,'none'), #	4:07:00	04:12:00	20441	5c	455		87500	90500
        (30500,33200,'1b',0.6,1,7.0,50.0,'none'), #	2:31:00	02:36:00	20441	1b	707		29900	32900
        (22900,26000,'1d',0.6,1,7.0,50.0,'none'), #	2:19:00	02:24:00	20441	1d	707		22700	25700
        (500,4500,'2b',0.6,1,7.0,50.0,'none'),   #	1:41:10	01:48:00	20441	2b	890		0	4100
        (18200,21100,'2d',0.6,1,7.0,50.0,'none'), #	2:11:00	02:16:00	20441	2d	890	horizontal	17900	20900
        (42800,46000,'3b#1',0.8,1,7.0,50.0,'none'), #	2:52:00	02:57:00	20441	3b	1047	vertical	45500	48500
        (61700,63800,'3b#2',0.8,1,7.0,50.0,'none'), #	3:24:00	03:26:00	20441	3b	1047		61700	62900
        (69200,72400,'4b',0.8,1,7.0,50.0,'none'), #	3:36:00	03:41:00	20441	4b	1335		68900	71900
        (76800,77500,'4d#1',0.65,1,7.0,50.0,'none'), #	3:49:00	03:50:00	20441	4d	1335		76700	77300
        (78400,82000,'4d#2',0.71,1,7.0,50.0,'none'), #	3:51:00	03:57:00	20441	4d	1335		77900	81500
        
        (6500,9500,'bg2ab',0.6,1,7.0,50.0,'none'), #	1:52:00	01:57:00	20441	background 2a 2b YAG+FEL			6500	9500
        (91100,92300,'bg5cd',0.6,1,7.0,50.0,'none') #	4:13:00	04:15:00	20441	background 5c 5d			91100	92300
    ],
    'FLASH2_USER1-2017-11-26T1916.h5':[
        #wavelength_set = 18.0
        (24300,27700,'1a',0.6,1,7.0,50.0,'none'), #	19:56:00	20:02:00	20461	1a	50	vertical	23900	27500
        (34200,37300,'1c',0.6,1,7.0,50.0,'none'), #	20:12:00	20:17:00	20461	1c	50	horizontal	33500	36500
        (47000,50300,'2a',0.6,1,7.0,50.0,'none'), #	20:34:00	20:39:00	20461	2a	107	vertical	46700	49700
        (54200,57600,'2c',0.6,1,7.0,50.0,'none'), #	20:46:00	20:52:00	20461	2c	107	horizontal	53900	57500
        (500,3500,'3a',0.6,1,7.0,50.0,'none'), #	19:17:00	19:22:00	20461	3a	215	vertical	500	3500
        (11250,14700,'3c',0.6,1,7.0,50.0,'none'), #	19:34:00	19:39:00	20461	3c	215	horizontal	10700	13700
        (64050,67400,'4a',0.6,1,7.0,50.0,'none'), #	21:03:00	21:08:00	20461	4a	322	vertical	64100	67100
        (71800,74500,'4c',0.6,1,7.0,50.0,'none'), #	21:15:00	21:20:00	20461	4c	322	horizontal	71300	74300
        (83100,85750,'5a#1',0.6,1,7.0,50.0,'none'), #21:34:00	21:39:00	20461	5a	445	vertical	82700	85700
        (86800,90300,'5a#2',0.6,1,7.0,50.0,'none'), #21:41:00	21:46:00	20461	5a	445	vertical	86900	89900
        (112500,115800,'5c',0.6,1,7.0,50.0,'none'), #22:23:00	22:29:00	20461	5c	445	horizontal	112100	115700
        (29700,33100,'1b',0.6,1,7.0,50.0,'none'), #	20:06:00	20:11:00	20461	1b	707	vertical	29900	32900
        (40100,40400,'1d#1',0.6,1,7.0,50.0,'none'), #20:22:00	20:23:00	20461	1d	707	horizontal	39500	40100
        (41300,45400,'1d#2',0.6,1,7.0,50.0,'none'), #20:25:00	20:30:00	20461	1d	707	horizontal	41300	44300
        (50650,53700,'2b',0.6,1,7.0,50.0,'none'), #	20:40:00	20:45:00	20461	2b	890	vertical	50300	53300
        (58700,62000,'2d',0.6,1,7.0,50.0,'none'), #	20:54:00	20:59:00	20461	2d	890	horizontal	58700	61700
        (7100,10400,'3b',0.6,1,7.0,50.0,'none'), #	19:28:00	19:33:00	20461	3b	1047vertical	7100	10100
        (17000,20900,'3d',0.6,1,7.0,50.0,'none'), #	19:45:00	19:51:00	20461	3d	1047horizontal	17300	20900
        (68100,71100,'4b',0.6,1,7.0,50.0,'none'), #	21:09:00	21:14:00	20461	4b	1335vertical	67700	70700
        (76950,81200,'4d',0.6,1,7.0,50.0,'none'), #	21:24:00	21:31:00	20461	4d	1335horizontal	76700	80900
        (108050,111700,'5b',0.6,1,7.0,50.0,'none'), #22:16:00	22:22:00	20461	5b	1570vertical	107900	111500
        (120100,122600,'5d',0.6,1,7.0,50.0,'none'), #22:35:00	22:40:00	20461	5d	1570horizontal	119300	122300
        
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        (129500,131500,'bg2ab',0.6,1,7.0,50.0,'none'), #	22:52:00	22:55:00	20461	background 2a 2b			129500	131300
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	background 3a 3b			124700	127700
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	background 4c 4d	74900	76100
        (116300,118800,'bg5cd',0.6,1,7.0,50.0,'none') #	22:29:00	22:34:00	20461	5c	445	horizontal	115700	118700
    ],
    'FLASH2_USER1-2017-11-26T2300.h5':[
        #wavelength_set = 18.0
        (54600,57950,'1a',0.6,1,7.0,1.5,'none'), #	27.11.17 0:31	00:36:00	20462	1a	50	vertical	54500	57500
        (33500,37050,'1c#1 ',0.6,1,7.0,50.0,'none'), #	26.11.17 23:55	00:01:00	20462	1c	50	horizontal	32900	36500
        (38800,42100,'1c#2',0.6,1,7.0,1.5,'none'), #	27.11.17 0:05	00:10:00	20462	1c	50	horizontal	38900	41900
        (11300,14800,'2a',0.6,1,7.0,50.0,'none'), #	26.11.17 23:19	23:24:00	20462	2a	107	vertical	11300	14300
        (19300,22850,'2c',0.6,1,7.0,50.0,'none'), #	26.11.17 23:32	23:38:00	20462	2c	107	horizontal	19100	22700
        (64100,66800,'3a',0.6,1,7.0,1.5,'none'), #	27.11.17 0:46	00:51:00	20462	3a	215	vertical	63500	665000
        (75400,79000,'3c',0.6,1,7.0,1.5,'none'), #	27.11.17 1:05	01:11:00	20462	3c	215	horizontal	74900	78500
        (84750,88000,'4a',0.6,1,7.0,1.5,'none'), #	27.11.17 1:21	01:26:00	20462	4a	322	vertical	84500	87500        
        (95300,97800,'4c',0.6,1,7.0,1.5,'none'), #	27.11.17 1:38	01:43:00	20462	4c	322	horizontal	94700	97700
        (103200,106400,'5a',0.6,1,7.0,1.5,'none'), #27.11.17 1:52	01:57:00	20462	5a	445	vertical	103100	106100
        (115400,118800,'5c',0.6,1,7.0,1.5,'none'), #27.11.17 2:12	02:17:00	20462	5c	445	horizontal	115100	118100  
        (46900,50750,'1b',0.6,1,7.0,1.5,'none'), #	27.11.17 0:18	00:24:00	20462	1b	707	vertical	46700	50300
        (28900,32700,'1d#1',0.6,1,7.0,50.0,'none'), #	26.11.17 23:48	23:54:00	20462	1d	707	horizontal	28700	32300
        (42700,45700,'1d#2',0.6,1,7.0,1.5,'none'), #	27.11.17 0:11	00:16:00	20462	1d	707	vertical	42500	45500
        (15500,18800,'2b',0.6,1,7.0,50.0,'none'), #	26.11.17 23:26	23:31:00	20462	2b	890	vertical	15500	18500
        (23800,27300,'2d',0.6,1,7.0,50.0,'none'), #	26.11.17 23:40	23:45:00	20462	2d	890	horizontal	23900	26900
        (71500,74700,'3b',0.8,1,7.0,1.5,'none'), #	27.11.17 0:58	01:04:00	20462	3b	1047vertical	70700	74300 | 5x dunkel pinhole bzw hable pixis bilder dazwischen
        (80000,81750,'3d',0.83,1,7.0,1.5,'none'), #27.11.17 1:12	01:15:00	20462	3d	1047horizontal	79100	80900
        (91600,947500,'4b',0.8,1,7.0,1.5,'none'), # 27.11.17 1:32	01:37:00	20462	4b	1335vertical	91100	94100 | 1x rauslaufendes Pixis Bild
        (98550,101650,'4d',0.83,1,7.0,1.5,'none'), # 27.11.17 1:44	01:49:00	20462	4d	1335horizontal	98300	101300 | 1x dunkel, 1x pixis rauslaufend(letztes bild)
#       (110900,114500,'5b',0.83,1), #27.11.17 2:05	02:11:00	20462	5b	1570vertical	110900	114500 | keine Bilder gefunden
#       (119000,123500,'5d',0.2,1) #27.11.17 2:18	02:25:00	20462	5d	1570horizontal	118700	122900 | ein Bild gefunden                
        
        
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	background 1b to 1a			50900	53900
        (7200,10400,'bg2ab',0.6,1,7.0,50.0,'none'), #	26.11.17 23:12	23:17:00	20462	background 2a to2b			7100	10100
        (67450,70900,'bg3ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:52	00:58:00	20462	background 3a 3b			67100	70700
        (88200,91100,'bg4ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:27	01:32:00	20462	background 4a to 4b			88100	91100
        (106600,110700,'bg5ab',0.6,1,7.0,1.5,'none') #	27.11.17 1:57	02:04:00	20462	background 5a to 5b			106100	110300
    ],
    
    'FLASH2_USER1-2017-11-27T1101.h5':[
        #wavelength_set = 8.0
        # high energy?
        #wavelength_set = 8.0
        # high energy?

        (39125,41700,'1a',0.6,1,7.0,50.0,'none'), #	no filter 12:06:43	12:11:00	20469	1a	50	vertical	38700	41100
#       (45600,45900,'1c',0.6,0), #	no filter 12:17:30	12:18:00	20469	1c	50	horizontal	45300	45900 RAUS, keine Daten, da zu viel Intensität
        (46300,48900,'1c',0.6,1,7.0,50.0,'none'), #	Nb 197nm 12:18:40	12:23:00	20469	1c	50	horizontal	45900	48900
        (33350,33500,'2a#1',0.6,1,7.0,50.0,'Nb405'), #	Nb 405nm 11:57:05	11:57:20	20469	2a	107	vertical	33300	33900
        (34400,36600,'2a#2',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:58:50	12:02:30	20469	2a	107	vertical	33900	36300
        (23700,24900,'2c',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:41:00	11:43:00	20469	2c	107	horizontal	18900	23700
        (3300,6100,'3a',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:07:00	11:11:40	20469	3a	215	vertical	3300	5700
        (6800,7400,'3c#1',0.6,1,7.0,50.0,'none'), #	no Filter 11:12:50	11:13:50	20469	3c	215	horizontal	6300	9900
        (7700,9900,'3c#2',0.6,1,7.0,50.0,'Nb405'), #	Nb 405nm 11:14:20	11:18:00	20469	3c	215	horizontal	6300	9900
        (64500,67600,'4a',0.6,1,7.0,50.0,'none'), #	no filter 12:49:01	12:54:10	20469	4a	322	vertical	63900	67500
        (57800,60400,'4c',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:37:50	12:42:10	20469	4c	322	horizontal	57300	60300
        (70900,74100,'5a#1',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:59:41	13:05:01	20469	5a	445	vertical	71100	74100
        (74700,77300,'5a#2',0.6,1,7.0,50.0,'none'), #	no filter 13:06:01	13:10:21	20469	5a	445	vertical	74700	77100
#       (69900,70500,'5a',0.6,0)  #	Nb 405nm 12:58:01	12:59:00	20469	5a	445	vertical	69900	70500 RAUS, keine Daten, da zu wenig Intensität
        (83900,87400,'5c',0.6,1,7.0,50.0,'none'), #	no filter 13:21:21	13:27:11	20469	5c	445	horizontal	83700	86700
        (42200,44800,'1b',0.6,1,7.0,50.0,'none'), #	no filter 12:11:50	12:16:10	20469	1b	707	vertical	41700	44700
        (49700,51700,'1d',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:24:20	12:27:40	20469	1d	707	horizontal	49500	51300
        (18550,18850,'2d#1',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:32:25	11:32:55	20469	2d	890	horizontal	15900	18900
        (19250,21900,'2d#2',0.6,1,7.0,50.0,'none'), #	no filter 11:33:35	11:38:00	20469	2d	890	horizontal	18900	23700
        (27900,29600,'2b',0.6,1,7.0,50.0,'none'), #	no filter 11:48:00	11:50:50	20469	2b	890	vertical	27300	29700
#       (26300,27500,'2b',0.6,0), #	Nb 197nm 11:46:00	11:46:00	20469	2b	890	vertical	25500	26700 RAUS, keine Daten, da zu wenig Intensität
        (250,900,'3b',0.6,1,7.0,50.0,'none'), 	  #	no filter 11:01:55	11:03:00	20469	3b	1047	vertical	0	900
        (11150,13700,'3d',0.6,1,7.0,50.0,'Nb405'), #	Nb 405nm 11:20:05	11:24:20	20469	3d	1047	horizontal	11100	13500
        (60950,61200,'4b#1',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:43:06	12:43:31	20469	4b	1335	vertical	60900	61500
        (61550,63700,'4b#2',0.6,1,7.0,50.0,'none'), #	no filter 12:44:06	12:47:41	20469	4b	1335	vertical	61500	63300
        (54970,56600,'4d',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:33:07	12:35:50	20469	4d	1335	horizontal	54900	56700
        (80900,83300,'5b',0.6,1,7.0,50.0,'none'), #	no filter 13:16:21	13:20:21	20469	5b	1570	vertical	80700	83100
        (87750,90200,'5d',0.6,1,7.0,50.0,'none'), #	no filter 13:27:46	13:31:51	20469	5d	1570	horizontal	87300	90300
        
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
#       (31900,32000,'background 2a to 2b Nb405',0.6,0), #	Nb 405nm 11:55:00	11:56:00	20469	background			32100	32700 RAUS, keine Daten, da zu wenig Intensität
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
        (91100,93300,'bg5cd',0.6,1,7.0,50.0,'none') #	no filter 13:33:21	13:37:01	20469	background			90900	93900
    ],
    'FLASH2_USER1-2017-11-27T1529.h5':[
        #wavelength_set = 8.0
        (40200,42500,'1a',0.6,1,7.0,50.0,'none'), #	16:36:00	16:42:00		1a	50	vertical	39900	43500
        (32800,35700,'1c',0.6,1,7.0,50.0,'none'), #	16:23:00	16:28:00		1c	50	horizontal	32100	35100
        (20700,23100,'2c#1',0.6,1,7.0,50.0,'none'), #	16:04:00	16:08:00		2c	107	horizontal	20700	23100
        (11900,14000,'3a#1',0.6,1,7.0,50.0,'none'), #	15:49:00	15:53:00		3a	215	vertical	11700	14100
        (15800,17700,'2a#1',0.6,1,7.0,50.0,'none'), #	15:55:00	15:59:00		3a	215	vertical	15300	17700
        (0,2100,'3c',0.6,1,7.0,50.0,'none'), #	15:29:30	15:33:00	20476	3c	215	horizontal	0	2100
        (44000,46500,'4a',0.6,1,7.0,50.0,'none'), #	16:42:00	16:46:00		4a	322	vertical	43500	45900
        (47900,50500,'4c#1',0.6,1,7.0,50.0,'none'), #	16:49:00	16:54:00		4c	322	horizontal	47700	50700
        (51300,53400,'4c#2',0.6,1,7.0,50.0,'none'), #	16:55:00	16:58:00		4c	322	horizontal	51300	53100
        (56400,58600,'5a',0.6,1,7.0,50.0,'none'), #	17:03:00	17:07:00		5a	445	vertical	56100	58500
        (36600,39500,'1b',0.6,1,7.0,50.0,'none'), #	16:30:00	16:35:00		1b	707	vertical	36300	39300
        (27300,29700,'1d',0.6,1,7.0,50.0,'none'), #	16:15:00	16:19:00		1d	707	horizontal	27300	29700
        (23800,25100,'2d',0.6,1,7.0,50.0,'none'), #	16:09:00	16:13:00		2d	890	horizontal	23700	26100
        (6600,8500,'3d',0.95,1,7.0,50.0,'none'), #	15:40:00	15:44:00		3d	1047	horizontal	6300	8700
        (9500,11000,'3b#1',0.9,1,7.0,50.0,'none'), #	15:44:00	15:48:00		3b	1047	vertical	8700	11100
        (18300,20100,'3b#2',0.6,1,7.0,50.0,'none'), #	15:59:00	16:03:00		3b	1047	vertical	17700	20100
        #no data(46625,46700,'4b',0.9,0), #	16:47:00	16:48:00		4b	1335	vertical	46500	47100
        (54000,54800,'4d',0.91,1,7.0,50.0,'none'), #	16:59:00	17:01:00		4d	1335	horizontal	53700	54900
        (61600,64500,'5c',0.6,1,7.0,50.0,'none'), #	17:12:00	17:17:00		5b	1570	vertical	61500	64500
        
        (30300,31500,'bg1cd',0.6,1,7.0,50.0,'none'), #	16:20:00	16:22:00		background 1d 1c			30300	31500
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        (59200,60400,'bg5ab',0.6,1,7.0,50.0,'none') #	17:08:00	17:10:00		Backgound 5a 5b			59100	60300
    ],
    'FLASH2_USER1-2017-11-29T1007.h5':[
        #wavelength_set = 13.5
        # beamsize 1.5mm
        #aperture5 = 7
        
        (26300,28100,'1a#1',0.6,1,7.0,50.0,'none'), #	10:51	10:54		1a	50	vertical	26100	27900
        (20900,23500,'1c#1',0.6,1,7.0,50.0,'none'), #	10:42	10:46		1c	50	horizontal	20700	23100
        
        (3500,6500,'2a#1',0.6,1,7.0,50.0,'none'), # 	10:13	10:18		2a	107	vertical
        (7000,9600,'2c#1',0.6,1,7.0,50.0,'none'), # 	10:19	10:23		2c	107	horizontal
        
        (36600,38700,'3a#1',0.6,1,7.0,50.0,'none'), #	11:08	11:12		3a	215	vertical	36300	38700
        (42700,44800,'3c#1',0.6,1,7.0,50.0,'none'), #	11:18	11:22		3c	215	horizontal	42300	44700
        
        (56100,58400,'4a#1',0.6,1,7.0,50.0,'none'), #	11:41	11:44		4a	322	vertical	56100	57900
        (51300,53500,'4c#1',0.6,1,7.0,50.0,'none'), #	11:33	11:36		4c	322	horizontal	51300	53100
        
        (61400,64100,'5a#01',0.6,1,7.0,3.0,'none'), #	11:50	11:53		5a	445	vertical	61500	63300
        (66600,68400,'5c#1',0.85,1,7.0,3.0,'none'), #11:58	12:01		5c	445	horizontal	66300	68100
                
        (15000,17100,'1d#0',0.6,1,7.0,50.0,'none'), #	10:32	10:36		1d	707	horizontal	14700	17100
        (23700,26000,'1b#1',0.6,1,7.0,50.0,'none'), #	10:46	10:50		1b	707	vertical	23100	25500
        
        (0,3500,'2b#0',0.6,1,7.0,50.0,'none'),   # 	10:09	10:13		2b	890	vertical
        (10500,13000,'2d#1',0.6,1,7.0,50.0,'none'), #	10:25	10:28		2d	890	horizontal
        
        (40300,42300,'3b#1',0.6,1,7.0,50.0,'none'), #	11:14	11:17		3b	1047	vertical	39900	41700
        (45300,47600,'3d#1',0.6,1,7.0,50.0,'none'), #	11:23	11:26		3d	1047	horizontal	45300	47100
        
        (48900,50900,'4d#1',0.6,1,7.0,50.0,'none'), #	11:29	11:32		4d	1335	vertical	48900	50700
        (53600,55700,'4b#1',0.6,1,7.0,50.0,'none'), #	11:37	11:40		4b	1335	vertical	53700	55500
        
        (64500,66200,'5b#1',0.9,1,7.0,3.0,'none'), #	11:54	11:57		5b	1570	vertical	63900	65700, 
        (69100,70100,'5d#1',0.85,1,7.0,3.0,'none'), #	12:02	12:04		5d	1570	horizontal	68700	69900
        
        # aperture5=8.8

        (111500,113500,'1a#2',0.6,1,8.8,3.0,'none'), #	13:13	16:16		1a	50	vertical	111300	113100
        (104500,106500,'2a#2',0.6,1,8.8,3.0,'none'), #	13:02	13:05		2a	107	vertical	104700	106500
        (89000,91500,'3a#2',0.6,1,8.8,50.0,'none'), #	12:35	12:40		3a	215	vertical	88500	91500     
        (84700,87200,'4a#2',0.6,1,8.8,50.0,'none'), #	12:27	12:32		4a	322	vertical	83700	86700
        (74900,77050,'5a#02',0.6,1,8.8,3.0,'none'), #	12:13	12:15		5a	445	vertical	75300	76500
        (113700,115500,'1b#2',0.6,1,8.8,3.0,'none'), #	13:17	13:20		1b	707	vertical	113700	115500
        (95900,98100,'2b#1',0.6,1,8.8,50.0,'none'), #	12:47	12:50		2b	890	vertical	95700	97500
        (99500,101300,'2b#2',0.6,1,8.8,3.0,'none'), #12:53	12:56		2b	890	vertical	99300	101100
        (91900,94100,'3b#2',0.6,1,8.8,50.0,'none'), #	12:40	12:44		3b	1047	vertical	91500	93900
        (82200,84000,'4b#2',0.6,1,8.8,50.0,'none'), #	12:24	12:27		4b	1335	vertical	81900	83700
        (77500,79400,'5b#2',0.85,1,8.8,3.0,'none'), #	12:16	12:19		5b	1570	vertical	77100	78900

        #beamsize
        #wavelength_set = 13.5
        #aperture5 = 7
        # focus at PMMA
        
        (125700,128000,'1a#3',0.6,1,7.0,1.5,'none'), #	13:37	13:40		1a	50	vertical	125700	127500
        (128350,130450,'1c#2',0.6,1,7.0,1.5,'none'), #	13:41	13:44		1c	50	horizontal	128100	129900
        
        (142700,144600,'2a#3',0.6,1,7.0,1.5,'none'), #	14:05	14:08		2a	107	vertical	142500	144300
        (138200,140100,'2c#2',0.6,1,7.0,1.5,'none'), #	13:57	14:01		2c	107	horizontal	137700	140100
        
        (149050,151000,'3a#3',0.6,1,7.0,1.5,'none'), #	14:16	14:19		3a	215	vertical	149100	150900
        (155800,157500,'3c#2',0.6,1,7.0,1.5,'none'), #	14:27	14:30		3c	215	horizontal	155700	157500
        
        (168700,170200,'4a#3',0.6,1,7.0,1.5,'none'), #	14:49	14:52		4a	322	vertical	168900	170700
        (190200,191400,'4a#4',0.6,1,7.0,1.5,'none'), #	15:24	15:26		4a	322	vertical	189900	191100
        (164250,166200,'4c#2',0.6,1,7.0,1.5,'none'), #	14:41	14:44		4c	322	horizontal	164100	165900
        
        (171600,174400,'5a#03',0.6,1,7.0,1.5,'none'), #	14:53	14:58		5a	445	vertical	171300	174300
        (177400,179800,'5c#2',0.6,1,7.0,1.5,'none'), #	15:03	15:06		5c	445	horizontal	177300	179100
        
        (123300,125100,'1b#3',0.6,1,7.0,1.5,'none'), #	13:33	13:36		1b	707	vertical	123300	125100
        (130600,132000,'1d#1',0.6,1,7.0,1.5,'none'), #	13:45	13:47		1d	707	horizontal	130500	131700
        (132800,134600,'1d#2',0.6,1,7.0,1.5,'none'), #	13:48	13:51		1d	707	horizontal	132300	134100
       
        (140400,142400,'2b#3',0.6,1,7.0,1.5,'none'), #	14:01	14:04		2b	890	vertical	140100	141900
        (202000,202400,'2b#4',0.6,1,7.0,1.5,'none'), #	15:44	15:44		2b	890	vertical	201900	201900
        (135900,137700,'2d#2',0.6,1,7.0,1.5,'none'), #	13:54	13:57		2d	890	horizontal	135900	137700
        
        (153350,155400,'3b#3',0.6,1,7.0,1.5,'none'), #	14:23	14:26		3b	1047	vertical	153300	155100
        (157950,160500,'3d#2',0.6,1,7.0,1.5,'none'), #	14:30	14:34		3d	1047	horizontal	157500	159900
        
        #(204300,204700,'4b',0.8,1), #	15:48	15:48		4b	1335	vertical	204300	204300 # not working
        (166800,168300,'4b#3',0.6,1,7.0,1.5,'none'), #	14:45	14:48		4b	1335	vertical	166500	168300
        (161950,163700,'4d#2',0.6,1,7.0,1.5,'none'), #	14:37	14:40		4d	1335	horizontal	161700	163500
        
        (176100,177100,'5b#3',0.6,1,7.0,1.5,'none'), #	15:00	15:02		5b	1570	vertical	175500	176700
        (180000,181100,'5d#2',0.6,1,7.0,1.5,'none'), #	15:07	15:09		5d	1570	horizontal	179700	180900
        
       
        # beam size shrinking
        (205100,206000,'4a#3',0.6,1,7.0,3.0,'none'), #	15:49	15:50		4a	445	vertical	204900	205500
        (209700,212700,'5a#2',0.6,1,7.0,1.5,'none'), #	15:57	16:01		5a	445	vertical	209700	212100 # not working?
        (229200,231000,'5a#3',0.6,1,7.0,1.5,'none'), #	16:28	16:32		5a	445	vertical	228300	230700
        (231500,233800,'5a#4',0.6,1,7.0,1.5,'none'), #	16:33	16:37		5a	445	vertical	231300	233700
        (234300,234900,'5a#5',0.6,1,7.0,1.5,'none'), #	16:38	16:39		5a	445	vertical	234300	234900
        (236100,237900,'5a#6',0.6,1,7.0,1.5,'none'), #	16:41	16:44		5a	445	vertical	236100	237900
        (238500,239700,'5a#7',0.6,1,7.0,1.5,'none'), #	16:45	16:47		5a	445	vertical	238500	239700
        (240900,243300,'5a#8',0.6,1,7.0,1.5,'none'), #	16:49	16:53		5a	445	vertical	240900	243300
        (245300,246900,'5a#9',0.6,1,7.0,1.5,'none'), #	16:56	16:59		5a	445	vertical	245100	246900
        
        
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		background 1d 1c	17700	20100 KAOS=1.5mm
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700 KAOS=PMMA
        (101700,103600,'bg2ab#2',0.6,1,8.8,3.0,'none'), #	12:57	13:01		backgroung			101700	104100 KAOS=1.5mm
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700 KAOS=PMMA
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none') #	12:08	12:11		background			72300	7410 KAOS=PMMA        
]
}
dataset_image_args = collections.OrderedDict(sorted(dataset_image_args_d.items(), key=lambda t: t[0]))

dataset_args = dataset_image_args

##  Definitions of combinations of images and corresponding backgrounds

In [10]:
settings_d = {
    
     # 2017-11-24T1309
    #wavelength_set = 13.5
    # beamsize of PMMA focus
    #aperture5 = ?

   '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0050um (1a#1) ap5=7.0 ap7=50.0 (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T1309.h5',
        (105800,107000,'1a#1',0.6,1,7.0,50.0,'none'), #     20395    1a    50    vertical 
    ],
   '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0050um (1a#2) ap5=7.0 ap7=50.0 (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T1309.h5',
        (107500,112000,'1a#2',0.6,1,7.0,50.0,'none'), #    1a 50
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0050um (1a#3) ap5=7.0 ap7=50.0 (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T1309.h5',
        (177300,178500,'1a#3',0.6,1,7.0,50.0,'none'), # 1a 50
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0050um (1c) ap5=7.0 ap7=50.0 (bg1cd)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (88400,90000,'bg1cd',0.6,1,7.0,50.0,'none'), #    15:36:00    15:39:00    20395    bg1cd           
        'FLASH2_USER1-2017-11-24T1309.h5',
        (91500,94400,'1c',0.6,1,7.0,50.0,'none'), # 15:41:00    15:46:00    20395    1c    50    horizontal 91000    94000
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0107um (2a) ap5=7.0 ap7=50.0 (bg2cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T1309.h5',
        (56000,59000,'2a',0.6,1,7.0,50.0,'none'), # 14:41:00    14:48:00    20395    2a    107    vertical    55000 59200
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0107um (2c) ap5=7.0 ap7=50.0 (bg2cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T1309.h5',
        (67200,70500,'2c',0.6,1,7.0,50.0,'none'), # 15:00:00    15:07:00    20395    2c    107    horizontal 66400    70600
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0215um (3a#1) ap5=7.0 ap7=50.0 (bg3cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T1309.h5',
        (0,5000,'3a#1',0.6,1,7.0,50.0,'none'),     # 13:09:20    13:17:00    20395    3a    215    vertical    0 4600
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0215um (3a#2) ap5=7.0 ap7=50.0 (bg3cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T1309.h5',
        (46400,48400,'3a#2',0.6,1,7.0,50.0,'none'), # 14:26:00    14:30:00    20395    3a    215    vertical    46000 48400
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0215um (3c) ap5=7.0 ap7=50.0 (bg3cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T1309.h5',
        (28000,30500,'3c',0.6,1,7.0,50.0,'none'), # 13:54:00    13:59:00    20395    3c    215    horizontal 26800    29800
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0322um (4a) ap5=7.0 ap7=50.0 (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T1309.h5',
        (115500,118000,'4a',0.6,1,7.0,50.0,'none'), # 4a 322 vertical
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0322um (4c#1) ap5=7.0 ap7=50.0 (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T1309.h5',
        (132500,136500,'4c#1',0.6,1,7.0,50.0,'none'), # 4c 322 horizontal
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0322um (4c#2) ap5=7.0 ap7=50.0 (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T1309.h5',
        (139700,140400,'4c#2',0.6,1,7.0,50.0,'none'), # 20395    4c    322    vertical
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0445um (5a) ap5=7.0 ap7=50.0 (bg5ab)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400         
        'FLASH2_USER1-2017-11-24T1309.h5',
        (184000,187000,'5a',0.6,1,7.0,50.0,'none'), # 5a 445
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0445um (5c#1) ap5=7.0 ap7=50.0 (bg5ab)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400           
        'FLASH2_USER1-2017-11-24T1309.h5',
        (153000,156000,'5c#1',0.6,1,7.0,50.0,'none'), # 5c 445
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0445um (5c#2) ap5=7.0 ap7=50.0 (bg5ab)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400           
        'FLASH2_USER1-2017-11-24T1309.h5',
        (197500,201500,'5c#2',0.6,1,7.0,50.0,'none'), # 18:39:00    18:44:00    20395    5c    445    horizontal 197800    200800
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0707um (1b) ap5=7.0 ap7=50.0 (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600           
        'FLASH2_USER1-2017-11-24T1309.h5',
        (99300,103900,'1b',0.6,1,7.0,50.0,'none'), # 16:02:00    16:15:00    20395    1b    707    vertical
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0707um (1d) ap5=7.0 ap7=50.0 (bg1cd)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (88400,90000,'bg1cd',0.6,1,7.0,50.0,'none'), #    15:36:00    15:39:00    20395    bg1cd           
        'FLASH2_USER1-2017-11-24T1309.h5',
        (83000,87400,'1d',0.6,1,7.0,50.0,'none'), # 15:29:00    15:35:00    20395    1d    707    horizontal 83800    87400
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0890um (2b) ap5=7.0 ap7=50.0 (bg2cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T1309.h5',
        (60400,65300,'2b',0.6,1,7.0,50.0,'none'), # 14:50:00    15:57:00    20395    2b    890    vertical    60400 10060
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0890um (2d) ap5=7.0 ap7=50.0 (bg2cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T1309.h5',
        (74500,79200,'2d',0.6,1,7.0,50.0,'none'), # 15:13:00    15:21:00    20395    2d    890    horizontal 74200    79000
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 1047um (3b) ap5=7.0 ap7=50.0 (bg3cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T1309.h5',
        (6600,10000,'3b#1',0.6,1,7.0,50.0,'none'), # 13:20:00    13:25:00    20395    3b    1047    vertical    6400 9400
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 1047um (3b) ap5=7.0 ap7=50.0 (bg3cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T1309.h5',
        (41500,44800,'3b#2',0.6,1,7.0,50.0,'none'), # 14:18:00    14:23:00    20395    3b    1047    vertical 41200    44200
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 1047um (3d) ap5=7.0 ap7=50.0 (bg3cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T1309.h5',
        (36500,40000,'3d',0.6,1,7.0,50.0,'none'), # 14:10:00    14:15:00    20395    3d    1047    horizontal 36400    39400
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 1335um (4b) ap5=7.0 ap7=50.0 (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T1309.h5',
        (127300,130000,'4b',0.6,1,7.0,50.0,'none'), # 4b 1335 vertical
    ],


   # backgrounds
    
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0000um (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600           
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600           
    ],    
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0000um (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (88400,90000,'bg1cd',0.6,1,7.0,50.0,'none'), #    15:36:00    15:39:00    20395    bg1cd           
        'FLASH2_USER1-2017-11-24T1309.h5',
        (88400,90000,'bg1cd',0.6,1,7.0,50.0,'none'), #    15:36:00    15:39:00    20395    bg1cd           
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0000um (bg2cd-chopper-open) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
    ],    
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0000um (bg3cd-chopper-open) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0000um (bg4ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0000um (bg5ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400           
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400           
    ],
    
    
    
    

    
    # 2017-11-24T2020
    #wavelength_set = 13.5
    # beamsize 1.5mm FWHM 
    #aperture5 = ?

    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0050um (1a) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (60000,63000,'1a',0.6,1,7.0,50.0,'none'), # 21:59:00 22:05:00    20428    1a    50        59200    62800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0050um (1c) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (88400,90000,'bg1cd',0.6,1,7.0,50.0,'none'), #    15:36:00    15:39:00    20395    bg1cd           
        'FLASH2_USER1-2017-11-24T2020.h5',
        (55500,58000,'1c',0.6,1,7.0,50.0,'none'), # 21:52:00 21:57:00    20428    1c    50        55000    58000
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0050um (1c) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (55500,58000,'1c',0.6,1,7.0,50.0,'none'), # 21:52:00 21:57:00    20428    1c    50        55000    58000
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0107um (2a) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg2cd-chopper-open) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (35200,38800,'2a',0.6,1,7.0,50.0,'none'), # 21:19:00 21:25:00    20428    2a    107    vertical    35200 38800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0107um (2a) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (35200,38800,'2a',0.6,1,7.0,50.0,'none'), # 21:19:00 21:25:00    20428    2a    107    vertical    35200 38800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0107um (2a) ap5=7.0 ap7=50.0 (bg2bc) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (27000,27600,'bg2bc',0.6,1,7.0,50.0,'none'), #    20:56:00    21:01:00    20428    background 2b to 2c            21400    24400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (35200,38800,'2a',0.6,1,7.0,50.0,'none'), # 21:19:00 21:25:00    20428    2a    107    vertical    35200 38800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0107um (2c) ap5=7.0 ap7=50.0 (bg2cd-chopper-open_of1309)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (41600,44800,'2c',0.6,1,7.0,50.0,'none'), # 21:29:00 21:35:00    20428    2c    107        41200    44800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0107um (2c) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (41600,44800,'2c',0.6,1,7.0,50.0,'none'), # 21:29:00 21:35:00    20428    2c    107        41200    44800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0107um (2c) ap5=7.0 ap7=50.0 (bg2bc)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (27000,27600,'bg2bc',0.6,1,7.0,50.0,'none'), #    20:56:00    21:01:00    20428    background 2b to 2c            21400    24400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (41600,44800,'2c',0.6,1,7.0,50.0,'none'), # 21:29:00 21:35:00    20428    2c    107        41200    44800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0215um (3a) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg3cChopperOpen)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T2020.h5',
        (71200,74200,'3a',0.6,1,7.0,50.0,'none'), # 22:18:00 22:24:00    20428    3a    215    vertical    70600 74200
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0215um (3a) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (71200,74200,'3a',0.6,1,7.0,50.0,'none'), # 22:18:00 22:24:00    20428    3a    215    vertical    70600 74200
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0215um (3a) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T2020.h5',
        (71200,74200,'3a',0.6,1,7.0,50.0,'none'), # 22:18:00 22:24:00    20428    3a    215    vertical    70600 74200
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0215um (3c) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg3cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T2020.h5',
        (79600,82000,'3c',0.6,1,7.0,50.0,'none'), # 22:32:00 22:37:00    20428    3c    215    horizontal    79000 82000
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0215um (3c) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (79600,82000,'3c',0.6,1,7.0,50.0,'none'), # 22:32:00 22:37:00    20428    3c    215    horizontal    79000 82000
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0215um (3c) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T2020.h5',
        (79600,82000,'3c',0.6,1,7.0,50.0,'none'), # 22:32:00 22:37:00    20428    3c    215    horizontal    79000 82000
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0322um (4a) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T2020.h5',
        (88000,91600,'4a',0.6,1,7.0,50.0,'none'), # 22:47:00 22:53:00    20428    4a    322        88000    91600
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0322um (4c) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T2020.h5',
        (106600,110500,'4c',0.6,1,7.0,50.0,'none'), # 23:18:00 23:24:00  20428    4c    322    horizontal    106600 110200   
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0445um (5a) ap5=7.0 ap7=50.0 (bg5ab)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (116800,120000,'5a',0.6,1,7.0,50.0,'none'), # 23:35:00 23:40:00  20428    5a    445    vertical    116800 119800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0445um (5c) ap5=7.0 ap7=50.0 (bg5ab)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (126800,129200,'5c',0.6,1,7.0,50.0,'none'), # 23:51:00 23:55:00  20428    5c    445    horizontal    126400 128800
    ],  
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0707um (1b) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (63700,67000,'1b',0.6,1,7.0,50.0,'none'), # 22:06:00 22:12:00    20428    1b    707        63400    67000
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0707um (1d) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1c)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (88400,90000,'bg1cd',0.6,1,7.0,50.0,'none'), #    15:36:00    15:39:00    20395    bg1cd           
        'FLASH2_USER1-2017-11-24T2020.h5',
        (52500,54400,'1d',0.6,1,7.0,50.0,'none'), # 21:45:00 21:51:00    20428    1d    707    horizontal    50800 54400
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0890um (2b#1) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg2cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (1000,2500,'2b#1',0.6,1,7.0,50.0,'none'), #20:20:20    20:24:00 20428    2b    890        0    2200
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0890um (2b#1) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (1000,2500,'2b#1',0.6,1,7.0,50.0,'none'), #20:20:20    20:24:00 20428    2b    890        0    2200
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0890um (2b#2) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg2cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (22000,25000,'2b#2',0.6,1,7.0,50.0,'none'), # 20:45:00 20:47:00    20428    2b    890        14800    16000
    ],
    '2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0890um (2b#1) ap5=7.0 ap7=50.0 2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm (bg2bc)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (27000,27600,'bg2bc',0.6,1,7.0,50.0,'none'), #    20:56:00    21:01:00    20428    background 2b to 2c            21400    24400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (1000,2500,'2b#1',0.6,1,7.0,50.0,'none'), #20:20:20    20:24:00 20428    2b    890        0    2200
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0890um (2b#2) ap5=7.0 ap7=50.0 (bg2bc)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (27000,27600,'bg2bc',0.6,1,7.0,50.0,'none'), #    20:56:00    21:01:00    20428    background 2b to 2c            21400    24400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (22000,25000,'2b#2',0.6,1,7.0,50.0,'none'), # 20:45:00 20:47:00    20428    2b    890        14800    16000
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0890um (2b#2) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (22000,25000,'2b#2',0.6,1,7.0,50.0,'none'), # 20:45:00 20:47:00    20428    2b    890        14800    16000
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0707um (1d#2) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg2cChopperOpen)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (71200,72400,'bg2cd-chopper-open',0.6,1,7.0,50.0,'none'), #    15:07:00    15:10:00    20395    bg2cd            70600    72400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (51300,52300,'1d#2',0.6,1,7.0,50.0,'none'), # 21:37:00 21:43:00    20428    2d    890        46000    49600
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0707um (1d#2) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg1ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (95000,98200,'bg1ab',0.6,1,7.0,50.0,'none'), # 15:54:00    16:02:00    20395    1b    707    vertical    98800 103600
        'FLASH2_USER1-2017-11-24T2020.h5',
        (51300,52300,'1d#2',0.6,1,7.0,50.0,'none'), # 21:37:00 21:43:00    20428    2d    890        46000    49600
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 1047um (3b) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg3cChopperOpen)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T2020.h5',
        (74800,77800,'3b',0.6,1,7.0,50.0,'none'), # 22:25:00 22:30:00    20428    3b    1047    vertical    74800 77800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 1047um (3b) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg4a)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T2020.h5',
        (74800,77800,'3b',0.6,1,7.0,50.0,'none'), # 22:25:00 22:30:00    20428    3b    1047    vertical    74800 77800
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 1047um (3d) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg3cd-chopper-open)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (32200,34000,'bg3cd-chopper-open',0.6,1,7.0,50.0,'none'), #    14:03:00    14:06:00    20395    bg3cd        88000    89800
        'FLASH2_USER1-2017-11-24T2020.h5',
        (83200,85600,'3d',0.6,1,7.0,50.0,'none'), # 22:38:00 22:43:00    20428    3d    1047        82600    85600
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 1047um (3d) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T2020.h5',
        (83200,85600,'3d',0.6,1,7.0,50.0,'none'), # 22:38:00 22:43:00    20428    3d    1047        82600    85600
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 1335um (4b) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg4ab)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (119400,122000,'bg4ab',0.6,1,7.0,50.0,'none'), #            
        'FLASH2_USER1-2017-11-24T2020.h5',
        (102800,105500,'4b',0.6,1,7.0,50.0,'none'), # 23:11:00 23:16:00    20428    4b    1335    vertical    102400 105400
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 1335um (4d) ap5=7.0 ap7=50.0 2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA (bg4cd)': [
        'FLASH2_USER1-2017-11-24T1309.h5',
        (123000,126200,'bg4cd',0.6,1,7.0,50.0,'none'), # 17:05:00    17:11:00    20395    4d    1335    horizontal 141400    145000 
        'FLASH2_USER1-2017-11-24T2020.h5',
        (111500,115500,'4d',0.6,1,7.0,50.0,'none'), # 23:25:00 23:32:00    20428    4d    1335    horizontal 110800    115000
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 1570um (5b) ap5=7.0 ap7=50.0 (bg5ab)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (124000,125400,'5b',0.8,1,7.0,50.0,'none'), # 23:47:00 23:49:00    20428    5b    1570    vertical    124000 125200
    ],
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 1570um (5d) ap5=7.0 ap7=50.0 (bg5ab)': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (129700,130800,'5d',0.8,1,7.0,50.0,'none'), # 23:56:00 23:59:00    20428    5d    1570    horizontal    129400 131200
    ],
    
    
    # backgrounds
    
    
    
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0000um (bg2bc) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (27000,27600,'bg2bc',0.6,1,7.0,50.0,'none'), #    20:56:00    21:01:00    20428    background 2b to 2c            21400    24400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (27000,27600,'bg2bc',0.6,1,7.0,50.0,'none'), #    20:56:00    21:01:00    20428    background 2b to 2c            21400    24400
    ],    
    '2017-11-24T2020 13.5nm 68uJ 7Und. KOAS=1.5mm 0000um (bg5ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400
        'FLASH2_USER1-2017-11-24T2020.h5',
        (120400,123000,'bg5ab',0.6,1,7.0,50.0,'none'), # 23:41:00    23:46:00    20428    bg5a 5b            120400    123400
    ],
    
    
    
    
    
     # 2017-11-25T0141
    #wavelength_set = 18.0
    # beamsize ?
    #aperture5 = ?

    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0050um (1a) ap5=7.0 ap7=50.0 (bg1ab)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-25T0141.h5',
        (33500,37000,'1a',0.6,1,7.0,50.0,'none'), #	2:37:00	02:42:00	20441	1a	50		33500	36500
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0050um (1c) ap5=7.0 ap7=50.0 2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA (bg1ab)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-25T0141.h5',
        (26400,29500,'1c',0.6,1,7.0,50.0,'none'), #	2:25:00	02:30:00	20441	1c	50		26300	29300
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0107um (2a) ap5=7.0 ap7=50.0 (bg2ab)': [
        'FLASH2_USER1-2017-11-25T0141.h5',
        (6500,9500,'bg2ab',0.6,1,7.0,50.0,'none'), #	1:52:00	01:57:00	20441	bg2a 2b YAG+FEL			6500	9500
        'FLASH2_USER1-2017-11-25T0141.h5',
        (10200,13500,'2a',0.6,1,7.0,50.0,'none'), #	1:57:00	02:04:00	20441	2a	107	vertical	9500	13700
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0107um (2a) ap5=7.0 ap7=50.0 2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA (bg2ab)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (129500,131500,'bg2ab',0.6,1,7.0,50.0,'none'), #	22:52:00	22:55:00	20461	bg2a 2b			129500	131300
        'FLASH2_USER1-2017-11-25T0141.h5',
        (10200,13500,'2a',0.6,1,7.0,50.0,'none'), #	1:57:00	02:04:00	20441	2a	107	vertical	9500	13700
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0107um (2c) ap5=7.0 ap7=50.0 (bg2ab)': [
        'FLASH2_USER1-2017-11-25T0141.h5',
        (6500,9500,'bg2ab',0.6,1,7.0,50.0,'none'), #	1:52:00	01:57:00	20441	bg2a 2b YAG+FEL			6500	9500
        'FLASH2_USER1-2017-11-25T0141.h5',
        (14500,17500,'2c',0.6,1,7.0,50.0,'none'), #	2:05:00	02:10:00	20441	2c	107		14300	17300
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0215um (3a#1) ap5=7.0 ap7=50.0 (bg3ab#1)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-25T0141.h5',
        (39100,42300,'3a#1',0.6,1,7.0,50.0,'none'), #	2:46:00	02:52:00	20441	3a	215	vertical	58100	61100
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0215um (3a#1) ap5=7.0 ap7=50.0 (bg3ab#2)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-25T0141.h5',
        (39100,42300,'3a#1',0.6,1,7.0,50.0,'none'), #	2:46:00	02:52:00	20441	3a	215	vertical	58100	61100
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0215um (3a#2) ap5=7.0 ap7=50.0 (bg3a#1)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-25T0141.h5',
        (46200,48600,'3a#2',0.6,1,7.0,50.0,'none'), #	2:57:00	03:02:00	20441	3a	215	vertical	58100	61100
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0215um (3a#2) ap5=7.0 ap7=50.0 (bg3ab#2)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-25T0141.h5',
        (46200,48600,'3a#2',0.6,1,7.0,50.0,'none'), #	2:57:00	03:02:00	20441	3a	215	vertical	58100	61100
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0215um (3a#3) ap5=7.0 ap7=50.0 (bg3ab#1)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-25T0141.h5',
        (58700,61000,'3a#3',0.6,1,7.0,50.0,'none'), #	3:18:00	03:23:00	20441	3a	215	vertical	58100	61100
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0215um (3a#3) ap5=7.0 ap7=50.0 (bg3a#2)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-25T0141.h5',
        (58700,61000,'3a#3',0.6,1,7.0,50.0,'none'), #	3:18:00	03:23:00	20441	3a	215	vertical	58100	61100
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0322um (4a) ap5=7.0 ap7=50.0 (bg4cd)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-25T0141.h5',
        (65700,68700,'4a',0.6,1,7.0,50.0,'none'), #	3:30:00	03:35:00	20441	4a	322		65300	68300
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0322um (4c) ap5=7.0 ap7=50.0 (bg4cd)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-25T0141.h5',
        (73100,76200,'4c',0.6,1,7.0,50.0,'none'), #	3:43:00	03:48:00	20441	4c	322		73100	76100
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0445um (5a) ap5=7.0 ap7=50.0 (bg5cd)': [
        'FLASH2_USER1-2017-11-25T0141.h5',
        (91100,92300,'bg5cd',0.6,1,7.0,50.0,'none'), #	4:13:00	04:15:00	20441	bg5c 5d			91100	92300
        'FLASH2_USER1-2017-11-25T0141.h5',
        (83800,86300,'5a',0.6,1,7.0,50.0,'none'), #	4:00:00	04:05:00	20441	5a	445		83300	86300
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0445um (5c) ap5=7.0 ap7=50.0 (bg5cd)': [
        'FLASH2_USER1-2017-11-25T0141.h5',
        (91100,92300,'bg5cd',0.6,1,7.0,50.0,'none'), #	4:13:00	04:15:00	20441	bg5c 5d			91100	92300
        'FLASH2_USER1-2017-11-25T0141.h5',
        (87700,90500,'5c',0.6,1,7.0,50.0,'none'), #	4:07:00	04:12:00	20441	5c	455		87500	90500
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0707um (1b) ap5=7.0 ap7=50.0 (bg1ab)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-25T0141.h5',
        (30500,33200,'1b',0.6,1,7.0,50.0,'none'), #	2:31:00	02:36:00	20441	1b	707		29900	32900
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0707um (1d) ap5=7.0 ap7=50.0 (bg1ab)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-25T0141.h5',
        (22900,26000,'1d',0.6,1,7.0,50.0,'none'), #	2:19:00	02:24:00	20441	1d	707		22700	25700
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0890um (2b) ap5=7.0 ap7=50.0 (bg2ab)': [
        'FLASH2_USER1-2017-11-25T0141.h5',
        (6500,9500,'bg2ab',0.6,1,7.0,50.0,'none'), #	1:52:00	01:57:00	20441	bg2a 2b YAG+FEL			6500	9500
        'FLASH2_USER1-2017-11-25T0141.h5',
        (500,4500,'2b',0.6,1,7.0,50.0,'none'),   #	1:41:10	01:48:00	20441	2b	890		0	4100
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0890um (2d) ap5=7.0 ap7=50.0 (bg2ab)': [
        'FLASH2_USER1-2017-11-25T0141.h5',
        (6500,9500,'bg2ab',0.6,1,7.0,50.0,'none'), #	1:52:00	01:57:00	20441	bg2a 2b YAG+FEL			6500	9500
        'FLASH2_USER1-2017-11-25T0141.h5',
        (18200,21100,'2d',0.6,1,7.0,50.0,'none'), #	2:11:00	02:16:00	20441	2d	890	horizontal	17900	20900
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 1047um (3b#1) ap5=7.0 ap7=50.0 (bg3ab#1)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-25T0141.h5',
        (42800,46000,'3b#1',0.8,1,7.0,50.0,'none'), #	2:52:00	02:57:00	20441	3b	1047	vertical	45500	48500
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 1047um (3b#1) ap5=7.0 ap7=50.0 (bg3ab#2)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-25T0141.h5',
        (42800,46000,'3b#1',0.8,1,7.0,50.0,'none'), #	2:52:00	02:57:00	20441	3b	1047	vertical	45500	48500
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 1047um (3b#2) ap5=7.0 ap7=50.0 (bg3ab#1)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-25T0141.h5',
        (61700,63800,'3b#2',0.8,1,7.0,50.0,'none'), #	3:24:00	03:26:00	20441	3b	1047		61700	62900
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 1047um (3b#2) ap5=7.0 ap7=50.0 (bg3ab#2)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-25T0141.h5',
        (61700,63800,'3b#2',0.8,1,7.0,50.0,'none'), #	3:24:00	03:26:00	20441	3b	1047		61700	62900
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 1335um (4b) ap5=7.0 ap7=50.0 (bg4cd)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-25T0141.h5',
        (69200,72400,'4b',0.8,1,7.0,50.0,'none'), #	3:36:00	03:41:00	20441	4b	1335		68900	71900
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 1335um (4d#1) ap5=7.0 ap7=50.0 (bg4cd)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-25T0141.h5',
        (76800,77500,'4d#1',0.65,1,7.0,50.0,'none'), #	3:49:00	03:50:00	20441	4d	1335		76700	77300
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 1335um (4d#2) ap5=7.0 ap7=50.0 (bg4cd)': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-25T0141.h5',
        (78400,82000,'4d#2',0.71,1,7.0,50.0,'none'), #	3:51:00	03:57:00	20441	4d	1335		77900	81500
    ],
    
    # backgrounds
    

    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0000um (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0000um (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-25T0141.h5',
        (6500,9500,'bg2ab',0.6,1,7.0,50.0,'none'), #	1:52:00	01:57:00	20441	bg2a 2b YAG+FEL			6500	9500
        'FLASH2_USER1-2017-11-25T0141.h5',
        (6500,9500,'bg2ab',0.6,1,7.0,50.0,'none'), #	1:52:00	01:57:00	20441	bg2a 2b YAG+FEL			6500	9500
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0000um (bg3ab#1) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0000um (bg3ab#2) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0000um (bg4cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
    ],
    '2017-11-25T0141 18.0nm 46uJ 7Und. KOAS=1.5mm 0000um (bg5cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-25T0141.h5',
        (91100,92300,'bg5cd',0.6,1,7.0,50.0,'none'), #	4:13:00	04:15:00	20441	bg5c 5d			91100	92300
        'FLASH2_USER1-2017-11-25T0141.h5',
        (91100,92300,'bg5cd',0.6,1,7.0,50.0,'none'), #	4:13:00	04:15:00	20441	bg5c 5d			91100	92300
    ],
    
    
    
    
    
    # 2017-11-26T1916
    #wavelength_set = 18.0
    # beamsize of PMMA focus
    #aperture5 = ?

    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0050um (1a) ap5=7.0 ap7=50.0 (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (24300,27700,'1a',0.6,1,7.0,50.0,'none'), #	19:56:00	20:02:00	20461	1a	50	vertical	23900	27500
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0050um (1c) ap5=7.0 ap7=50.0 (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (34200,37300,'1c',0.6,1,7.0,50.0,'none'), #	20:12:00	20:17:00	20461	1c	50	horizontal	33500	36500
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0107um (2a) ap5=7.0 ap7=50.0 (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (129500,131500,'bg2ab',0.6,1,7.0,50.0,'none'), #	22:52:00	22:55:00	20461	bg2a 2b			129500	131300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (47000,50300,'2a',0.6,1,7.0,50.0,'none'), #	20:34:00	20:39:00	20461	2a	107	vertical	46700	49700
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0107um (2c) ap5=7.0 ap7=50.0 (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (129500,131500,'bg2ab',0.6,1,7.0,50.0,'none'), #	22:52:00	22:55:00	20461	bg2a 2b			129500	131300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (54200,57600,'2c',0.6,1,7.0,50.0,'none'), #	20:46:00	20:52:00	20461	2c	107	horizontal	53900	57500
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0215um (3a) ap5=7.0 ap7=50.0 (bg3a#1) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-26T1916.h5',
        (500,3500,'3a',0.6,1,7.0,50.0,'none'), #	19:17:00	19:22:00	20461	3a	215	vertical	500	3500
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0215um (3a) ap5=7.0 ap7=50.0 (bga3#1 ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-26T1916.h5',
        (500,3500,'3a',0.6,1,7.0,50.0,'none'), #	19:17:00	19:22:00	20461	3a	215	vertical	500	3500
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0215um (3c) ap5=7.0 ap7=50.0 (bg3a#2) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (11250,14700,'3c',0.6,1,7.0,50.0,'none'), #	19:34:00	19:39:00	20461	3c	215	horizontal	10700	13700
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0215um (3c) ap5=7.0 ap7=50.0 (bg3a#2) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (11250,14700,'3c',0.6,1,7.0,50.0,'none'), #	19:34:00	19:39:00	20461	3c	215	horizontal	10700	13700
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0322um (4a) ap5=7.0 ap7=50.0 (bg4cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-26T1916.h5',
        (64050,67400,'4a',0.6,1,7.0,50.0,'none'), #	21:03:00	21:08:00	20461	4a	322	vertical	64100	67100
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0322um (4c) ap5=7.0 ap7=50.0 (bg4cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-26T1916.h5',
        (71800,74500,'4c',0.6,1,7.0,50.0,'none'), #	21:15:00	21:20:00	20461	4c	322	horizontal	71300	74300
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0445um (5a#1) ap5=7.0 ap7=50.0 (bg5cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (116300,118800,'bg5cd',0.6,1,7.0,50.0,'none'), #	22:29:00	22:34:00	20461	5c	445	horizontal	115700	118700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (83100,85750,'5a#1',0.6,1,7.0,50.0,'none'), #21:34:00	21:39:00	20461	5a	445	vertical	82700	85700
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0445um (5a#2) ap5=7.0 ap7=50.0 (bg5cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (116300,118800,'bg5cd',0.6,1,7.0,50.0,'none'), #	22:29:00	22:34:00	20461	5c	445	horizontal	115700	118700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (86800,90300,'5a#2',0.6,1,7.0,50.0,'none'), #21:41:00	21:46:00	20461	5a	445	vertical	86900	89900
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0445um (5c) ap5=7.0 ap7=50.0 (bg5cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (116300,118800,'bg5cd',0.6,1,7.0,50.0,'none'), #	22:29:00	22:34:00	20461	5c	445	horizontal	115700	118700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (112500,115800,'5c',0.6,1,7.0,50.0,'none'), #22:23:00	22:29:00	20461	5c	445	horizontal	112100	115700
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0707um (1b) ap5=7.0 ap7=50.0 (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (29700,33100,'1b',0.6,1,7.0,50.0,'none'), #	20:06:00	20:11:00	20461	1b	707	vertical	29900	32900
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0707um (1d #1) ap5=7.0 ap7=50.0 (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (40100,40400,'1d#1',0.6,1,7.0,50.0,'none'), #20:22:00	20:23:00	20461	1d	707	horizontal	39500	40100
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0707um (1d #2) ap5=7.0 ap7=50.0 (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (41300,45400,'1d#2',0.6,1,7.0,50.0,'none'), #20:25:00	20:30:00	20461	1d	707	horizontal	41300	44300
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0890um (2b) ap5=7.0 ap7=50.0 (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (129500,131500,'bg2ab',0.6,1,7.0,50.0,'none'), #	22:52:00	22:55:00	20461	bg2a 2b			129500	131300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (50650,53700,'2b',0.6,1,7.0,50.0,'none'), #	20:40:00	20:45:00	20461	2b	890	vertical	50300	53300
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0890um (2d) ap5=7.0 ap7=50.0 (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (129500,131500,'bg2ab',0.6,1,7.0,50.0,'none'), #	22:52:00	22:55:00	20461	bg2a 2b			129500	131300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (58700,62000,'2d',0.6,1,7.0,50.0,'none'), #	20:54:00	20:59:00	20461	2d	890	horizontal	58700	61700
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 1047um (3b) ap5=7.0 ap7=50.0 (bg3ab#1) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-26T1916.h5',
        (7100,10400,'3b',0.6,1,7.0,50.0,'none'), #	19:28:00	19:33:00	20461	3b	1047vertical	7100	10100
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 1047um (3b) ap5=7.0 ap7=50.0 (bg3ab#2) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (7100,10400,'3b',0.6,1,7.0,50.0,'none'), #	19:28:00	19:33:00	20461	3b	1047vertical	7100	10100
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 1047um (3d) ap5=7.0 ap7=50.0 (bg3ab#1) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-26T1916.h5',
        (17000,20900,'3d',0.6,1,7.0,50.0,'none'), #	19:45:00	19:51:00	20461	3d	1047horizontal	17300	20900
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 1047um (3d) ap5=7.0 ap7=50.0 (bg3ab#2) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (17000,20900,'3d',0.6,1,7.0,50.0,'none'), #	19:45:00	19:51:00	20461	3d	1047horizontal	17300	20900
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 1335um (4b) ap5=7.0 ap7=50.0 (bg4cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-26T1916.h5',
        (68100,71100,'4b',0.6,1,7.0,50.0,'none'), #	21:09:00	21:14:00	20461	4b	1335vertical	67700	70700
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 1335um (4d) ap5=7.0 ap7=50.0 (bg4cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-26T1916.h5',
        (76950,81200,'4d',0.6,1,7.0,50.0,'none'), #	21:24:00	21:31:00	20461	4d	1335horizontal	76700	80900
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 1570um (5b) ap5=7.0 ap7=50.0 (bg5cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (116300,118800,'bg5cd',0.6,1,7.0,50.0,'none'), #	22:29:00	22:34:00	20461	5c	445	horizontal	115700	118700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (108050,111700,'5b',0.6,1,7.0,50.0,'none'), #22:16:00	22:22:00	20461	5b	1570vertical	107900	111500
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 1570um (5d) ap5=7.0 ap7=50.0 (bg5cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (116300,118800,'bg5cd',0.6,1,7.0,50.0,'none'), #	22:29:00	22:34:00	20461	5c	445	horizontal	115700	118700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (120100,122600,'5d',0.6,1,7.0,50.0,'none'), #22:35:00	22:40:00	20461	5d	1570horizontal	119300	122300
    ],
    
    #backgrounds
    
    
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0000um (bg1ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (28100,29100,'bg1ab',0.6,1,7.0,50.0,'none'), #	20:03:00	20:05:00	20461	1a	50	vertical	28100	29300
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0000um (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (129500,131500,'bg2ab',0.6,1,7.0,50.0,'none'), #	22:52:00	22:55:00	20461	bg2a 2b			129500	131300
        'FLASH2_USER1-2017-11-26T1916.h5',
        (129500,131500,'bg2ab',0.6,1,7.0,50.0,'none'), #	22:52:00	22:55:00	20461	bg2a 2b			129500	131300
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0000um (bg3a#1) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
        'FLASH2_USER1-2017-11-26T1916.h5',
        (4900,6500,'bg3ab#1',0.6,1,7.0,50.0,'none'), #	19:24:00	19:26:00	20461	background			4700	5900
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0000um (bg3a#2) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (124950,128100,'bg3ab#2',0.6,1,7.0,50.0,'none'), #	22:44:00	22:49:00	20461	bg3a 3b			124700	127700
    ],    
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0000um (bg4cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
        'FLASH2_USER1-2017-11-26T1916.h5',
        (74850,76400,'bg4cd',0.6,1,7.0,50.0,'none'), #	21:21:00	21:23:00	20461	bg4c 4d	74900	76100
    ],
    '2017-11-26T1916 18.0nm 70uJ 7Und. KOAS=PMMA 0000um (bg5cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T1916.h5',
        (116300,118800,'bg5cd',0.6,1,7.0,50.0,'none'), #	22:29:00	22:34:00	20461	5c	445	horizontal	115700	118700
        'FLASH2_USER1-2017-11-26T1916.h5',
        (116300,118800,'bg5cd',0.6,1,7.0,50.0,'none'), #	22:29:00	22:34:00	20461	5c	445	horizontal	115700	118700
    ],
    
    
    
    
    
    
    
    # 2017-11-26T23000
    #wavelength_set = 18.0
    # beamsize ?
    #aperture5 = ?

    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0050um (1a) ap5=7.0 ap7=1.5 (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-26T2300.h5',
        (54600,57950,'1a',0.6,1,7.0,1.5,'none'), #	27.11.17 0:31	00:36:00	20462	1a	50	vertical	54500	57500
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0050um (1c#1) ap5=7.0 ap7=50.0 (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-26T2300.h5',
        (33500,37050,'1c#1 ',0.6,1,7.0,50.0,'none'), #	26.11.17 23:55	00:01:00	20462	1c	50	horizontal	32900	36500
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0050um (1c#2) ap5=7.0 ap7=1.5 (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-26T2300.h5',
        (38800,42100,'1c#2',0.6,1,7.0,1.5,'none'), #	27.11.17 0:05	00:10:00	20462	1c	50	horizontal	38900	41900
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0107um (2a) ap5=7.0 ap7=50.0 (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (7200,10400,'bg2ab',0.6,1,7.0,50.0,'none'), #	26.11.17 23:12	23:17:00	20462	bg2a to2b			7100	10100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (11300,14800,'2a',0.6,1,7.0,50.0,'none'), #	26.11.17 23:19	23:24:00	20462	2a	107	vertical	11300	14300
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0107um (2c) ap5=7.0 ap7=50.0 (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (7200,10400,'bg2ab',0.6,1,7.0,50.0,'none'), #	26.11.17 23:12	23:17:00	20462	bg2a to2b			7100	10100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (19300,22850,'2c',0.6,1,7.0,50.0,'none'), #	26.11.17 23:32	23:38:00	20462	2c	107	horizontal	19100	22700
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0215um (3a) ap5=7.0 ap7=50.0 (bg3ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (67450,70900,'bg3ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:52	00:58:00	20462	bg3a 3b			67100	70700
        'FLASH2_USER1-2017-11-26T2300.h5',
        (64100,66800,'3a',0.6,1,7.0,1.5,'none'), #	27.11.17 0:46	00:51:00	20462	3a	215	vertical	63500	665000
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0215um (3c) ap5=7.0 ap7=1.5 (bg3ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (67450,70900,'bg3ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:52	00:58:00	20462	bg3a 3b			67100	70700
        'FLASH2_USER1-2017-11-26T2300.h5',
        (75400,79000,'3c',0.6,1,7.0,1.5,'none'), #	27.11.17 1:05	01:11:00	20462	3c	215	horizontal	74900	78500
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0322um (4a) ap5=7.0 ap7=1.5 (bg4ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (88200,91100,'bg4ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:27	01:32:00	20462	bg4ab			88100	91100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (84750,88000,'4a',0.6,1,7.0,1.5,'none'), #	27.11.17 1:21	01:26:00	20462	4a	322	vertical	84500	87500        
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0322um (4c) ap5=7.0 ap7=1.5 (bg4ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (88200,91100,'bg4ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:27	01:32:00	20462	bg4ab			88100	91100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (95300,97800,'4c',0.6,1,7.0,1.5,'none'), #	27.11.17 1:38	01:43:00	20462	4c	322	horizontal	94700	97700
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0445um (5a) ap5=7.0 ap7=1.5 (bg5ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (106600,110700,'bg5ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:57	02:04:00	20462	bg5ab			106100	110300
        'FLASH2_USER1-2017-11-26T2300.h5',
        (103200,106400,'5a',0.6,1,7.0,1.5,'none'), #27.11.17 1:52	01:57:00	20462	5a	445	vertical	103100	106100
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0445um (5c) ap5=7.0 ap7=1.5 (bg5ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (106600,110700,'bg5ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:57	02:04:00	20462	bg5ab			106100	110300
        'FLASH2_USER1-2017-11-26T2300.h5',
        (115400,118800,'5c',0.6,1,7.0,1.5,'none'), #27.11.17 2:12	02:17:00	20462	5c	445	horizontal	115100	118100  
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0707um (1b) ap5=7.0 ap7=1.5 (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-26T2300.h5',
        (46900,50750,'1b',0.6,1,7.0,1.5,'none'), #	27.11.17 0:18	00:24:00	20462	1b	707	vertical	46700	50300
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0707um (1d#1) ap5=7.0 ap7=50.0 (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-26T2300.h5',
        (28900,32700,'1d#1',0.6,1,7.0,50.0,'none'), #	26.11.17 23:48	23:54:00	20462	1d	707	horizontal	28700	32300
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0707um (1d#2) ap5=7.0 ap7=1.5 (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-26T2300.h5',
        (42700,45700,'1d#2',0.6,1,7.0,1.5,'none'), #	27.11.17 0:11	00:16:00	20462	1d	707	vertical	42500	45500
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0890um (2b) ap5=7.0 ap7=50.0 (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (7200,10400,'bg2ab',0.6,1,7.0,50.0,'none'), #	26.11.17 23:12	23:17:00	20462	bg2a to2b			7100	10100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (15500,18800,'2b',0.6,1,7.0,50.0,'none'), #	26.11.17 23:26	23:31:00	20462	2b	890	vertical	15500	18500
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0890um (2d) ap5=7.0 ap7=50.0 (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (7200,10400,'bg2ab',0.6,1,7.0,50.0,'none'), #	26.11.17 23:12	23:17:00	20462	bg2a to2b			7100	10100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (23800,27300,'2d',0.6,1,7.0,50.0,'none'), #	26.11.17 23:40	23:45:00	20462	2d	890	horizontal	23900	26900
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 1047um (3b) ap5=7.0 ap7=1.5 (bg3ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (67450,70900,'bg3ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:52	00:58:00	20462	bg3a 3b			67100	70700
        'FLASH2_USER1-2017-11-26T2300.h5',
        (71500,74700,'3b',0.8,1,7.0,1.5,'none'), #	27.11.17 0:58	01:04:00	20462	3b	1047vertical	70700	74300 | 5x dunkel pinhole bzw hable pixis bilder dazwischen
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 1047um (3d) ap5=7.0 ap7=1.5 (bg3ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (67450,70900,'bg3ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:52	00:58:00	20462	bg3a 3b			67100	70700
        'FLASH2_USER1-2017-11-26T2300.h5',
        (80000,81750,'3d',0.83,1,7.0,1.5,'none'), #27.11.17 1:12	01:15:00	20462	3d	1047horizontal	79100	80900
     ],
     '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 1335um (4b) ap5=7.0 ap7=50.0 (bg4ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (88200,91100,'bg4ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:27	01:32:00	20462	bg4ab			88100	91100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (91600,947500,'4b',0.8,1,7.0,1.5,'none'), # 27.11.17 1:32	01:37:00	20462	4b	1335vertical	91100	94100 | 1x rauslaufendes Pixis Bild
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 1335um (4d) ap5=7.0 ap7=1.5 (bg4ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (88200,91100,'bg4ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:27	01:32:00	20462	bg4ab			88100	91100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (98550,101650,'4d',0.83,1,7.0,1.5,'none'), # 27.11.17 1:44	01:49:00	20462	4d	1335horizontal	98300	101300 | 1x dunkel, 1x pixis rauslaufend(letztes bild)
    ],
 
    # backgrounds
    
    
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0000um (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0000um (bg2ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (7200,10400,'bg2ab',0.6,1,7.0,50.0,'none'), #	26.11.17 23:12	23:17:00	20462	bg2a to2b			7100	10100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (7200,10400,'bg2ab',0.6,1,7.0,50.0,'none'), #	26.11.17 23:12	23:17:00	20462	bg2a to2b			7100	10100
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0000um (bg3ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (67450,70900,'bg3ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:52	00:58:00	20462	bg3a 3b			67100	70700
        'FLASH2_USER1-2017-11-26T2300.h5',
        (67450,70900,'bg3ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:52	00:58:00	20462	bg3a 3b			67100	70700
     ],
     '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0000um (bg4ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (88200,91100,'bg4ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:27	01:32:00	20462	bg4ab			88100	91100
        'FLASH2_USER1-2017-11-26T2300.h5',
        (88200,91100,'bg4ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:27	01:32:00	20462	bg4ab			88100	91100
    ],
    '2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm 0000um (bg5ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (106600,110700,'bg5ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:57	02:04:00	20462	bg5ab			106100	110300
        'FLASH2_USER1-2017-11-26T2300.h5',
        (106600,110700,'bg5ab',0.6,1,7.0,1.5,'none'), #	27.11.17 1:57	02:04:00	20462	bg5ab			106100	110300
    ],
    
    
    # 2017-11-27T1101
    #wavelength_set = 8.0
    # beamsize ?
    #aperture5 = 7.0
    # energy 50uJ

    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0050um (1a-none) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (39125,41700,'1a',0.6,1,7.0,50.0,'none'), #	no filter 12:06:43	12:11:00	20469	1a	50	vertical	38700	41100
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0050um (1a-none) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (39125,41700,'1a',0.6,1,7.0,50.0,'none'), #	no filter 12:06:43	12:11:00	20469	1a	50	vertical	38700	41100
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0107um (2a#1-Nb405) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (33350,33500,'2a#1',0.6,1,7.0,50.0,'Nb405'), #	Nb 405nm 11:57:05	11:57:20	20469	2a	107	vertical	33300	33900
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0107um (2a#2-Nb197) ap5=7.0 ap7=50.0 (bg2ab-Nb197)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1101.h5',
        (34400,36600,'2a#2',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:58:50	12:02:30	20469	2a	107	vertical	33900	36300
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0107um (2c-Nb197) ap5=7.0 ap7=50.0 (bg2ab-Nb197)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1101.h5',
        (23700,24900,'2c',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:41:00	11:43:00	20469	2c	107	horizontal	18900	23700
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0215um (3a-Nb197) ap5=7.0 ap7=50.0 (bg2ab-Nb197)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1101.h5',
        (3300,6100,'3a',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:07:00	11:11:40	20469	3a	215	vertical	3300	5700
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0215um (3a-Nb197) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (3300,6100,'3a',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:07:00	11:11:40	20469	3a	215	vertical	3300	5700
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0215um (3c#1-none) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (6800,7400,'3c#1',0.6,1,7.0,50.0,'none'), #	no Filter 11:12:50	11:13:50	20469	3c	215	horizontal	6300	9900
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0215um (3c#2-Nb405) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (7700,9900,'3c#2',0.6,1,7.0,50.0,'Nb405'), #	Nb 405nm 11:14:20	11:18:00	20469	3c	215	horizontal	6300	9900
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0322um (4a-none) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (64500,67600,'4a',0.6,1,7.0,50.0,'none'), #	no filter 12:49:01	12:54:10	20469	4a	322	vertical	63900	67500
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0322um (4c-Nb197) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (57800,60400,'4c',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:37:50	12:42:10	20469	4c	322	horizontal	57300	60300
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0322um (4c-Nb197) ap5=7.0 ap7=50.0 (bg5ab-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
        'FLASH2_USER1-2017-11-27T1101.h5',
        (57800,60400,'4c',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:37:50	12:42:10	20469	4c	322	horizontal	57300	60300
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0445um (5a#1-Nb197) ap5=7.0 ap7=50.0 (bg5ab-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
        'FLASH2_USER1-2017-11-27T1101.h5',
        (70900,74100,'5a#1',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:59:41	13:05:01	20469	5a	445	vertical	71100	74100
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0445um (5a#2-none) ap5=7.0 ap7=50.0 (bg5ab-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
        'FLASH2_USER1-2017-11-27T1101.h5',
        (74700,77300,'5a#2',0.6,1,7.0,50.0,'none'), #	no filter 13:06:01	13:10:21	20469	5a	445	vertical	74700	77100
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0445um (5c-none) ap5=7.0 ap7=50.0 (bg5ab-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
        'FLASH2_USER1-2017-11-27T1101.h5',
        (83900,87400,'5c',0.6,1,7.0,50.0,'none'), #	no filter 13:21:21	13:27:11	20469	5c	445	horizontal	83700	86700
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0707um (1b-none) ap5=7.0 ap7=50.0 (bg2ab-Nb197)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1101.h5',
        (42200,44800,'1b',0.6,1,7.0,50.0,'none'), #	no filter 12:11:50	12:16:10	20469	1b	707	vertical	41700	44700
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0707um (1d-Nb197) ap5=7.0 ap7=50.0 (bg2ab-Nb197)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1101.h5',
        (49700,51700,'1d',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:24:20	12:27:40	20469	1d	707	horizontal	49500	51300
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0890um (2d#1-Nb197) ap5=7.0 ap7=50.0 (bg2ab-Nb197)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1101.h5',
        (18550,18850,'2d#1',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:32:25	11:32:55	20469	2d	890	horizontal	15900	18900
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0890um (2d#2-none) ap5=7.0 ap7=50.0 (bg2ab-Nb197)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1101.h5',
        (19250,21900,'2d#2',0.6,1,7.0,50.0,'none'), #	no filter 11:33:35	11:38:00	20469	2d	890	horizontal	18900	23700
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0890um (2b-none) ap5=7.0 ap7=50.0 (bg2ab-Nb197)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1101.h5',
        (27900,29600,'2b',0.6,1,7.0,50.0,'none'), #	no filter 11:48:00	11:50:50	20469	2b	890	vertical	27300	29700
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 1047um (3b-none) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (250,900,'3b',0.6,1,7.0,50.0,'none'), 	  #	no filter 11:01:55	11:03:00	20469	3b	1047	vertical	0	900
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 1047um (3d-Nb405) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (11150,13700,'3d',0.6,1,7.0,50.0,'Nb405'), #	Nb 405nm 11:20:05	11:24:20	20469	3d	1047	horizontal	11100	13500
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 1335um (4b#1-Nb197) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (60950,61200,'4b#1',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:43:06	12:43:31	20469	4b	1335	vertical	60900	61500
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 1335um (4b#2-none) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (61550,63700,'4b#2',0.6,1,7.0,50.0,'none'), #	no filter 12:44:06	12:47:41	20469	4b	1335	vertical	61500	63300
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 1335um (4d-Nb197) ap5=7.0 ap7=50.0 (bg3cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (54970,56600,'4d',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 12:33:07	12:35:50	20469	4d	1335	horizontal	54900	56700
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 1570um (5b-none) ap5=7.0 ap7=50.0 (bg5ab-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
        'FLASH2_USER1-2017-11-27T1101.h5',
        (80900,83300,'5b',0.6,1,7.0,50.0,'none'), #	no filter 13:16:21	13:20:21	20469	5b	1570	vertical	80700	83100
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 1570um (5d-none) ap5=7.0 ap7=50.0 (bg5cd-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (91100,93300,'bg5cd',0.6,1,7.0,50.0,'none'), #	no filter 13:33:21	13:37:01	20469	background			90900	93900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (87750,90200,'5d',0.6,1,7.0,50.0,'none'), #	no filter 13:27:46	13:31:51	20469	5d	1570	horizontal	87300	90300
    ],
    
    # backgrounds
    
    
    
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0000um (bg2ab-Nb197) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0000um (bg3cd-none) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (15200,15900,'bg3cd',0.6,1,7.0,50.0,'none'), #	no filter 11:26:50	11:28:00	20469	background			14700	15900
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0000um (bg5ab-none) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
        'FLASH2_USER1-2017-11-27T1101.h5',
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0000um (5c-none) ap5=7.0 ap7=50.0 (bg5ab-none)': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (77750,80250,'bg5ab',0.6,1,7.0,50.0,'none'), # no filter	13:11:06	13:15:16	20469	background			77700	80100
        'FLASH2_USER1-2017-11-27T1101.h5',
        (83900,87400,'5c',0.6,1,7.0,50.0,'none'), #	no filter 13:21:21	13:27:11	20469	5c	445	horizontal	83700	86700
    ],
    '2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm 0000um (bg5cd-none) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (91100,93300,'bg5cd',0.6,1,7.0,50.0,'none'), #	no filter 13:33:21	13:37:01	20469	background			90900	93900
        'FLASH2_USER1-2017-11-27T1101.h5',
        (91100,93300,'bg5cd',0.6,1,7.0,50.0,'none'), #	no filter 13:33:21	13:37:01	20469	background			90900	93900
    ],    
    
    
    
    # 2017-11-27T1529
    #wavelength_set = 8.0
    # beamsize of PMMA focus
    # aperture5 = ?
    # energy 50uJ

    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0050um (1a) ap5=7.0 ap7=50.0 (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (30300,31500,'bg1cd',0.6,1,7.0,50.0,'none'), #	16:20:00	16:22:00		bg1d 1c			30300	31500
        'FLASH2_USER1-2017-11-27T1529.h5',
        (40200,42500,'1a',0.6,1,7.0,50.0,'none'), #	16:36:00	16:42:00		1a	50	vertical	39900	43500
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0050um (1c) ap5=7.0 ap7=50.0 (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (30300,31500,'bg1cd',0.6,1,7.0,50.0,'none'), #	16:20:00	16:22:00		bg1d 1c			30300	31500
        'FLASH2_USER1-2017-11-27T1529.h5',
        (32800,35700,'1c',0.6,1,7.0,50.0,'none'), #	16:23:00	16:28:00		1c	50	horizontal	32100	35100
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0107um (2c#1) ap5=7.0 ap7=50.0 2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm (bg2ab-Nb197) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1529.h5',
        (20700,23100,'2c#1',0.6,1,7.0,50.0,'none'), #	16:04:00	16:08:00		2c	107	horizontal	20700	23100
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0215um (3a#1) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (11900,14000,'3a#1',0.6,1,7.0,50.0,'none'), #	15:49:00	15:53:00		3a	215	vertical	11700	14100
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0107um (2a#1) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (15800,17700,'2a#1',0.6,1,7.0,50.0,'none'), #	15:55:00	15:59:00		3a	215	vertical	15300	17700
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0215um (3c) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (0,2100,'3c',0.6,1,7.0,50.0,'none'), #	15:29:30	15:33:00	20476	3c	215	horizontal	0	2100
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0322um (4a) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (44000,46500,'4a',0.6,1,7.0,50.0,'none'), #	16:42:00	16:46:00		4a	322	vertical	43500	45900
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0322um (4c) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (47900,50500,'4c#1',0.6,1,7.0,50.0,'none'), #	16:49:00	16:54:00		4c	322	horizontal	47700	50700
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0322um (4c) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (51300,53400,'4c#2',0.6,1,7.0,50.0,'none'), #	16:55:00	16:58:00		4c	322	horizontal	51300	53100
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0445um (5a) ap5=7.0 ap7=50.0 (bg5ab) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (59200,60400,'bg5ab',0.6,1,7.0,50.0,'none'), #	17:08:00	17:10:00		Backgound 5a 5b			59100	60300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (56400,58600,'5a',0.6,1,7.0,50.0,'none'), #	17:03:00	17:07:00		5a	445	vertical	56100	58500
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0445um (5c) ap5=7.0 ap7=50.0 (bg5ab) ap5=7.0 ap7=50.0  ': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (59200,60400,'bg5ab',0.6,1,7.0,50.0,'none'),
        'FLASH2_USER1-2017-11-27T1529.h5',
        (61600,64500,'5c',0.6,1,7.0,50.0,'none'), #	17:12:00	17:17:00		5b	1570	vertical	61500	64500
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0707um (1b) ap5=7.0 ap7=50.0 (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (30300,31500,'bg1cd',0.6,1,7.0,50.0,'none'), #	16:20:00	16:22:00		bg1d 1c			30300	31500
        'FLASH2_USER1-2017-11-27T1529.h5',
        (36600,39500,'1b',0.6,1,7.0,50.0,'none'), #	16:30:00	16:35:00		1b	707	vertical	36300	39300
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0707um (1d) ap5=7.0 ap7=50.0 (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (30300,31500,'bg1cd',0.6,1,7.0,50.0,'none'), #	16:20:00	16:22:00		bg1d 1c			30300	31500
        'FLASH2_USER1-2017-11-27T1529.h5',
        (27300,29700,'1d',0.6,1,7.0,50.0,'none'), #	16:15:00	16:19:00		1d	707	horizontal	27300	29700
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0890um (2d) ap5=7.0 ap7=50.0 2017-11-27T1101 8.0nm 45uJ 12Und. KOAS=_mm (bg2ab-Nb197) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1101.h5',
        (30900,31600,'bg2ab',0.6,1,7.0,50.0,'Nb197'), #	Nb 197nm 11:53:00	11:54:10	20469	background			30300	31500
        'FLASH2_USER1-2017-11-27T1529.h5',
        (23800,25100,'2d',0.6,1,7.0,50.0,'none'), #	16:09:00	16:13:00		2d	890	horizontal	23700	26100
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 1047um (3b#1) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (9500,11000,'3b#1',0.9,1,7.0,50.0,'none'), #	15:44:00	15:48:00		3b	1047	vertical	8700	11100
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 1047um (3b#2) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (18300,20100,'3b#2',0.6,1,7.0,50.0,'none'), #	15:59:00	16:03:00		3b	1047	vertical	17700	20100
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 1047um (3d) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (6600,8500,'3d',0.95,1,7.0,50.0,'none'), #	15:40:00	15:44:00		3d	1047	horizontal	6300	8700
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 1335um (4d) ap5=7.0 ap7=50.0 (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (54000,54800,'4d',0.91,1,7.0,50.0,'none'), #	16:59:00	17:01:00		4d	1335	horizontal	53700	54900
    ],

    # backgrounds
    
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0000um (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (30300,31500,'bg1cd',0.6,1,7.0,50.0,'none'), #	16:20:00	16:22:00		bg1d 1c			30300	31500
        'FLASH2_USER1-2017-11-27T1529.h5',
        (30300,31500,'bg1cd',0.6,1,7.0,50.0,'none'), #	16:20:00	16:22:00		bg1d 1c			30300	31500
    ],
     '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0000um (bg3cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
        'FLASH2_USER1-2017-11-27T1529.h5',
        (4700,6300,'bg3cd',0.6,1,7.0,50.0,'none'), #	15:37:00	15:40:00		background			4500	6300
    ],
    '2017-11-27T1529 8.0nm 45uJ 12Und. KOAS=PMMA 0000um (bg5ab) ap5=7.0 ap7=50.0  ': [
        'FLASH2_USER1-2017-11-27T1529.h5',
        (59200,60400,'bg5ab',0.6,1,7.0,50.0,'none'),
        'FLASH2_USER1-2017-11-27T1529.h5',
        (59200,60400,'bg5ab',0.6,1,7.0,50.0,'none'),
        
    ],

    
    
    
     #wavelength_set = 13.5
        # beamsize 1.5mm
        #aperture5 = 7
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0050um (1a#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (26300,28100,'1a#1',0.6,1,7.0,50.0,'none'), #	10:51	10:54		1a	50	vertical	26100	27900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0050um (1c#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (20900,23500,'1c#1',0.6,1,7.0,50.0,'none'), #	10:42	10:46		1c	50	horizontal	20700	23100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0107um (2a#1) ap5=7.0 ap7=50.0.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (3500,6500,'2a#1',0.6,1,7.0,50.0,'none'), # 	10:13	10:18		2a	107	vertical
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0107um (2c#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (7000,9600,'2c#1',0.6,1,7.0,50.0,'none'), # 	10:19	10:23		2c	107	horizontal
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0215um (3a#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (36600,38700,'3a#1',0.6,1,7.0,50.0,'none'), #	11:08	11:12		3a	215	vertical	36300	38700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0215um (3c#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (42700,44800,'3c#1',0.6,1,7.0,50.0,'none'), #	11:18	11:22		3c	215	horizontal	42300	44700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0322um (4a#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
         (56100,58400,'4a#1',0.6,1,7.0,50.0,'none'), #	11:41	11:44		4a	322	vertical	56100	57900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0322um (4c#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (51300,53500,'4c#1',0.6,1,7.0,50.0,'none'), #	11:33	11:36		4c	322	horizontal	51300	53100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0445um (5a#01) ap5=7.0 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (61400,64100,'5a#01',0.6,1,7.0,3.0,'none'), #	11:50	11:53		5a	445	vertical	61500	63300
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0445um (5c#1) ap5=7.0 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (66600,68400,'5c#1',0.85,1,7.0,3.0,'none'), #11:58	12:01		5c	445	horizontal	66300	68100
    ],   
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0707um (1d#0) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (15000,17100,'1d#0',0.6,1,7.0,50.0,'none'), #	10:32	10:36		1d	707	horizontal	14700	17100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0707um (1b#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (23700,26000,'1b#1',0.6,1,7.0,50.0,'none'), #	10:46	10:50		1b	707	vertical	23100	25500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0890um (2b#0) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (0,3500,'2b#0',0.6,1,7.0,50.0,'none'),   # 	10:09	10:13		2b	890	vertical
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0890um (2d#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (10500,13000,'2d#1',0.6,1,7.0,50.0,'none'), #	10:25	10:28		2d	890	horizontal
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 1047um (3b#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (40300,42300,'3b#1',0.6,1,7.0,50.0,'none'), #	11:14	11:17		3b	1047	vertical	39900	41700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 1047um (3d#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (45300,47600,'3d#1',0.6,1,7.0,50.0,'none'), #	11:23	11:26		3d	1047	horizontal	45300	47100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 1335um (4d#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (48900,50900,'4d#1',0.6,1,7.0,50.0,'none'), #	11:29	11:32		4d	1335	vertical	48900	50700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 1335um (4b#1) ap5=7.0 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (53600,55700,'4b#1',0.6,1,7.0,50.0,'none'), #	11:37	11:40		4b	1335	vertical	53700	55500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 1570um (5b#1) ap5=7.0 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (64500,66200,'5b#1',0.9,1,7.0,3.0,'none'), #	11:54	11:57		5b	1570	vertical	63900	65700, 
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 1570um (5d#1) ap5=7.0 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (69100,70100,'5d#1',0.85,1,7.0,3.0,'none'), #	12:02	12:04		5d	1570	horizontal	68700	69900
    ],

    # backgrounds
    
    
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0000um (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0000um (bg2ab#1) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0000um (bg2ab#2) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (101700,103600,'bg2ab#2',0.6,1,8.8,3.0,'none'), #	12:57	13:01		backgroung			101700	104100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (101700,103600,'bg2ab#2',0.6,1,8.8,3.0,'none'), #	12:57	13:01		backgroung			101700	104100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0000um (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0000um (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
    ],
    
    
    
    
    #wavelength_set = 13.5
    # beamsize 1.5mm
    # aperture5=8.8
    # this should have been also different backgrounds, actually
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0050um (1a#2) ap5=8.8 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (111500,113500,'1a#2',0.6,1,8.8,3.0,'none'), #	13:13	16:16		1a	50	vertical	111300	113100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0107um (2a#2) ap5=8.8 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg2ab#2) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (101700,103600,'bg2ab#2',0.6,1,8.8,3.0,'none'), #	12:57	13:01		backgroung			101700	104100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (104500,106500,'2a#2',0.6,1,8.8,3.0,'none'), #	13:02	13:05		2a	107	vertical	104700	106500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0215um (3a#2) ap5=8.8 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (89000,91500,'3a#2',0.6,1,8.8,50.0,'none'), #	12:35	12:40		3a	215	vertical	88500	91500     
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0322um (4a#2) ap5=8.8 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (84700,87200,'4a#2',0.6,1,8.8,50.0,'none'), #	12:27	12:32		4a	322	vertical	83700	86700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0445um (5a#02) ap5=8.8 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (74900,77050,'5a#02',0.6,1,8.8,3.0,'none'), #	12:13	12:15		5a	445	vertical	75300	76500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0707um (1b#2) ap5=8.8 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=8.8 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (113700,115500,'1b#2',0.6,1,8.8,3.0,'none'), #	13:17	13:20		1b	707	vertical	113700	115500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0890um (2b#1) ap5=8.8 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg2ab#2) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (101700,103600,'bg2ab#2',0.6,1,8.8,3.0,'none'), #	12:57	13:01		backgroung			101700	104100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (95900,98100,'2b#1',0.6,1,8.8,50.0,'none'), #	12:47	12:50		2b	890	vertical	95700	97500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 0890um (2b#1) ap5=8.8 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (101700,103600,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	12:57	13:01		backgroung			101700	104100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (95900,98100,'2b#1',0.6,1,8.8,50.0,'none'), #	12:47	12:50		2b	890	vertical	95700	97500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 1047um (3b#2) ap5=8.8 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (91900,94100,'3b#2',0.6,1,8.8,50.0,'none'), #	12:40	12:44		3b	1047	vertical	91500	93900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 1335um (4b#2) ap5=8.8 ap7=50.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (82200,84000,'4b#2',0.6,1,8.8,50.0,'none'), #	12:24	12:27		4b	1335	vertical	81900	83700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm 1570um (5b#2) ap5=8.8 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (77500,79400,'5b#2',0.85,1,8.8,3.0,'none'), #	12:16	12:19		5b	1570	vertical	77100	78900
    ],
    
    
    
    
    
    #beamsize
    #wavelength_set = 13.5
    #aperture5 = 7
    # focus at PMMA

    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0050um (1a#3) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (125700,128000,'1a#3',0.6,1,7.0,1.5,'none'), #	13:37	13:40		1a	50	vertical	125700	127500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0050um (1c#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (128350,130450,'1c#2',0.6,1,7.0,1.5,'none'), #	13:41	13:44		1c	50	horizontal	128100	129900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0107um (2a#3) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (142700,144600,'2a#3',0.6,1,7.0,1.5,'none'), #	14:05	14:08		2a	107	vertical	142500	144300
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0107um (2c#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (138200,140100,'2c#2',0.6,1,7.0,1.5,'none'), #	13:57	14:01		2c	107	horizontal	137700	140100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0215um (3a#3) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (149050,151000,'3a#3',0.6,1,7.0,1.5,'none'), #	14:16	14:19		3a	215	vertical	149100	150900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0215um (3c#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (155800,157500,'3c#2',0.6,1,7.0,1.5,'none'), #	14:27	14:30		3c	215	horizontal	155700	157500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0322um (4a#3) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (168700,170200,'4a#3',0.6,1,7.0,1.5,'none'), #	14:49	14:52		4a	322	vertical	168900	170700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0322um (4a#4) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (190200,191400,'4a#4',0.6,1,7.0,1.5,'none'), #	15:24	15:26		4a	322	vertical	189900	191100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0322um (4c#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (164250,166200,'4c#2',0.6,1,7.0,1.5,'none'), #	14:41	14:44		4c	322	horizontal	164100	165900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0445um (5a#03) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (171600,174400,'5a#03',0.6,1,7.0,1.5,'none'), #	14:53	14:58		5a	445	vertical	171300	174300
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0445um (5c#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (177400,179800,'5c#2',0.6,1,7.0,1.5,'none'), #	15:03	15:06		5c	445	horizontal	177300	179100
    ],

    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0707um (1b) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (123300,125100,'1b#3',0.6,1,7.0,1.5,'none'), #	13:33	13:36		1b	707	vertical	123300	125100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0707um (1b) ap5=7.0 ap7=1.5 2017-11-26T2300 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-29T1007.h5',
        (123300,125100,'1b#3',0.6,1,7.0,1.5,'none'), #	13:33	13:36		1b	707	vertical	123300	125100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0707um (1b) ap5=7.0 ap7=1.5 2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-29T1007.h5',
        (123300,125100,'1b#3',0.6,1,7.0,1.5,'none'), #	13:33	13:36		1b	707	vertical	123300	125100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0707um (1d) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100,
        'FLASH2_USER1-2017-11-29T1007.h5',
        (130600,132000,'1d#1',0.6,1,7.0,1.5,'none'), #	13:45	13:47		1d	707	horizontal	130500	131700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0707um (1d) ap5=7.0 ap7=1.5 2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-29T1007.h5',
        (130600,132000,'1d#1',0.6,1,7.0,1.5,'none'), #	13:45	13:47		1d	707	horizontal	130500	131700
    ],
        
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0707um (1d) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg1cd) ap5=7.0 ap7=50.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (17800,20100,'bg1cd',0.6,1,7.0,50.0,'none'), #	10:37	10:41		bg1d 1c	17700	20100,
        'FLASH2_USER1-2017-11-29T1007.h5',
        (132800,134600,'1d#2',0.6,1,7.0,1.5,'none'), #	13:48	13:51		1d	707	horizontal	132300	134100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0707um (1d) ap5=7.0 ap7=1.5 2017-11-26T2300 18.0nm 70uJ 7Und. KOAS=1.5mm (bg1ab) ap5=7.0 ap7=1.5': [
        'FLASH2_USER1-2017-11-26T2300.h5',
        (51100,53750,'bg1ab',0.6,1,7.0,1.5,'none'), #	27.11.17 0:25	00:30:00	20462	bg1ba			50900	53900
        'FLASH2_USER1-2017-11-29T1007.h5',
        (132800,134600,'1d#2',0.6,1,7.0,1.5,'none'), #	13:48	13:51		1d	707	horizontal	132300	134100
    ],
        
        
        
        
        
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0890um (2b#1) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (140400,142400,'2b#3',0.6,1,7.0,1.5,'none'), #	14:01	14:04		2b	890	vertical	140100	141900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0890um (2b#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (202000,202400,'2b#4',0.6,1,7.0,1.5,'none'), #	15:44	15:44		2b	890	vertical	201900	201900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 0890um (2d#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg2ab#1) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (144900,146650,'bg2ab#1',0.6,1,7.0,3.0,'none'), #	14:09	14:12		BG			144900	146700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (135900,137700,'2d#2',0.6,1,7.0,1.5,'none'), #	13:54	13:57		2d	890	horizontal	135900	137700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 1047um (3b#3) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (153350,155400,'3b#3',0.6,1,7.0,1.5,'none'), #	14:23	14:26		3b	1047	vertical	153300	155100
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 1047um (3d#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (157950,160500,'3d#2',0.6,1,7.0,1.5,'none'), #	14:30	14:34		3d	1047	horizontal	157500	159900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 1335um (4b#3) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (166800,168300,'4b#3',0.6,1,7.0,1.5,'none'), #	14:45	14:48		4b	1335	vertical	166500	168300
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 1335um (4d#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA (bg3ab) ap5=7.0 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (151300,153100,'bg3ab',0.6,1,7.0,3.0,'none'), #	14:19	14:22		Bg			150900	152700
        'FLASH2_USER1-2017-11-29T1007.h5',
        (161950,163700,'4d#2',0.6,1,7.0,1.5,'none'), #	14:37	14:40		4d	1335	horizontal	161700	163500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 1570um (5b#3) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (176100,177100,'5b#3',0.6,1,7.0,1.5,'none'), #	15:00	15:02		5b	1570	vertical	175500	176700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=PMMA 1570um (5d#2) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'), #	12:08	12:11		background			72300	74100
        'FLASH2_USER1-2017-11-29T1007.h5',
        (180000,181100,'5d#2',0.6,1,7.0,1.5,'none'), #	15:07	15:09		5d	1570	horizontal	179700	180900
    ],

    
         #wavelength_set = 13.5
         #aperture5 = 7
         # beam size shrinking
    
    
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm-variations 0445um (4a#3-13.5nm) ap5=7.0 ap7=3.0 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'),
        'FLASH2_USER1-2017-11-29T1007.h5',
        (205100,206000,'4a#3',0.6,1,7.0,3.0,'none'), #	15:49	15:50		5a	445	vertical	204900	205500
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm-variations 0445um (5a#2-13.5nm) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'),
        'FLASH2_USER1-2017-11-29T1007.h5',
        (209700,212700,'5a#2',0.6,1,7.0,1.5,'none'), #	15:57	16:01		5a	445	vertical	209700	212100 # not working?
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm-variations 0445um (5a#3-12.5nm) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'),
        'FLASH2_USER1-2017-11-29T1007.h5',
        (229200,231000,'5a#3',0.6,1,7.0,1.5,'none'), #	16:28	16:32		5a	445	vertical	228300	230700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm-variations 0445um (5a#4-14.5nm) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'),
        'FLASH2_USER1-2017-11-29T1007.h5',
        (231500,233800,'5a#4',0.6,1,7.0,1.5,'none'), #	16:33	16:37		5a	445	vertical	231300	233700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm-variations 0445um (5a#5-14.5nm) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'),
        'FLASH2_USER1-2017-11-29T1007.h5',
        (234300,234900,'5a#5',0.6,1,7.0,1.5,'none'), #	16:38	16:39		5a	445	vertical	234300	234900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm-variations 0445um (5a#6-13.5nm) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'),
        'FLASH2_USER1-2017-11-29T1007.h5',
        (236100,237900,'5a#6',0.6,1,7.0,1.5,'none'), #	16:41	16:44		5a	445	vertical	236100	237900
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm-variations 0445um (5a#7-13.5nm) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'),
        'FLASH2_USER1-2017-11-29T1007.h5',
        (238500,239700,'5a#7',0.6,1,7.0,1.5,'none'), #	16:45	16:47		5a	445	vertical	238500	239700
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm-variations 0445um (5a#8-13.5nm) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'),
        'FLASH2_USER1-2017-11-29T1007.h5',
        (240900,243300,'5a#8',0.6,1,7.0,1.5,'none'), #	16:49	16:53		5a	445	vertical	240900	243300
    ],
    '2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm-variations 0445um (5a#9-13.5nm) ap5=7.0 ap7=1.5 2017-11-29T1007 13.5nm 111uJ 12Und. KOAS=1.5mm (bg5ab) ap5=8.8 ap7=3.0': [
        'FLASH2_USER1-2017-11-29T1007.h5',
        (72550,74250,'bg5ab',0.6,1,8.8,3.0,'none'),
        'FLASH2_USER1-2017-11-29T1007.h5',
        (245300,246900,'5a#9',0.6,1,7.0,1.5,'none'), #	16:56	16:59		5a	445	vertical	245100	246900
    ]  
}

settings = collections.OrderedDict(sorted(settings_d.items(), key=lambda t: t[0]))



In [11]:
def print_time(t_sec, t_microsec, pulse_id):
    """Converts seconds in human-readable time and prints formatted time"""

    tloc = time.localtime(t_sec)  # converts sec to the tuple struct_time in local
    print('Input time :', t_sec, 'sec,', t_microsec, 'microsec, ')
    print('Local time :', time.strftime('%Y-%m-%d %H:%M:%S', tloc))
    print('pulse_id :', pulse_id)

#def show_event(pinholes_event_id,pixis_event_id,pinholes_dataset,pixis_dataset)
# at least in interact_output is giving the datasets as function arguments making it very very slow ...

def show_event(pinholes_dataset, pinholes_event_id, pixis_dataset, pixis_event_id, timestamp_dataset, begin, end, ph, thr, chk):
#     begin = dataset_args[0]
#     end = dataset_args[1]
#     ph = dataset_args[2]
#     thr = dataset_args[3]
#     chk = dataset_args[4]

    pinholes = ph
    choices = {'1a': (50, 'vertical'), '1b': (707, 'vertical'),'1c': (50, 'horizontal'), '1d': (707, 'horizontal'),
               '2a': (107, 'vertical'), '2b': (890, 'vertical'), '2c': (107, 'horizontal'), '2d': (890, 'horizontal'),
               '3a': (215, 'vertical'), '3b': (1047, 'vertical'), '3c': (215, 'horizontal'), '3d': (1047, 'horizontal'),
               '4a': (322, 'vertical'), '4b': (1335, 'vertical'), '4c': (322, 'horizontal'), '4d': (1335, 'horizontal'),
               '5a': (445, 'vertical'), '5b': (1570, 'vertical'), '5c': (445, 'horizontal'), '5d': (1570, 'horizontal')}
    (sep, orient) = choices.get(pinholes,(0,'none'))
    
    pixis_2darr = pixis_dataset[pixis_event_id]
    pinholes_2darr = pinholes_dataset[pinholes_event_id]

    pinholes_time_arr = timestamp_dataset[pinholes_event_id]
    pinholes_t_sec = pinholes_time_arr[0]
    pinholes_t_microsec = pinholes_time_arr[1]
    pinholes_pulse_id = pinholes_time_arr[2]

    pixis_time_arr = timestamp_dataset[pixis_event_id]
    pixis_t_sec = pixis_time_arr[0]
    pixis_t_microsec = pixis_time_arr[1]
    pixis_pulse_id = pixis_time_arr[2]

    fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), facecolor='w', edgecolor='w', frameon=True)
    
    im_ax0 = ax0.imshow(ndimage.rotate(pinholes_2darr, 90), origin='lower', interpolation='nearest')
    fig.colorbar(im_ax0, ax=ax0, pad=0.01, fraction=0.1, shrink=1.00, aspect=20)
    ax0.set_title('Pinholes: event_id: '
                  + str(pinholes_event_id)
                  + '\n pulse ID: '
                  + str(pinholes_pulse_id)
                  + '\n time: '
                  + str(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(pinholes_t_sec)))
                  + '.' + str(pinholes_t_microsec) + ' usec'
                  )

    im_ax1 = ax1.imshow(ndimage.rotate(pixis_2darr, 90), origin='lower', interpolation='nearest')
    fig.colorbar(im_ax1, ax=ax1, pad=0.01, fraction=0.1, shrink=1.00, aspect=20)
    ax1.set_title('PIXIS: event_id: '
                  + str(pixis_event_id)
                  + '\n pulse ID: '
                  + str(pixis_pulse_id)
                  + '\n time: '
                  + str(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(pixis_t_sec)))
                  + '.' + str(pixis_t_microsec) + ' usec'
                  )

    fig.suptitle('Pinholes vs. PIXIS' + ' delta=' + str(pixis_event_id - pinholes_event_id)
                + 'pair: ' + pinholes + '(' + str(sep) + 'um ' + orient +')')
    plt.show()
    with status_widget:
        print('showevent:' + str(begin) + ' ' + str(end) + ' ' + str(ph) + ' ' + str(thr) + ' ' + str(chk))

#def show_event_filtered(index,pinholes_dataset,pixis_dataset):
#   show_event(pinholes_filtered_id[index],pixis_filtered_id[index],pinholes_dataset,pixis_dataset)

def show_event_useful(index):
    show_event(pinholes_useful_id[index], pixis_useful_id[index])

def show_event_global_datasets(pinholes_event_id, pixis_event_id, begin, end, ph, thr, chk):
    global pinholes_dataset
    global pixis_dataset
    global timestamp_dataset    
    show_event(pinholes_dataset, pinholes_event_id, pixis_dataset, pixis_event_id, timestamp_dataset, begin, end, ph, thr, chk)


def find_pixis_events(pixis_dataset, hdf5_file_name):
    print('Searching for PIXIS events ...')
    pixis_event_id = []
    dataset_length = pixis_dataset.shape[0]
    for event_id in range(0, dataset_length):
        #print('Progress finding Pixis events: ' + str(event_id / dataset_length * 100) + '%')
        matrix = pixis_dataset[event_id]
        if np.max(matrix) > 0:
            pixis_event_id.append(event_id)
    #np.savetxt(useful_dir + hdf5_file_name + 'pixis_event_id.txt', pixis_event_id, fmt='%d')
    print('... done finding pixis events')
    return pixis_event_id

daq_parameter=[
'FL2/Photon Diagnostic/Wavelength/OPIS tunnel/Processed/mean wavelength',
'FL2/Photon Diagnostic/Wavelength/OPIS tunnel/Processed/mean phtoton energy',    
'FL2/Electron Diagnostic/Undulator setting/SASE03 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE04 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE05 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE06 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE07 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE08 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE09 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE10 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE11 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE12 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE13 gap',
'FL2/Electron Diagnostic/Undulator setting/SASE14 gap',
'FL2/Electron Diagnostic/Undulator setting/set wavelength',
'FL2/Photon Diagnostic/GMD/Average energy/hall',
'FL2/Photon Diagnostic/GMD/Average energy/tunnel',
'FL2/Photon Diagnostic/GMD/Beam position/Average/position hall horizontal',
'FL2/Photon Diagnostic/GMD/Beam position/Average/position hall vertical',
'FL2/Photon Diagnostic/GMD/Beam position/Average/position tunnel horizontal',
'FL2/Photon Diagnostic/GMD/Beam position/Average/position tunnel vertical',
'FL2/Photon Diagnostic/GMD/Beam position/Pulse resolved/hall x',
'FL2/Photon Diagnostic/GMD/Beam position/Pulse resolved/hall y',
'FL2/Photon Diagnostic/GMD/Beam position/Pulse resolved/tunnel x',
'FL2/Photon Diagnostic/GMD/Beam position/Pulse resolved/tunnel y',
'FL2/Photon Diagnostic/GMD/Pulse resolved energy/energy aux hall',
'FL2/Photon Diagnostic/GMD/Pulse resolved energy/energy aux tunnel',
'FL2/Photon Diagnostic/GMD/Pulse resolved energy/energy hall',
'FL2/Photon Diagnostic/GMD/Pulse resolved energy/energy tunnel',
'FL2/Photon Diagnostic/GMD/Pulse resolved energy/position horizontal hall',
'FL2/Photon Diagnostic/GMD/Pulse resolved energy/position horizontal tunnel',
'FL2/Photon Diagnostic/GMD/Pulse resolved energy/position vertical hall',
'FL2/Photon Diagnostic/GMD/Pulse resolved energy/position vertical tunnel'
]   

def get_datasets(data_dir, hdf5_file_name, start, end):
    hdf5_file = h5py.File(data_dir + hdf5_file_name, 'r')  # 'r' means that hdf5 file is open in read-only mode
    pixis_dataset = hdf5_file['/Experiment/Camera/Pixis 1/image'][start:end]
    pinholes_dataset = hdf5_file['/Experiment/Camera/FL24/Pinhole B/image'][start:end]
    timestamp_dataset = hdf5_file['/Timing/time stamp/fl2user1'][start:end]
    
    daq_parameter_dataset = []
    for parameter in daq_parameter:
        daq_parameter_dataset.append(hdf5_file[parameter][start:end])
    
    hdf5_file.close()
    return (pixis_dataset, pinholes_dataset, timestamp_dataset, daq_parameter_dataset)

def get_hdf5_file_length(data_dir, hdf5_file_name):
    hdf5_file = h5py.File(data_dir + hdf5_file_name, 'r')  # 'r' means that hdf5 file is open in read-only mode

    length = hdf5_file['/Experiment/Camera/Pixis 1/image'].len()

    hdf5_file.close()

    return length

# Create useful datasets

In [12]:
def update_eventid_widgets():
    global pixis_dataset
    global pinholes_dataset
    pixis_event_id_play.max = pixis_dataset.shape[0]-1
    pixis_event_id_slider.max = pixis_dataset.shape[0]-1
    pinholes_event_id_play.max = pinholes_dataset.shape[0]-1
    pinholes_event_id_slider.max = pinholes_dataset.shape[0]-1


def create_useful_datasets(hdf5_file_name, begin, end, ph, thr, chk):
    with status_widget:
        print('create_datasets:' + str(hdf5_file_name) + str(begin) + ' ' + str(end) + ' ' + str(ph) + ' ' + str(thr) + ' ' + str(chk))
    slice_beginning = begin
    slice_ending = end 
    pinholes = ph

    choices = {'1a': (50, 'vertical'), '1b': (707, 'vertical'),'1c': (50, 'horizontal'), '1d': (707, 'horizontal'),
               '2a': (107, 'vertical'), '2b': (890, 'vertical'), '2c': (107, 'horizontal'), '2d': (890, 'horizontal'),
               '3a': (215, 'vertical'), '3b': (1047, 'vertical'), '3c': (215, 'horizontal'), '3d': (1047, 'horizontal'),
               '4a': (322, 'vertical'), '4b': (1335, 'vertical'), '4c': (322, 'horizontal'), '4d': (1335, 'horizontal'),
               '5a': (445, 'vertical'), '5b': (1570, 'vertical'), '5c': (445, 'horizontal'), '5d': (1570, 'horizontal')}
    (sep, orient) = choices.get(pinholes,(0,'none'))

    useful_hdf5_file_name = hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_useful.h5'

    pinholes_useful_id_txtfile = useful_dir + hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_pinholes_useful_id.txt'
    pixis_useful_id_txtfile = useful_dir + hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_pixis_useful_id.txt'
    pinholes_sum_txtfile = useful_dir + hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_pinholes_sum.txt'
    pixis_sum_txtfile = useful_dir + hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_pixis_sum.txt'
    pinholes_sum_norm_txtfile = useful_dir + hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_pinholes_sum_norm.txt'
    pixis_sum_norm_txtfile = useful_dir + hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_pixis_sum_norm.txt'
    pinholes_useful_sum_norm_txtfile = useful_dir + hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_pinholes_useful_sum_norm.txt'
    pixis_useful_sum_norm_txtfile = useful_dir + hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_pixis_useful_sum_norm.txt'
    
    
    
    # Plot both pinhole and Pixis synchronized

    if os.path.exists(useful_dir + useful_hdf5_file_name) and (chk == 1 or chk == True):
        
        with status_widget:
            print('File already exists and checked, loading useful events ...')
        (pixis_dataset_useful, pinholes_dataset_useful, timestamp_dataset_useful, daq_parameter_dataset_useful) = get_datasets(useful_dir,useful_hdf5_file_name,None,None)
        pixis_useful_id = range(pixis_dataset_useful.shape[0]-1)
        pinholes_useful_id = range(pinholes_dataset_useful.shape[0]-1)
        #update_eventid_widgets()
        with status_widget:
            print('done.')
        return (pixis_dataset_useful, pixis_useful_id, pinholes_dataset_useful, pinholes_useful_id, timestamp_dataset_useful, daq_parameter_dataset_useful) 

    else:
        with status_widget:
            print('File does NOT exist or selection is not good yet, loading all events ...')
        (pixis_dataset, pinholes_dataset, timestamp_dataset, daq_parameter_dataset) = get_datasets(data_dir, 
                                                                       hdf5_file_name, 
                                                                       slice_beginning, 
                                                                       slice_ending)
        event_id_range = list(range(pixis_dataset.shape[0]))
        # Either load useful events or find them
        # if already found, load the txt file of useful events
        if os.path.exists(pinholes_useful_id_txtfile) and os.path.exists(pixis_useful_id_txtfile) \
        and os.path.exists(pinholes_sum_txtfile) and os.path.exists(pixis_sum_txtfile) \
        and os.path.exists(pinholes_sum_norm_txtfile) and os.path.exists(pixis_sum_norm_txtfile) \
        and os.path.exists(pinholes_useful_sum_norm_txtfile) and os.path.exists(pixis_useful_sum_norm_txtfile):
            with status_widget:
                print('loading useful event ids')
            pinholes_useful_id = np.loadtxt(pinholes_useful_id_txtfile, dtype='int')
            pixis_useful_id = np.loadtxt(pixis_useful_id_txtfile, dtype='int')
            pinholes_sum = np.loadtxt(pinholes_sum_txtfile, dtype='int')
            pixis_sum = np.loadtxt(pixis_sum_txtfile, dtype='int')
            pinholes_sum_norm = np.loadtxt(pinholes_sum_norm_txtfile, dtype='float')
            pixis_sum_norm = np.loadtxt(pixis_sum_norm_txtfile, dtype='float')
            pinholes_useful_sum_norm = np.loadtxt(pinholes_useful_sum_norm_txtfile, dtype='float')
            pixis_useful_sum_norm = np.loadtxt(pixis_useful_sum_norm_txtfile, dtype='float')
        # if no txt file of useful events exists, find them
        else:
            # Sum up all the counts for each image and normalize to the maximum counts
            
            pixis_sum = [np.sum(pixis_dataset[i].astype(np.int64)) for i in event_id_range]
            pixis_sum_max = np.max(pixis_sum)
            pixis_sum_norm = np.divide(pixis_sum, pixis_sum_max)
            pinholes_sum = [np.sum(pinholes_dataset[i].astype(np.int64)) for i in event_id_range]
            pinholes_sum_max = np.max(pinholes_sum)
            pinholes_sum_norm = np.divide(pinholes_sum, pinholes_sum_max)

            # Select only those pinhole events where there is a signal, threshold in relation when the fast shutter was open
            pinholes_event_id_with_beam = np.where(pinholes_sum_norm > thr)
            pinholes_event_id_with_beam = pinholes_event_id_with_beam[0]

            # Select only those events in chopper mode
            pinholes_event_id_with_beam_chopper = []
            for idx in range(pinholes_event_id_with_beam.size-1):
                if pinholes_event_id_with_beam[idx+1]-pinholes_event_id_with_beam[idx] > 1 and pinholes_event_id_with_beam[idx]-pinholes_event_id_with_beam[idx-1] > 1:
                    pinholes_event_id_with_beam_chopper.append(pinholes_event_id_with_beam[idx])

            # Find all pixis events, i.e. where there is a signal   
            #pixis_event_id = find_pixis_events(pixis_dataset, hdf5_file_name)
            pixis_event_id = []
            pixis_event_id = np.where(pixis_sum_norm > 0)  # np.where only works if it is an array, pixis_sum_norm is one, pixis_sum is not ...
            pixis_event_id = pixis_event_id[0]

            # For each pixis event find the corresponding pinhole event
            pixis_useful_id = []
            pixis_useful_sum_norm = []
            pinholes_useful_id = []
            pinholes_useful_sum_norm = []

            for pixis_idx in pixis_event_id:
                for pinholes_idx in pinholes_event_id_with_beam_chopper:
                    if pixis_idx - pinholes_idx < 16 and pixis_idx - pinholes_idx > 6:
                        pixis_useful_id.append(pixis_idx)
                        pixis_useful_sum_norm.append(pixis_sum_norm[pixis_idx])
                        pinholes_useful_id.append(pinholes_idx)  
                        pinholes_useful_sum_norm.append(pinholes_sum_norm[pinholes_idx])

            # Saving useful event id to txt files:
            np.savetxt(pinholes_useful_id_txtfile, pinholes_useful_id, fmt='%d')
            np.savetxt(pixis_useful_id_txtfile, pixis_useful_id, fmt='%d')
            np.savetxt(pinholes_sum_txtfile, pinholes_sum, fmt='%d')
            np.savetxt(pixis_sum_txtfile, pixis_sum, fmt='%d')
            np.savetxt(pinholes_sum_norm_txtfile, pinholes_sum_norm, fmt='%f')
            np.savetxt(pixis_sum_norm_txtfile, pixis_sum_norm, fmt='%f')
            np.savetxt(pinholes_useful_sum_norm_txtfile, pinholes_useful_sum_norm, fmt='%f')
            np.savetxt(pixis_useful_sum_norm_txtfile, pixis_useful_sum_norm, fmt='%f')
            with status_widget:
                print('done saving txt files.')

        # Plot the normalized counts for each image and the found useful events

        fig = plt.figure(figsize=(9.5, 5))
        plt.subplot(111)
        slice_range = np.arange(slice_beginning,slice_ending)

        plt.bar(np.asarray(event_id_range)+slice_beginning, pinholes_sum_norm, label ='pinhole', width=1, color='darkcyan')
        plt.scatter(np.asarray(event_id_range)+slice_beginning, pinholes_sum_norm, label ='pinhole', color='darkcyan')
        plt.scatter(np.asarray(pinholes_useful_id)+slice_beginning, pinholes_useful_sum_norm, label ='pinhole useful', color='cyan')

        plt.bar(np.asarray(event_id_range)+slice_beginning, pixis_sum_norm, label='pixis', width=1, color='darkmagenta')
        plt.scatter(np.asarray(event_id_range)+slice_beginning, pixis_sum_norm, label='pixis', color='darkmagenta')
        plt.scatter(np.asarray(pixis_useful_id)+slice_beginning, pixis_useful_sum_norm, label='pixis useful', color='violet')

        plt.legend()
        plt.xlim([slice_beginning, slice_ending])
        plt.ylim([0, 1])
        plt.show()

        def stringtime(pinholes_event_id):
            pinholes_time_arr = timestamp_dataset[pinholes_event_id]
            pinholes_t_sec = pinholes_time_arr[0]
            pinholes_t_microsec = pinholes_time_arr[1]
            pinholes_pulse_id = pinholes_time_arr[2]
            return str(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(pinholes_t_sec)))

        print('begin: ' + stringtime(0))
        print('end  : ' + stringtime(pixis_dataset.shape[0]-1) + ' event:' + str(pixis_dataset.shape[0]))


        # if found useful event ids make sense, chk will be set by user to 1 or True. In this case 
        # save the useful event ids to txt files
        if (chk == 1 or chk == True):

            # Saving useful events to new hdf5 file using checked useful ids from the txt files

            daq_parameter=[
            'FL2/Photon Diagnostic/Wavelength/OPIS tunnel/Processed/mean wavelength',
            'FL2/Photon Diagnostic/Wavelength/OPIS tunnel/Processed/mean phtoton energy',    
            'FL2/Electron Diagnostic/Undulator setting/SASE03 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE04 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE05 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE06 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE07 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE08 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE09 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE10 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE11 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE12 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE13 gap',
            'FL2/Electron Diagnostic/Undulator setting/SASE14 gap',
            'FL2/Electron Diagnostic/Undulator setting/set wavelength',
            'FL2/Photon Diagnostic/GMD/Average energy/hall',
            'FL2/Photon Diagnostic/GMD/Average energy/tunnel',
            'FL2/Photon Diagnostic/GMD/Beam position/Average/position hall horizontal',
            'FL2/Photon Diagnostic/GMD/Beam position/Average/position hall vertical',
            'FL2/Photon Diagnostic/GMD/Beam position/Average/position tunnel horizontal',
            'FL2/Photon Diagnostic/GMD/Beam position/Average/position tunnel vertical',
            'FL2/Photon Diagnostic/GMD/Beam position/Pulse resolved/hall x',
            'FL2/Photon Diagnostic/GMD/Beam position/Pulse resolved/hall y',
            'FL2/Photon Diagnostic/GMD/Beam position/Pulse resolved/tunnel x',
            'FL2/Photon Diagnostic/GMD/Beam position/Pulse resolved/tunnel y',
            'FL2/Photon Diagnostic/GMD/Pulse resolved energy/energy aux hall',
            'FL2/Photon Diagnostic/GMD/Pulse resolved energy/energy aux tunnel',
            'FL2/Photon Diagnostic/GMD/Pulse resolved energy/energy hall',
            'FL2/Photon Diagnostic/GMD/Pulse resolved energy/energy tunnel',
            'FL2/Photon Diagnostic/GMD/Pulse resolved energy/position horizontal hall',
            'FL2/Photon Diagnostic/GMD/Pulse resolved energy/position horizontal tunnel',
            'FL2/Photon Diagnostic/GMD/Pulse resolved energy/position vertical hall',
            'FL2/Photon Diagnostic/GMD/Pulse resolved energy/position vertical tunnel'
            ]        
            hdf5_file = h5py.File(data_dir + hdf5_file_name, 'r') 
            useful_hdf5_file = h5py.File(useful_dir + useful_hdf5_file_name, "w")
            pixis_dataset_useful = useful_hdf5_file.create_dataset('/Experiment/Camera/Pixis 1/image',(len(pixis_useful_id),1024,1024),'uint16')
            for idx in range(len(pixis_useful_id)):
                pixis_dataset_useful[idx] = hdf5_file['/Experiment/Camera/Pixis 1/image'][slice_beginning+pixis_useful_id[idx]]

            pinholes_dataset_useful = useful_hdf5_file.create_dataset('/Experiment/Camera/FL24/Pinhole B/image',(len(pinholes_useful_id),1280,960),'uint16')
            for idx in range(len(pinholes_useful_id)):
                pinholes_dataset_useful[idx] = hdf5_file['/Experiment/Camera/FL24/Pinhole B/image'][slice_beginning+pinholes_useful_id[idx]]

            timestamp_dataset_useful = useful_hdf5_file.create_dataset('/Timing/time stamp/fl2user1',(len(pinholes_useful_id),3),'uint32')
            for idx in range(len(pinholes_useful_id)):
                timestamp_dataset_useful[idx] = hdf5_file['/Timing/time stamp/fl2user1'][slice_beginning+pinholes_useful_id[idx]]

            for j in range(len(daq_parameter)):
                daq_parameter_dataset_useful = useful_hdf5_file.create_dataset(daq_parameter[j],(len(pinholes_useful_id),1),'float64')
                for idx in range(len(pinholes_useful_id)):
                    daq_parameter_dataset_useful[idx] = hdf5_file[daq_parameter[j]][slice_beginning+pinholes_useful_id[idx]]

            hdf5_file.close()
            useful_hdf5_file.close()

            (pixis_dataset_useful, timestamp_dataset_useful, daq_parameter_dataset_useful) = get_datasets(useful_dir,useful_hdf5_file_name,None,None)
            update_eventid_widgets()
            with status_widget:
                print('done generating useful hdf5 file.')
            return (pixis_dataset_useful, pixis_useful_id, pinholes_dataset_useful, pinholes_useful_id, timestamp_dataset_useful, daq_parameter_dataset_useful) 

        else:
            update_eventid_widgets()  #not working??
            with status_widget:
                print('done loading all events in hdf5 file selection.')
            return (pixis_dataset, pixis_useful_id,  pinholes_dataset, pinholes_useful_id, timestamp_dataset, daq_parameter_dataset)
        


def run_create_useful_datasets(run, hdf5_file_name, begin, end, ph, thr, chk):
    if run == True:
        with status_widget:
            print('datasets are created or loaded ...')
            (pixis_dataset, pixis_useful_id, pinholes_dataset, pinholes_useful_id, timestamp_dataset, daq_parameter_dataset) = create_useful_datasets(hdf5_file_name, begin, end, ph, thr, chk)
        return (pixis_dataset, pixis_useful_id, pinholes_dataset, pinholes_useful_id, timestamp_dataset, daq_parameter_dataset) 
    else: #default datasets or empty datasets ...
        with status_widget:
            print('datasets are NOT created or loaded ... loading standard datasets')
        hdf5_file_name='FLASH2_USER1-2017-11-24T1309.h5'
        dataset_args = (105800,107000,'1a#1',0.6,1,7.0,50.0,'none')
        #global pixis_dataset
        #global pinholes_dataset
        #global timestamp_datset
        #global daq_parameter_dataset
        (pixis_dataset, pixis_useful_id, pinholes_dataset, pinholes_useful_id, timestamp_dataset, daq_parameter_dataset) = create_useful_datasets(hdf5_file_name, dataset_args[0], dataset_args[1], dataset_args[2][0:2], dataset_args[3], dataset_args[4])
        return (pixis_dataset, pixis_useful_id, pinholes_dataset, pinholes_useful_id, timestamp_dataset, daq_parameter_dataset) 

In [13]:
# create useful datasets for all datasets

run_create_useful_dataset_hdf5files = False

if run_create_useful_dataset_hdf5files == True:
    for k,v in dataset_image_args.items():
        print(k)
        for j in v:
            print(str(k) + str(j))
            hdf5_file_name = k
            dataset_args = j
    #        create_useful_dataset(hdf5_file_name, dataset_args)
            
    for k,v in dataset_bg_args.items():
        print(k)
        for j in v:
            print(str(k) + str(j))
            hdf5_file_name = k
            dataset_args = j
            create_useful_dataset(hdf5_file_name, dataset_args)
    
#hdf5_file_name = list(dataset_image_args.keys())[0]
#dataset_args = dataset_image_args[hdf5_file_name][1]
#create_useful_dataset(hdf5_file_name, dataset_args)

# Display images of defined dataset or select or own range

In [14]:
def select_dataset_args(hdf5_file_name):
    dataset_args_widget.options = dataset_args[hdf5_file_name]

hdf5_file_name_widget = widgets.Dropdown(options=dataset_args.keys())
dataset_args_widget = widgets.Dropdown(options=dataset_args[hdf5_file_name_widget.value])

select_dataset_args_interactive = interactive(select_dataset_args, 
                                                    hdf5_file_name=hdf5_file_name_widget)

display(dataset_args_widget)



def update_dataset_args_subwidgets(change):
    run_widget.value = False
    begin_widget.value = dataset_args_widget.value[0]
    end_widget.value = dataset_args_widget.value[1]
    ph_widget.value = dataset_args_widget.value[2][0:2]
    thr_widget.value = dataset_args_widget.value[3]
    chk_widget.value = dataset_args_widget.value[4]
    
    #create_useful_datasets_interactive.update()
    pixis_event_id_play.max = pixis_dataset.shape[0]-1
    pixis_event_id_slider.max = pixis_dataset.shape[0]-1
    pinholes_event_id_play.max = pinholes_dataset.shape[0]-1
    pinholes_event_id_slider.max = pinholes_dataset.shape[0]-1
    
dataset_args_widget.observe(update_dataset_args_subwidgets, names='value')

run_widget = widgets.Checkbox(
    value=False,
    description='Run (Disable before changing multiple parameters or pick a defined dataset!)',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Enable to run create_dataset_useful',
    icon='check'
)

def run_widget_on_click(change):
    with status_widget:
        clear_output(wait=True)
    create_useful_datasets_interactive.update()
    global pixis_dataset
    global pixis_useful_id
    global pinholes_dataset
    global pinholes_useful_id
    global timestamp_dataset
    global daq_parameter_dataset
    (pixis_dataset, pixis_useful_id, pinholes_dataset, pinholes_useful_id, timestamp_dataset, daq_parameter_dataset) = create_useful_datasets_interactive.result
    pixis_event_id_play.max = pixis_dataset.shape[0]-1
    pixis_event_id_slider.max = pixis_dataset.shape[0]-1
    pinholes_event_id_play.max = pinholes_dataset.shape[0]-1
    pinholes_event_id_slider.max = pinholes_dataset.shape[0]-1
    pinholes_useful_event_id_slider.options=pinholes_useful_id
    pixis_useful_event_id_slider.options=pixis_useful_id
    useful_event_id_slider.options=pixis_useful_id
    show_event_interactive.update()

run_widget.observe(run_widget_on_click, names='value')


begin_widget = widgets.IntText(value=dataset_args_widget.value[0])
end_widget = widgets.IntText(value=dataset_args_widget.value[1])
ph_widget = widgets.Text(value=dataset_args_widget.value[2][0:2])
thr_widget = widgets.FloatText(value=dataset_args_widget.value[3])
chk_widget = widgets.FloatText(value=dataset_args_widget.value[4])

chk_widget.observe(run_widget_on_click, names='value')

status_widget = widgets.Output()
display(status_widget)

create_useful_datasets_interactive = interactive(run_create_useful_datasets,
                                                    run = run_widget,
                                                    hdf5_file_name=hdf5_file_name_widget, 
                                                    begin=begin_widget,
                                                    end=end_widget,
                                                    ph=ph_widget, 
                                                    thr=thr_widget, 
                                                    chk=chk_widget)

display(create_useful_datasets_interactive)

(pixis_dataset, pixis_useful_id, pinholes_dataset, pinholes_useful_id, timestamp_dataset, daq_parameter_dataset) = create_useful_datasets_interactive.result


#clear_output()
pinholes_event_id_play = widgets.Play(
interval=500,
value=0,
min=0,
#max=pixis_dataset.shape[0]-1,
step=1,
description="Pinhole play",
disabled=False)

pinholes_event_id_slider = widgets.IntSlider(
value=0,
min=0,
max=pixis_dataset.shape[0]-1,
step=1,
#description='Pinhole Event ID',
disabled=False,
continuous_update=False,
orientation='horizontal',
readout=True,
readout_format='d')

pixis_event_id_play = widgets.Play(
interval=500,
value=0,
min=0,
#max=pixis_dataset.shape[0]-1,
step=1,
description="Pixis play",
disabled=False)

pixis_event_id_slider = widgets.IntSlider(
value=0,
min=0,
#max=pixis_dataset.shape[0]-1,
step=1,
description='Pixis Event ID',
disabled=False,
continuous_update=False,
orientation='horizontal',
readout=True,
readout_format='d')



pinholes_useful_event_id_slider = widgets.SelectionSlider(
value=0,
options=pinholes_useful_id,
#min=0,
#max=pinholes_useful_id[len(pinholes_useful_id)],
#step=1,
description='Pinhole Useful Event ID',
disabled=False,
continuous_update=False,
orientation='horizontal',
readout=True,
readout_format='d')

def update_event_id_widgets(change):
    pinholes_event_id_slider.value = pinholes_useful_event_id_slider.value
    pixis_event_id_slider.value = pixis_useful_event_id_slider.value
    

pinholes_useful_event_id_slider.observe(update_event_id_widgets, names='value')


pixis_useful_event_id_slider = widgets.SelectionSlider(
value=0,
options=pixis_useful_id,
#min=0,
#max=pinholes_useful_id[len(pinholes_useful_id)],
#step=1,
description='Pixis Useful Event ID',
disabled=False,
continuous_update=False,
orientation='horizontal',
readout=True,
readout_format='d')
    

pixis_useful_event_id_slider.observe(update_event_id_widgets, names='value')


def sync_event_id_widgets(change):
    pinholes_event_id_slider.value = pinholes_useful_id[useful_event_id_slider.value]
    pixis_event_id_slider.value = pixis_useful_id[useful_event_id_slider.value]

useful_event_id_slider = widgets.IntSlider(
value=0,

min=0,
max=len(pinholes_useful_id),
step=1,
description='Useful Event ID',
disabled=False,
continuous_update=False,
orientation='horizontal',
readout=True,
readout_format='d')

useful_event_id_slider.observe(sync_event_id_widgets, names='value')

widgets.jslink((pinholes_event_id_play, 'value'), (pinholes_event_id_slider, 'value'))
widgets.jslink((pixis_event_id_play, 'value'), (pixis_event_id_slider, 'value'))
ui = widgets.HBox([pinholes_event_id_play, pinholes_event_id_slider, pixis_event_id_play, pixis_event_id_slider])

display(pinholes_useful_event_id_slider)
display(pixis_useful_event_id_slider)
display(useful_event_id_slider)

show_event_interactive = interactive(show_event_global_datasets, 
                                     pinholes_event_id = pinholes_event_id_slider,
                                     pixis_event_id = pixis_event_id_slider,
                                     begin = begin_widget,
                                     end = end_widget,
                                     ph = ph_widget,
                                     thr = thr_widget,
                                     chk = chk_widget)

display(ui)
display(show_event_interactive)


def stringtime(pinholes_event_id):
    pinholes_time_arr = timestamp_dataset[pinholes_event_id]
    pinholes_t_sec = pinholes_time_arr[0]
    pinholes_t_microsec = pinholes_time_arr[1]
    pinholes_pulse_id = pinholes_time_arr[2]
    return str(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(pinholes_t_sec)))

#print('begin: ' + stringtime(0))
#print('end  : ' + stringtime(pixis_dataset.shape[0]-1) + ' event:' + str(pixis_dataset.shape[0]))

#create_useful_dataset_interactive.update()

Dropdown(options=((105800, 107000, '1a#1', 0.6, 1, 7.0, 50.0, 'none'), (107500, 112000, '1a#2', 0.6, 1, 7.0, 5…

Output()

interactive(children=(Checkbox(value=False, description='Run (Disable before changing multiple parameters or p…

SelectionSlider(continuous_update=False, description='Pinhole Useful Event ID', options=(0, 1, 2, 3, 4, 5, 6, …

SelectionSlider(continuous_update=False, description='Pixis Useful Event ID', options=(0, 1, 2, 3, 4, 5, 6, 7,…

IntSlider(value=0, continuous_update=False, description='Useful Event ID', max=14)

interactive(children=(IntSlider(value=0, continuous_update=False, description='pinholes_event_id', max=14), In…

In [15]:
def get_images(hdf5_file_name, dataset_args):
    
    begin = dataset_args[0]
    end = dataset_args[1]
    ph = dataset_args[2]
    thr = dataset_args[3]
    chk = dataset_args[4]
    
    aperture5_mm = dataset_args[5]
    aperture7_mm = dataset_args[6]
    filter_used = dataset_args[7]
    
    slice_beginning = begin
    slice_ending = end 
    pinholes = ph[0:2]
    
    choices = {'1a': (50, 'vertical'), '1b': (707, 'vertical'),'1c': (50, 'horizontal'), '1d': (707, 'horizontal'),
               '2a': (107, 'vertical'), '2b': (890, 'vertical'), '2c': (107, 'horizontal'), '2d': (890, 'horizontal'),
               '3a': (215, 'vertical'), '3b': (1047, 'vertical'), '3c': (215, 'horizontal'), '3d': (1047, 'horizontal'),
               '4a': (322, 'vertical'), '4b': (1335, 'vertical'), '4c': (322, 'horizontal'), '4d': (1335, 'horizontal'),
               '5a': (445, 'vertical'), '5b': (1570, 'vertical'), '5c': (445, 'horizontal'), '5d': (1570, 'horizontal')}
    (sep, orient) = choices.get(pinholes,(np.nan,'bg'))

    useful_hdf5_file_name = hdf5_file_name + '_' + str(slice_beginning) + 'to' + str(slice_ending) + '_useful.h5'

    if os.path.exists(useful_dir + useful_hdf5_file_name):
        (pixis_dataset0, pinholes_dataset0, timestamp_dataset, daq_parameter_dataset) = get_datasets(useful_dir, useful_hdf5_file_name,None,None)

    pixis_dataset = np.rot90(pixis_dataset0,axes=(1,2))
    pixis_avg = np.average(pixis_dataset, axis=0)
    pixis_std = np.std(pixis_dataset, axis=0)
    pixis_std = ndimage.gaussian_filter(pixis_std,sigma=5,order=0)
    
    pinholes_dataset = np.rot90(pinholes_dataset0,axes=(1,2))
    pinholes_avg = np.average(pinholes_dataset, axis=0)
    
    return (pixis_dataset, pixis_avg, pixis_std, pinholes_dataset, pinholes_avg, timestamp_dataset, sep, orient, daq_parameter_dataset, aperture5_mm, aperture7_mm, filter_used)

def pixis_image_plots(imageid,bgfactor):
    
    # Choosing image
    #imageid = 1
    pixis_image = pixis_dataset[imageid]
    pixis_image_minus_bg = np.subtract(pixis_image,bgfactor*pixis_bg_avg)

    fig = plt.figure(figsize=(12, 12))
    ax1 = fig.add_subplot(1, 1, 1, aspect=1)
    im_ax1 = plt.imshow(pixis_bg_avg, origin='lower', interpolation='nearest')
    fig.colorbar(im_ax1, ax=ax1, pad=0.01, fraction=0.1, shrink=1.00, aspect=20)
    plt.title('bg avg')
    plt.show()
    
    fig = plt.figure(figsize=(12, 12))
    ax1 = fig.add_subplot(1, 1, 1, aspect=1)
    im_ax1 = plt.imshow(pixis_image, origin='lower', interpolation='nearest')
    fig.colorbar(im_ax1, ax=ax1, pad=0.01, fraction=0.1, shrink=1.00, aspect=20)
    plt.title('raw image')
    plt.show()
    
    fig = plt.figure(figsize=(12, 12))
    ax1 = fig.add_subplot(1, 1, 1, aspect=1)
    im_ax1 = plt.imshow(pixis_image_minus_bg, origin='lower', interpolation='nearest')
    fig.colorbar(im_ax1, ax=ax1, pad=0.01, fraction=0.1, shrink=1.00, aspect=20)
    plt.title('raw image minus bg')
    plt.show()

    # correct rotation and normalize, show where maximum is
    pixis_rotation=-1
    pixis_image_minus_bg_rot = ndimage.rotate(pixis_image_minus_bg, pixis_rotation, reshape=True)
    
    pixis_image_norm = pixis_image_minus_bg_rot / np.max(pixis_image_minus_bg_rot)

    # Finding center of beam:
    print('Maximum coordinates: ' + str(np.where(pixis_image_norm==1)))
    centerx = np.where(pixis_image_norm==1)[1][0]
    centery = np.where(pixis_image_norm==1)[0][0]

    fig = plt.figure(figsize=(12, 12))
    ax1 = fig.add_subplot(1, 1, 1, aspect=1)
    im_ax1 = plt.imshow(pixis_image_norm, origin='lower', interpolation='nearest')
    plt.scatter(centerx,centery, color='r')
    fig.colorbar(im_ax1, ax=ax1, pad=0.01, fraction=0.1, shrink=1.00, aspect=20)
    #plt.scatter(np.where(pixis_image_norm==1)[1],np.where(pixis_image_norm==1)[0],color='r')
    plt.title('background subtracted and rotation corrected (normalized)')
    plt.show()

    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)
    ax.scatter(list(range(pixis_image_norm.shape[1])),pixis_image_norm[centery,:],s=1,marker='o')
    plt.ylim(-0.3,1)
    plt.title('cross section at y=' + str(centery))
    plt.show()

    # choose only those not zero due to additional pixel after rotation correction
    if orient == 'horizontal':  
        pixis_profile = pixis_image_norm[centery,np.where(pixis_image_norm[centery,:]!=0)[0]]
    if orient == 'vertical':
        #pixis_profile = pixis_image_norm[np.where(pixis_image_norm[:,centerx]!=0)[0],centerx]
        pixis_profile = pixis_image_norm[:,centerx]
        pixis_profile_avg = pixis_image_norm[:,centerx-50:centerx+50]
        pixis_profile_avg = np.average(pixis_profile_avg,axis=1)
        pixis_profile_avg = pixis_profile_avg / np.max(pixis_profile_avg)
  
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)
    ax.scatter(list(range(pixis_profile.size)),pixis_profile,s=1,marker='o')
    ax.scatter(np.where(pixis_profile<0)[0],pixis_profile[np.where(pixis_profile<0)[0]],s=10,marker='x',color='r')
    
    ax.scatter(list(range(pixis_profile_avg.size)),pixis_profile_avg,s=1,marker='+', color='g')
    plt.ylim(-0.3,1)
    plt.title('cross section at y=' + str(centery) + 'trimmed after rotation')
    
    
    fig.tight_layout(pad=0.0, w_pad=0.0, h_pad=0)
    
    if savefigure == True:
        plt.savefig('w'+str(wavelength_nm) \
                + 's'+str(slitwidth_um) \
                + 'd'+str(separation_um) \
                + 'ap'+str(aperturewidth_mm) \
                + 'z' \
                + '.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None) 
    
    plt.show()
    
    return pixis_profile, pixis_profile_avg

# Create Dataframes

still have to reorder function definitions ...

In [16]:
delete_dataframes = False

if delete_dataframes == True:
    del df_image_args
    del df_dataset_timestamp
    del df_dataset_daq_parameter
    del df_all


In [17]:
#del df_settings

hdf5_file_name = []
hdf5_file_name_background = []
setting_wavelength_nm = []
setting_energy_uJ = []
setting_undulators = []
KAOS = []
separation_um = []
pinholes = []
background = []

for idx in range(len(settings.keys())):
    hdf5_file_name.append(settings[list(settings.keys())[idx]][2])
    hdf5_file_name_background.append(settings[list(settings.keys())[idx]][0])
    setting_wavelength_nm.append(float(list(settings.keys())[idx].split()[1][:-2]))
    setting_energy_uJ.append(int(list(settings.keys())[idx].split()[2][:-2]))
    setting_undulators.append(int(list(settings.keys())[idx].split()[3][:-4]))
    KAOS.append(list(settings.keys())[idx].split()[4][5:])
    separation_um.append(int(list(settings.keys())[idx].split()[5][:-2]))
    pinholes.append((settings[list(settings.keys())[idx]][3][2]))
    background.append((settings[list(settings.keys())[idx]][1][2]))

df_settings = pd.DataFrame({'hdf5_file_name' : hdf5_file_name,
                            'hdf5_file_name_background' : hdf5_file_name_background,
                           'setting_wavelength_nm': setting_wavelength_nm,
                           'setting_energy_uJ': setting_energy_uJ,
                           'setting_undulators': setting_undulators,
                           'KAOS': KAOS,
                           'separation_um': separation_um,
                           'pinholes': pinholes,
                           'background' : background})
df_settings

,hdf5_file_name,hdf5_file_name_background,setting_wavelength_nm,setting_energy_uJ,setting_undulators,KAOS,separation_um,pinholes,background
0,FLASH2_USER1-2017-11-24T1309.h5,FLASH2_USER1-2017-11-24T1309.h5,13.5,68,7,PMMA,0,bg1ab,bg1ab
1,FLASH2_USER1-2017-11-24T1309.h5,FLASH2_USER1-2017-11-24T1309.h5,13.5,68,7,PMMA,0,bg1cd,bg1cd
2,FLASH2_USER1-2017-11-24T1309.h5,FLASH2_USER1-2017-11-24T1309.h5,13.5,68,7,PMMA,0,bg2cd-chopper-open,bg2cd-chopper-open
3,FLASH2_USER1-2017-11-24T1309.h5,FLASH2_USER1-2017-11-24T1309.h5,13.5,68,7,PMMA,0,bg3cd-chopper-open,bg3cd-chopper-open
4,FLASH2_USER1-2017-11-24T1309.h5,FLASH2_USER1-2017-11-24T1309.h5,13.5,68,7,PMMA,0,bg4ab,bg4ab
5,FLASH2_USER1-2017-11-24T2020.h5,FLASH2_USER1-2017-11-24T2020.h5,13.5,68,7,PMMA,0,bg5ab,bg5ab
6,FLASH2_USER1-2017-11-24T1309.h5,FLASH2_USER1-2017-11-24T1309.h5,13.5,68,7,PMMA,50,1a#1,bg1ab
7,FLASH2_USER1-2017-11-24T1309.h5,FLASH2_USER1-2017-11-24T1309.h5,13.5,68,7,PMMA,50,1a#2,bg1ab
8,FLASH2_USER1-2017-11-24T1309.h5,FLASH2_USER1-2017-11-24T1309.h5,13.5,68,7,PMMA,50,1a#3,bg1ab
9,FLASH2_USER1-2017-11-24T1309.h5,FLASH2_USER1-2017-11-24T1309.h5,13.5,68,7,PMMA,50,1c,bg1cd


In [18]:
#load_df_all = True # this takes around 5 minutes!
load_df_all = False

if load_df_all == False: 

    # Create DataFrame of all available datasets (dataset_args or dataset_image_args)

    names_df_image_args = ['begin', 'end', 'pinholes', 'threshold', 'checked', 'aperture5_mm', 'aperture7_mm', 'filter']
    df_image_args = pd.DataFrame(columns=names_df_image_args)
    for key in dataset_image_args.keys():
        df_image_args_key = pd.DataFrame.from_records(dataset_image_args[key],columns=names_df_image_args, index=[key]*len(dataset_image_args[key]))
        df_image_args = df_image_args.append(df_image_args_key)
    df_image_args.index.name = 'hdf5_file_name'
    df_image_args.reset_index(inplace=True)

    # add separation_um and orientation based on pinhole"
    def get_sep_and_orient(pinholes):
        pinholes = pinholes[0:2]
        choices = {'1a': (50, 'vertical'), '1b': (707, 'vertical'),'1c': (50, 'horizontal'), '1d': (707, 'horizontal'),
                   '2a': (107, 'vertical'), '2b': (890, 'vertical'), '2c': (107, 'horizontal'), '2d': (890, 'horizontal'),
                   '3a': (215, 'vertical'), '3b': (1047, 'vertical'), '3c': (215, 'horizontal'), '3d': (1047, 'horizontal'),
                   '4a': (322, 'vertical'), '4b': (1335, 'vertical'), '4c': (322, 'horizontal'), '4d': (1335, 'horizontal'),
                   '5a': (445, 'vertical'), '5b': (1570, 'vertical'), '5c': (445, 'horizontal'), '5d': (1570, 'horizontal')}
        (sep, orient) = choices.get(pinholes,(np.nan,'bg'))
        return sep, orient

    df_image_args['separation_um'] = df_image_args['pinholes'].apply(lambda x: get_sep_and_orient(x)[0])
    df_image_args['orientation'] = df_image_args['pinholes'].apply(lambda x: get_sep_and_orient(x)[1])

    # Cast from object to int64, otherwise we cannot merge later
    df_image_args['begin']=df_image_args['begin'].astype('int64')
    df_image_args['end']=df_image_args['end'].astype('int64')

    # save to csv
    df_image_args.to_csv('/home/wodzinsk/PycharmProjects/coherence/'+'image_args.csv')
    #df_image_args = pd.read_csv('/home/wodzinsk/PycharmProjects/coherence/'+'image_args.csv', index_col=0)

    # Create DataFrames for each dataset for each shot

    names_df_dataset_timestamp = ['timestamp_t_sec','timestamp_t_microsec','timestamp_pulse_id']
    names_df_dataset_daq_parameter=['mean wavelength',
                    'mean photon energy',
                    'SASE03 gap',
                    'SASE04 gap',
                    'SASE05 gap',
                    'SASE06 gap',
                    'SASE07 gap',
                    'SASE08 gap',
                    'SASE09 gap',
                    'SASE10 gap',
                    'SASE11 gap',
                    'SASE12 gap',
                    'SASE13 gap',
                    'SASE14 gap',
                    'set wavelength',
                    'average energy hall',
                    'average energy tunnel',
                    'beam position hall horizontal average',
                    'beam position hall vertical average',
                    'beam position tunnel horizontal average',
                    'beam position tunnel vertical average',
                    'beam position hall horizontal pulse resolved',
                    'beam position hall vertical pulse resolved',
                    'beam position tunnel horizontal pulse resolved',
                    'beam position tunnel vertical pulse resolved',
                    'energy aux hall',
                    'energy aux tunnel',
                    'energy hall',
                    'energy tunnel',
                    'GMD position horizontal hall',
                    'GMD position horizontal tunnel',
                    'GMD position vertical hall',
                    'GMD position vertical tunnel'
                    ]

    names_df_all = list(df_image_args.columns) + names_df_dataset_timestamp + names_df_dataset_daq_parameter 
    df_all = pd.DataFrame(columns = names_df_all)


    for filename in list(df_image_args.hdf5_file_name.unique()):

        for dataset_begin in list(df_image_args[df_image_args.hdf5_file_name == filename]['begin']):
        #dataset_begin_index = 0

            df_selection = df_image_args[(df_image_args.hdf5_file_name==filename) & (df_image_args.begin==dataset_begin)]
            dataset_args = (df_selection.begin.item(),
                            df_selection.end.item(),
                            df_selection.pinholes.item(),
                            df_selection.threshold.item(),
                            df_selection.checked.item(),
                            df_selection.aperture5_mm.item(),
                            df_selection.aperture7_mm.item(),
                            df_selection['filter'].item())
            (pixis_dataset, pixis_avg, pixis_std, pinholes_dataset, pinholes_avg, timestamp_dataset, sep, orient, daq_parameter_dataset, aperture5_mm, aperture7_mm, filter_used) = get_images(filename,dataset_args)

            #pinholes_time_arr = timestamp_dataset[pinholes_event_id]
            #pinholes_t_sec = pinholes_time_arr[0]
            #pinholes_t_microsec = pinholes_time_arr[1]
            #pinholes_pulse_id = pinholes_time_arr[2]

            df_dataset_timestamp = pd.DataFrame(timestamp_dataset, columns=names_df_dataset_timestamp)

            df_dataset_daq_parameter = pd.DataFrame(dict(zip(names_df_dataset_daq_parameter,np.squeeze(daq_parameter_dataset))), columns=names_df_dataset_daq_parameter)

            df_dataset = pd.concat([df_dataset_timestamp, df_dataset_daq_parameter], axis=1)

            df_dataset['hdf5_file_name'] = filename
            df_dataset['begin']=dataset_begin

            df_dataset_merged = pd.merge(df_image_args,df_dataset)

            df_all = df_all.append(df_dataset_merged)


    df_all.to_csv('/home/wodzinsk/PycharmProjects/coherence/'+'df_all.csv')

else:
    df_all = pd.read_csv('/home/wodzinsk/PycharmProjects/coherence/'+'df_all.csv', index_col=0)
    
df_all['imageid']=df_all.index

In [24]:
df0 = []

In [33]:
df0 = pd.merge(df_all,df_settings)
df0['timestamp_pulse_id']=df0['timestamp_pulse_id'].astype('int64')

fits_header_list1 = ['bgfactor', 'pixis_rotation', 'pixis_centerx_px', 'pixis_centery_px', 'pinholes_centerx_px', 'pinholes_centery_px', 'pixis_profile_centerx_px_fit', 'pixis_profile_centery_px_fit', 'pinholes_cm_x_px', 'pinholes_cm_y_px', '_lambda_nm_fit', 'gamma_fit', 'sigma_F_gamma_um_opt', 'xi_um']
fits_header_list2 = ['d_um_at_detector', 'I_Airy1', 'I_Airy2']
fits_header_list = fits_header_list1 + fits_header_list2

#fits_header_list1 already exists in saves csv, only adding fits_header_list2, only initiate when
initiate_df_fits = True
if initiate_df_fits == True:
    df0 = df0.reindex(columns = df0.columns.tolist() + fits_header_list2)  
    #df_fits = df0[['timestamp_pulse_id'] + fits_header_list]


load_df_fits_csv = True
if load_df_fits_csv == True:
    df_fits = pd.read_csv('/home/wodzinsk/PycharmProjects/coherence/'+'df_fits_v2.csv', index_col=0)
    df_fits_clean = df_fits[df_fits['pixis_rotation'].notna()].drop_duplicates()
    df_fits = df_fits_clean
    df0 = pd.merge(df0,df_fits, on='timestamp_pulse_id')




In [36]:
df0.columns

Index(['hdf5_file_name', 'begin', 'end', 'pinholes', 'threshold', 'checked',
       'aperture5_mm', 'aperture7_mm', 'filter', 'separation_um',
       'orientation', 'timestamp_t_sec', 'timestamp_t_microsec',
       'timestamp_pulse_id', 'mean wavelength', 'mean photon energy',
       'SASE03 gap', 'SASE04 gap', 'SASE05 gap', 'SASE06 gap', 'SASE07 gap',
       'SASE08 gap', 'SASE09 gap', 'SASE10 gap', 'SASE11 gap', 'SASE12 gap',
       'SASE13 gap', 'SASE14 gap', 'set wavelength', 'average energy hall',
       'average energy tunnel', 'beam position hall horizontal average',
       'beam position hall vertical average',
       'beam position tunnel horizontal average',
       'beam position tunnel vertical average',
       'beam position hall horizontal pulse resolved',
       'beam position hall vertical pulse resolved',
       'beam position tunnel horizontal pulse resolved',
       'beam position tunnel vertical pulse resolved', 'energy aux hall',
       'energy aux tunnel', 'energy 

# Plot data and simulations

In [20]:
# calculation of sampling number for LightPipes
m=1
nm=1e-9*m
um=1e-6*m
mm=1e-3*m
cm=1e-2*m

wavelength = 13.5 * nm
window_size = 1047 * 13 * um
distance = 5.781 * m

sampling_interval = (wavelength * distance) / window_size
print('sampling interval = ' + str(sampling_interval/um) + 'um')

sampling_number = window_size / sampling_interval
print('sampling number = ' + str(sampling_number) + ' (' + str(sampling_number/1048) + '*1048)' ) 

sampling interval = 5.733854970244655um
sampling number = 2373.7956524246097 (2.265072187428063*1048)


Increase the lenght of the output to show all the plots:

In [37]:
def normalize(inputarray):
    normalized_array = inputarray / np.max(inputarray)
    return(normalized_array)

def get_imageids_with_bgs(beamposition_horizontal_interval):

    imageid_sequence = []
    for imageid in df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].sort_values('beam position hall horizontal pulse resolved')['imageid']:
        beamposition_horizontal_image = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])]['beam position hall horizontal pulse resolved'][imageid]
        matching_bg_indices = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2]) & (df_all['beam position hall horizontal pulse resolved'] > beamposition_horizontal_image - beamposition_horizontal_interval/2 ) & (df_all['beam position hall horizontal pulse resolved'] < beamposition_horizontal_image + beamposition_horizontal_interval/2 ) ]['beam position hall horizontal pulse resolved'].index
        if matching_bg_indices.empty == False:
            imageid_sequence.append(imageid)
    
    return imageid_sequence


def get_pixis_profiles(imageid,
                       use_pixis_avg,
                       bgfactor,
                       avg_width,
                       pixis_rotation,
                       pixis_center_autofind,
                       pixis_centerx_px,
                       pixis_centery_px,
                      crop_px):

    # using global datasets ... because of the way interactive works ... 'fixed(dataset)'
    # make a wrapper for this function, where datasets are global...s
    global pixis_dataset
    #gloabl pixis_bg_std
    global pixis_bg_avg

    # Choosing image
    #imageid = 1
    
    #imageid = 4
    
    pixis_centerx_px = int(pixis_centerx_px)
    pixis_centery_px = int(pixis_centery_px)


    if use_pixis_avg == True:
        imageid = -1
    
    if imageid == -1:
        pixis_image = pixis_avg
    else:
        pixis_image = pixis_dataset[imageid]
        beamposition_horizontal_interval = beamposition_horizontal_interval_widget.value
        beamposition_horizontal_image = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])]['beam position hall horizontal pulse resolved'][imageid]
        matching_bg_indices = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2]) & (df_all['beam position hall horizontal pulse resolved'] > beamposition_horizontal_image - beamposition_horizontal_interval/2 ) & (df_all['beam position hall horizontal pulse resolved'] < beamposition_horizontal_image + beamposition_horizontal_interval/2 ) ]['beam position hall horizontal pulse resolved'].index
        
        #beamposition_horizontal_interval = beamposition_horizontal_interval_widget.value
        #beamposition_horizontal_image = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])]['beam position hall horizontal pulse resolved'][imageid]
        #matching_bg_indices = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2]) & (df_all['beam position hall horizontal pulse resolved'] > beamposition_horizontal_image - beamposition_horizontal_interval/2 ) & (df_all['beam position hall horizontal pulse resolved'] < beamposition_horizontal_image + beamposition_horizontal_interval/2 ) ]['beam position hall horizontal pulse resolved'].index
        if matching_bg_indices.empty:
            print('no matching bg for that energy!!')
            pixis_bg_avg = pixis_image
        else:
            pixis_bg_avg = np.mean(pixis_bg_dataset[matching_bg_indices], axis=0)
            energy_bg_mean = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2])]['energy hall'][matching_bg_indices].mean(axis=0)
            energy_bg_std = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2])]['energy hall'][matching_bg_indices].std(axis=0)
        
    pixis_image_minus_bg = np.subtract(pixis_image,bgfactor*pixis_bg_avg)
    if orient == 'vertical':
        pixis_image_minus_bg = ndimage.rotate(pixis_image_minus_bg, 90)

   # correct rotation and normalize, show where maximum is
    #pixis_rotation=-1
    crop = int(pixis_image.shape[0]*np.abs(np.sin(np.pi/180*pixis_rotation)))
    pixis_image_minus_bg_rot = ndimage.rotate(pixis_image_minus_bg, pixis_rotation, reshape=True)
    pixis_image_minus_bg_rot_cropped = pixis_image_minus_bg_rot[crop:-crop,crop+20:-crop]
    
    pixis_image_norm = pixis_image_minus_bg_rot_cropped
    pixis_image_norm = pixis_image_norm[0:min(pixis_image_norm.shape),0:min(pixis_image_norm.shape)]
    pixis_image_norm = normalize(pixis_image_norm)
    
    #np.where(pixis_image_norm < 0)
      
    if crop_px > 0:
        pixis_image_norm = pixis_image_norm[crop_px:-crop_px,crop_px:-crop_px]
        pixis_image_norm = normalize(pixis_image_norm)
    
    set_negative_to_zero = True
    if set_negative_to_zero == True:
        pixis_image_norm[np.where(pixis_image_norm < 0)] = 0
    
    pixis_bg_std_rot = ndimage.rotate(pixis_bg_std, pixis_rotation, reshape=True)
    pixis_bg_std_rot_cropped = pixis_bg_std_rot[crop:-crop,crop+20:-crop]
    pixis_bg_std_norm = pixis_bg_std_rot_cropped / np.max(pixis_image_minus_bg_rot_cropped)
    pixis_bg_std_norm = pixis_bg_std_norm[0:min(pixis_bg_std_norm.shape),0:min(pixis_bg_std_norm.shape)]
    
    if crop_px > 0:
        pixis_bg_std_norm = pixis_bg_std_norm[crop_px:-crop_px,crop_px:-crop_px]

    pixis_cts = np.sum(pixis_image_minus_bg_rot_cropped)

#     if pixis_center_autofind == True:
#         pixis_centerx_px = np.where(pixis_image_norm==1)[1][0]
#         pixis_centery_px = np.where(pixis_image_norm==1)[0][0]
#     else:
#         pixis_centerx_px = int(pixis_centerx_px)
#         pixis_centery_px = int(pixis_centery_px)
        
        

    pixis_profile = pixis_image_norm[ int(pixis_centery_px),:] # lineout at pixis_centery_px
    pixis_profile_avg = pixis_image_norm[ int(pixis_centery_px)-int(avg_width/2): int(pixis_centery_px)+int(avg_width/2),:]
    pixis_profile_avg = np.average(pixis_profile_avg,axis=0)
    pixis_profile_avg = pixis_profile_avg / np.max(pixis_profile_avg)  # why was this commented out?
    
    pixis_profile_avg_centerx_px = np.where(pixis_profile_avg==1)[0]
    
    pixis_profile_alongy = pixis_image_norm[ :, int(pixis_centerx_px)] # lineout at pixis_centerx_px along y
    
    n = pixis_profile_avg.size # number of sampling point  # number of pixels
    dx = 13*1e-6 # sampling # pixel size
    xdata = list(range(n))
    ydata = pixis_profile_alongy
    
    pixis_yshift_px = pixis_centery_px
    p0 = (pixis_yshift_px, 400)
    popt_gauss, pcov_gaussian = curve_fit(lambda x, m, w: gaussianbeam(x, 1, m ,w, 0), xdata, ydata, p0)
    pixis_beamwidth_px = popt_gauss[1]  # this is 2 sigma!
    
    if pixis_center_autofind == True:
#         pixis_centerx_px = np.where(pixis_image_norm==1)[1][0]
#         pixis_centery_px = np.where(pixis_image_norm==1)[0][0]
        pixis_centery_px = popt_gauss[0]
        pixis_centery_px = pixis_centery_px
    else:
        pixis_centerx_px = pixis_centerx_px
        pixis_centery_px = pixis_centery_px
    
    pixis_profile = pixis_image_norm[ int(pixis_centery_px),:] # lineout at pixis_centery_px
    pixis_profile_avg = pixis_image_norm[ int(pixis_centery_px)-int(avg_width/2): int(pixis_centery_px)+int(avg_width/2),:]
    pixis_profile_avg = np.average(pixis_profile_avg,axis=0)
    pixis_profile_avg = pixis_profile_avg / np.max(pixis_profile_avg)  # why was this commented out?

    
    
    pixis_bg_std_avg = pixis_bg_std_norm[ int(pixis_centery_px)-int(avg_width/2): int(pixis_centery_px)+int(avg_width/2),:]
    pixis_bg_std_avg = np.average(pixis_bg_std_avg,axis=0)
    pixis_bg_std_avg = pixis_bg_std_avg / np.max(pixis_profile_avg)  # why was this commented out?

    (pixis_cm_y_px, pixis_cm_x_px) = ndimage.measurements.center_of_mass(pixis_image_norm)

    return pixis_image_norm, pixis_bg_std_norm, pixis_bg_std_avg, pixis_profile, pixis_profile_avg, pixis_profile_avg_centerx_px, pixis_centerx_px, pixis_centery_px, pixis_profile_alongy, pixis_cts, pixis_cm_x_px, pixis_cm_y_px

def get_pinholes_profiles(imageid,bgfactor,pinholes_rotation,centerx_px,centery_px,avg_width_alongy,avg_width_alongx):

    # using global datasets ... because of the way interactive works ... 'fixed(dataset)'
    # make a wrapper for this function, where datasets are global...s
    #global pixis_dataset
    #global pixis_bg_avg

    # Choosing image
    if imageid == -1:
        pinholes_image = np.average(pinholes_dataset, axis=0)
    else:
        pinholes_image = pinholes_dataset[imageid]
    pinholes_image_minus_bg = np.subtract(pinholes_image,bgfactor*pinholes_bg_avg)
    if orient == 'vertical':
        pinholes_image_minus_bg = ndimage.rotate(pinholes_image_minus_bg, 90)

   # correct rotation and normalize, show where maximum is

    pinholes_image_minus_bg_rot = ndimage.rotate(pinholes_image_minus_bg, pinholes_rotation)
    pinholes_image_norm = pinholes_image_minus_bg_rot / np.max(pinholes_image_minus_bg_rot)

    pinholes_cts = np.sum(pinholes_image_minus_bg_rot)

#     pixis_centerx = np.where(pinholes_image_norm==1)[1][0]
#     pixis_centery = np.where(pinholes_image_norm==1)[0][0]

    pinholes_profile_alongx = pinholes_image_norm[centery_px,:]
    pinholes_profile_alongx_avg = pinholes_image_norm[centery_px-int(avg_width_alongy/2):centery_px+int(avg_width_alongy/2),:]
    pinholes_profile_alongx_avg = np.average(pinholes_profile_alongx_avg,axis=0)
    pinholes_profile_alongx_avg = pinholes_profile_alongx_avg / np.max(pinholes_profile_alongx_avg)

    pinholes_profile_alongy = pinholes_image_norm[centerx_px,:]
    pinholes_profile_alongy_avg = pinholes_image_norm[centerx_px-int(avg_width_alongx/2):centerx_px+int(avg_width_alongx/2),:]
    pinholes_profile_alongy_avg = np.average(pinholes_profile_alongy_avg,axis=0)
    pinholes_profile_alongy_avg = pinholes_profile_alongy_avg / np.max(pinholes_profile_alongy_avg)

    (pinholes_cm_y_px, pinholes_cm_x_px) = ndimage.measurements.center_of_mass(pinholes_image_norm)


    #shiftx_px = pixis_centerx - pixis_profile_avg.size/2
    #shiftx_um = shiftx_px * 13
    #widget_shiftx_um.value = shiftx_um

    return pinholes_image_norm, pinholes_profile_alongx, pinholes_profile_alongx_avg, pinholes_profile_alongy, pinholes_profile_alongx_avg, pinholes_cts, pinholes_cm_x_px, pinholes_cm_y_px

def run_fit(I_D_normalized,theta,I_D):

    I_D_fit_normalized = []
    Error = []

    for i in range(100):

        alpha = 0.01 * i # covers the range (0,1) !

        Partial_Coherence_Factor_fit = 1 + alpha * np.cos(theta)
        I_D_fit = I_D * Partial_Coherence_Factor_fit
        I_D_fit_normalized.append(I_D_fit / np.max(I_D_fit))

        # Error of fit
        chi = I_D_normalized - I_D_fit_normalized[i]
        Error.append(abs(sum(chi) / 100))

    gamma_fit = []

    for ii in range(100):

        gamma_fit.append(0.01 * ii)

        #if Error[ii] <= 10 * 1e-2:        # indicates the best fit 1e-2:
            #clear_output()

            #print('gamma_fit:', gamma_fit[ii], 'error: ', Error[ii])
            #fig = plt.figure(figsize=(12, 6))
            #ax = fig.add_subplot(1, 1, 1)

            #plt.plot(R * x, I_D_Normalized,'r', label='Simulation')
            #plt.plot(R * x[::1], I_D_fit_Normalized[ii][::1], 'r-', label='Fit')
            #ax.scatter(np.linspace(-n/2+1, n/2, n),pixis_profile_avg,s=10,marker='o', label='data', color='b')

            #plt.title('Youngs double pinholes experiment, $\gamma_{12}(0) = ' + str(gamma_fit[ii]) +'$')
            #plt.xlabel('x(px)')
            #plt.ylabel('I(x)(a.u.)')
            #plt.legend()
            #plt.xlim(300,400)
            #plt.show()

    return I_D_fit_normalized, gamma_fit, Error


##### deconvolution method #######


def gaussianbeam(x, a, m ,w, offs):
    return a*np.exp(-2*(x-m)**2/w**2) + offs

def gauss2d(x,y,sigma):
    Gprofile = (1/(2*np.pi*sigma)) * np.exp(-(x**2+y**2)/(2*sigma**2))
    return Gprofile

def convolve(star, psf):
    star_fft = fftpack.fftshift(fftpack.fftn(star))
    psf_fft = fftpack.fftshift(fftpack.fftn(psf))
    return fftpack.fftshift(fftpack.ifftn(fftpack.ifftshift(star_fft*psf_fft)))

def deconvolve(star, psf):
    star_fft = fftpack.fftshift(fftpack.fftn(star))
    psf_fft = fftpack.fftshift(fftpack.fftn(psf))
    return fftpack.fftshift(fftpack.ifftn(fftpack.ifftshift(star_fft/psf_fft)))



def mean2(x):
    y = np.sum(x) / np.size(x);
    return y

def corr2(a,b):
    a = a - mean2(a)
    b = b - mean2(b)

    r = (a*b).sum() / math.sqrt((a*a).sum() * (b*b).sum());
    return r

z = 5781 * 1e-3
z_0 = 1067 * mm
z_T = z + z_0
z_eff = z * z_0 / (z_T)
dX_1 = 13 * 1e-6

def deconvmethod(partiallycoherent, z, dX_1, profilewidth, pixis_centery_px, wavelength):

    
    #number of pixels
    n = partiallycoherent.shape[0]
    nx = n
    ny = nx


    # pixel size of the detector dX_1
    
    dY_1 = dX_1

    # 2D grid and axes at the CCD:
    #x = np.arange(-n/2, n/2, 1) * dX_1
    #y = np.arange(-n/2, n/2, 1) * dX_1
    x = np.arange(-n/2, n/2, 1)
    y = np.arange(-n/2, n/2, 1)
    X1_axis, Y1_axis = np.meshgrid(x*dX_1, y*dX_1, sparse=False)
    #X1_axis, Y1_axis = np.meshgrid(x, y, sparse=False)


    # "pixelsize" at the pinholes:
    dX_2 = wavelength*z/(n*dX_1)
    dY_2 = wavelength*z/(n*dY_1)

    # 2D grid and axes at the double pinholes:
    X2_axis, Y2_axis = np.meshgrid(x*dX_2, y*dY_2, sparse=False)


    
    #psf = gauss2d(X1_axis,Y1_axis,1.9*13*1e-6)
    #psf = psf / np.max(psf)
    #sigma_F_gamma_um = 24.7
    
    sigma_F_gamma_um_list = np.arange(10,60,1)
    fullycoherent_profile_min_list = []
    i = 0
    for sigma_F_gamma_um in sigma_F_gamma_um_list:

        sigma_F_gamma = sigma_F_gamma_um * 1e-6
        F_gamma = gauss2d(X1_axis/dX_1,Y1_axis/dY_1,sigma_F_gamma/dX_1)
        #psf = gauss2d(X1_axis,Y1_axis,1.9)

        # Restore Image using Richardson-Lucy algorithm
        #fullycoherent = restoration.richardson_lucy(partiallycoherent, psf, 5)
        fullycoherent = restoration.wiener(partiallycoherent, F_gamma, 1)
        #fullycoherent = np.real(deconvolve(partiallycoherent, psf))

        

        fullycoherent = fullycoherent / np.max(fullycoherent)

        partiallycoherent_profile = np.mean(partiallycoherent[pixis_centery_px-int(profilewidth/2):pixis_centery_px+int(profilewidth/2),:], axis=0)
        partiallycoherent_profile = normalize(partiallycoherent_profile)
        
        fullycoherent_profile = np.mean(fullycoherent[pixis_centery_px-int(profilewidth/2):pixis_centery_px+int(profilewidth/2),:], axis=0)
        fullycoherent_profile = normalize(fullycoherent_profile)
        
        fullycoherent_profile_min = np.min(fullycoherent_profile)
        fullycoherent_profile_min_list.append(fullycoherent_profile_min)

        #print('sigma_F_gamma=' + str(sigma_F_gamma_um) + ' fullycoherent_profile_min=' + str(fullycoherent_profile_min))

        if fullycoherent_profile_min<0:
            break
        i = i + 1
    sigma_F_gamma_um_opt = sigma_F_gamma_um_list[i-1]
    
    # fine-tuning in stepsize of 0.1
    
    sigma_F_gamma_um_list = np.arange(sigma_F_gamma_um_opt,sigma_F_gamma_um_opt+1,0.1)
    fullycoherent_profile_min_list = []
    i = 0
    for sigma_F_gamma_um in sigma_F_gamma_um_list:

        sigma_F_gamma = sigma_F_gamma_um * 1e-6
        F_gamma = gauss2d(X1_axis/dX_1,Y1_axis/dY_1,sigma_F_gamma/dX_1)
        #psf = gauss2d(X1_axis,Y1_axis,1.9)

        # Restore Image using Richardson-Lucy algorithm
        #fullycoherent = restoration.richardson_lucy(partiallycoherent, psf, 5)
        fullycoherent = restoration.wiener(partiallycoherent, F_gamma, 1)
        #fullycoherent = np.real(deconvolve(partiallycoherent, psf))

        

        fullycoherent = normalize(fullycoherent)

        partiallycoherent_profile = np.mean(partiallycoherent[pixis_centery_px-int(profilewidth/2):pixis_centery_px+int(profilewidth/2),:], axis=0)
        partiallycoherent_profile = normalize(partiallycoherent_profile)
        
        fullycoherent_profile = np.mean(fullycoherent[pixis_centery_px-int(profilewidth/2):pixis_centery_px+int(profilewidth/2),:], axis=0)
        fullycoherent_profile = normalize(fullycoherent_profile)
        
        fullycoherent_profile_min = np.min(fullycoherent_profile)
        fullycoherent_profile_min_list.append(fullycoherent_profile_min)

        #print('sigma_F_gamma=' + str(sigma_F_gamma_um) + ' fullycoherent_profile_min=' + str(fullycoherent_profile_min))

        if fullycoherent_profile_min<0:
            break
        i = i + 1
    sigma_F_gamma_um_opt = sigma_F_gamma_um_list[i-1]
    F_gamma = gauss2d(X1_axis/dX_1,Y1_axis/dY_1,sigma_F_gamma_um_opt*1e-6/dX_1)
    gamma = fftpack.fftshift(fftpack.ifftn(fftpack.ifftshift(F_gamma)))
    
    partiallycoherent_rec = np.abs(convolve(fullycoherent,F_gamma))
    partiallycoherent_rec = normalize(partiallycoherent_rec)
    partiallycoherent_rec_profile = np.mean(partiallycoherent_rec[pixis_centery_px-int(profilewidth/2):pixis_centery_px+int(profilewidth/2),:], axis=0)
    partiallycoherent_rec_profile = normalize(partiallycoherent_rec_profile)
    cor = corr2(partiallycoherent, partiallycoherent_rec)
    
    print('sigma_F_gamma_opt=' + str(sigma_F_gamma_um_opt) + ' fullycoherent_profile_min=' + str(fullycoherent_profile_min_list[i-1]) + '\\' + ' correlation partiallycohernet measurement and reconstruction = ' + str(round(cor*100,2)) + '%')


    xdata = list(range(n))
    ydata = fullycoherent[pixis_centery_px,:]
    ydata = ydata / np.max(ydata)

    A_bp = fftpack.fftshift(fftpack.ifftn(fftpack.ifftshift(np.sqrt(partiallycoherent))))  # amplitude
    I_bp = np.abs(A_bp)**2  # intensity
    
    abs_gamma = np.abs(gamma)
    abs_gamma = abs_gamma / np.max(abs_gamma)
    xdata = list(range(n))
    ydata = abs_gamma[int(n/2),:]
    p0 = (int(n/2), 1)
    popt_gauss, pcov_gaussian = curve_fit(lambda x, m, w: gaussianbeam(x, 1, m ,w, 0), xdata, ydata, p0)
    xi_px = popt_gauss[1]/2
    xi_um = xi_px * dX_2 * 1e6
    print('coherence length xi/um = ' + str(xi_um))
    
    return(partiallycoherent_profile, fullycoherent, fullycoherent_profile,  partiallycoherent_rec, partiallycoherent_rec_profile, sigma_F_gamma_um_opt, F_gamma, abs_gamma, xi_um, I_bp, dX_2, cor)


def simulation_old(x, shiftx_um, _lambda_nm, z_mm, d_um, w1_um,w2_um,I_w1,I_w2, I_Airy1, I_Airy2, sep_factor, gamma):

    def Airy(x,a,_lambda,z,d):
        Z = (1.22 * math.pi/_lambda) * a * (x-d)/z
        f = (math.pi/_lambda)*(a**2/z)* (scipy.special.jv(1,Z)/Z)
        return f

    # Young's double pinholes experiment simulation

    #% Parameters are from Andrej's thesis

    # PIXIS 1024B: 13um pixelsize, 1024px, 13.3mmx13.3mm chip size

    _lambda = _lambda_nm*1e-9# wavelength
    w1 = w1_um*1e-6  # first slit width
    w2 = w2_um*1e-6  # second slit width
    z = z_mm*1e-3  # pinholess to detector #5896
    #gamma# degree of spatial coherence
    d = d_um*1e-6 # slit separation
    
    #R = 1 / dx# rescale factor

    #% Intensity simulation

    shiftx = shiftx_um*1e-6

    k = 2 * math.pi / _lambda
    theta = - (k * (d * (x-shiftx) / z))
    Partial_Coherence_Factor = gamma * np.cos(theta)

    # The amplitudes of the wave field in the far field:

    #Amp_1 = np.sqrt(I_w1) * Airy((x-shiftx), w1, _lambda, z, -d/2)
    #Amp_2 = np.sqrt(I_w2) * Airy((x-shiftx), w2, _lambda, z, +d/2)
    
    Amp_1 = np.sqrt(I_w1) * Airy((x-shiftx), w1, _lambda, z, -d/2)
    Amp_2 = np.sqrt(I_w2) * Airy((x-shiftx), w2, _lambda, z, +d/2)

    #I_D = (Amp_1**2 + Amp_2**2) + 2 * abs(Amp_1) * abs(Amp_2) * Partial_Coherence_Factor
    I_D = (Amp_1**2 + Amp_2**2) + 2 * abs(Amp_1) * abs(Amp_2) * Partial_Coherence_Factor + np.sqrt(I_Airy1) * Airy((x-shiftx), w1, _lambda, z, -d/2 * sep_factor) + np.sqrt(I_Airy2) * Airy((x-shiftx), w2, _lambda, z, +d/2 * sep_factor)
    I_D_normalized = I_D / np.max(I_D)

    return I_D_normalized


def simulation(x, shiftx_um, _lambda_nm, z_mm, d_um, w1_um,w2_um,I_w1,I_w2, I_Airy1, I_Airy2, sep_factor, gamma):

    def Airy(x,a,_lambda,z,d):
        Z = (1.22 * math.pi/_lambda) * a * (x-d)/z
        f = (math.pi/_lambda)*(a**2/z)* (scipy.special.jv(1,Z)/Z)
        return f

    # Young's double pinholes experiment simulation

    #% Parameters are from Andrej's thesis

    # PIXIS 1024B: 13um pixelsize, 1024px, 13.3mmx13.3mm chip size

    _lambda = _lambda_nm*1e-9# wavelength
    w1 = w1_um*1e-6  # first slit width
    w2 = w2_um*1e-6  # second slit width
    z = z_mm*1e-3  # pinholess to detector #5896
    #gamma# degree of spatial coherence
    d = d_um*1e-6 # slit separation
    
    #R = 1 / dx# rescale factor

    #% Intensity simulation

    shiftx = shiftx_um*1e-6

    k = 2 * math.pi / _lambda
    theta = - (k * (d * (x-shiftx) / z))
    Partial_Coherence_Factor = gamma * np.cos(theta/2)**2

    # The amplitudes of the wave field in the far field:

    #Amp_1 = np.sqrt(I_w1) * Airy((x-shiftx), w1, _lambda, z, -d/2)
    #Amp_2 = np.sqrt(I_w2) * Airy((x-shiftx), w2, _lambda, z, +d/2)
    
    Amp_1 = np.sqrt(I_w1) * Airy((x-shiftx), w1, _lambda, z, -d/2)
    Amp_2 = np.sqrt(I_w2) * Airy((x-shiftx), w2, _lambda, z, +d/2)

    #I_D = (Amp_1**2 + Amp_2**2) + 2 * abs(Amp_1) * abs(Amp_2) * Partial_Coherence_Factor
    I_D = 0 \
    + 0 * (Amp_1**2 + Amp_2**2) \
    + 2 * abs(Amp_1) * abs(Amp_2) * Partial_Coherence_Factor \
    + 0 * np.sqrt(I_Airy1) * Airy((x-shiftx), w1, _lambda, z, -d/2) \
    + 0 * np.sqrt(I_Airy2) * Airy((x-shiftx), w2, _lambda, z, +d/2) \
    + np.sqrt(I_Airy1) * Airy((x-shiftx), w1, _lambda, z, -d/2 * sep_factor)**2 \
    + np.sqrt(I_Airy2) * Airy((x-shiftx), w2, _lambda, z, +d/2 * sep_factor)**2
    I_D_normalized = I_D / np.max(I_D)

    return I_D_normalized


#pixis_profile_centerx_px_fit = [None] * 100

def plot_data_and_simulation(run_plot_data_and_simulation,
                             imageid,
                             use_pixis_avg,
                             imageid_max,
                             pixis_rotation,
                             #pixis_rotation_save,
                             #pixis_rotation_load,
                             pixis_bgfactor,
                             pixis_avg_width,
                             pinholes_rotation,
                             pinholes_bgfactor,
                             pinholes_avg_width,
                             _lambda_nm,
                             w1_um,
                             w2_um,
                             I_w1,
                             I_w2,
                             z_mm,
                             I_Airy1, I_Airy2, sep_factor,
                             gamma,
                             d_um,
                             aperture5_mm_bg,
                             aperture7_mm_bg,
                             aperture5_mm_image,
                             aperture7_mm_image,
                             filter_bg,
                             filter_image,
                             use_pixis_profile_avg_centerx,
                             pixis_center_autofind,
                             pixis_centerx_px,
                             pixis_centery_px,
                             shiftx_um_adjust,
                             pinholes_centerx_px,
                             pinholes_centery_px,
                             pinholes_avg_width_alongy,
                             pinholes_avg_width_alongx,
                             ap5_x_px,
                             ap5_y_px,
                             ap5_width_px,
                             ap5_height_px,
                             screenhole_x_px,
                             screenhole_y_px,
                             screenhole_width_px,
                             screenhole_height_px,
                             xlim_lower,
                             xlim_upper,
                             plot_lineout,
                             plot_simulation,
                             plot_simulation_fit,
                             plot_LightPipes_simulation,
                             plot_pixis_std_contour,
                             levels_auto,
                             levels_min,
                             levels_max,
                             zoomwidth,
                             zoomborderleft,
                             zoomborderright,
                             savefigure,
                            savefigure_all_images,
                            PlotTextbox,
                            initialize_fits_list,
                            cropforbp):
    
    global pixis_profile_centerx_px_fit
    global pinholes_cm_x_px
    global pinholes_cm_y_px
    global _lambda_nm_fit
    global gamma_fit
    global pixis_image_norm_dataset
    global pixis_profile_avg_dataset
    global pixis_image_avg_norm
    global xi_um
    global xi_um_of_average
    global sigma_F_gamma_um_opt
    global sigma_F_gamma_um_opt_of_average
    
    
    if run_plot_data_and_simulation == True:
        

        # Dataframe for this dataset:

        #df = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]
        
#         if pixis_rotation_save == True:
#             df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_rotation'] = pixis_rotation
#         if pixis_rotation_load == True:
#             pixis_rotation = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_rotation']
#             pixis_rotation_widget.value = pixis_rotation
            
        # use df_all to get the values, since bg images are not in the combined df0 consisting df_all and df_settings!
        imageid_sequence_by_energy_hall = get_imageids_with_bgs(beamposition_horizontal_interval_widget.value)
        #df = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2]) & df_all['imageid'].isin(imageid_sequence_by_energy_hall)]
        df = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & df0['imageid'].isin(imageid_sequence_by_energy_hall)]
        
        
        fits_list_len = len(df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].index) + 1
        
        
        print('# images for this DPH: ' + str(fits_list_len))
        
        if use_pixis_avg == True:
            imageid = [-1]
        
        
        #with plot_data_and_simulation_interactive_output:
         #   print('running')
        # Initialize datset fits lists:
        
        if initialize_fits_list == True:
            pixis_profile_centerx_px_fit = [None] * (fits_list_len)
            pinholes_cm_x_px = [None] * (fits_list_len)
            pinholes_cm_y_px = [None] * (fits_list_len)
            _lambda_nm_fit = [None] * (fits_list_len)
            gamma_fit = [None] * (fits_list_len)
            pixis_image_norm_dataset = [None] * (fits_list_len)
            pixis_profile_avg_dataset = [None] * (fits_list_len)
        
        if savefigure_all_images == True:
            #imageid_sequence = range(imageid_max+1)
            imageid_sequence = imageid_sequence_by_energy_hall
            imageid_sequence.append(-1)
            
        else:
            if use_pixis_avg == True:
                imageid_sequence = [-1]
            else:
                imageid_sequence = [imageid]
        
        wavelength_nm = _lambda_nm
        slitwidth1_um = w1_um
        slitwidth2_um = w2_um
        separation_um = d_um
        z_mm = z_mm
            

#         pixis_image_norm_dataset = []
#         pixis_profile_avg_dataset = []
        for imageid_loop in imageid_sequence:
            
            print('imageid_loop=' + str(imageid_loop))
            print('imageid_sequence=' + str(imageid_sequence))
            if imageid_loop == -1:
                beamposx = df['beam position hall horizontal pulse resolved'].mean(axis=0)
                beamposy = df['beam position hall vertical pulse resolved'].mean(axis=0)
                energy_hall_uJ = df['energy hall'].mean(axis=0)
            else:
                beamposx = df[df['imageid']==imageid_loop]['beam position hall horizontal pulse resolved']
                beamposy = df[df['imageid']==imageid_loop]['beam position hall vertical pulse resolved']
                energy_hall_uJ = df[df['imageid']==imageid_loop]['energy hall'].iloc[0]
            
            plt.ioff()
            
            

            hdf5_file_name_image = hdf5_file_name_image_widget.value
            dataset_image_args = dataset_image_args_widget.value
            ph = dataset_image_args[2]

            global pixis_image_norm
            global pixis_profile
            global pixis_profile_avg
            global pixis_profile_alongy
            
            
            (pixis_image_norm, pixis_bg_std_norm, pixis_bg_std_avg, pixis_profile, pixis_profile_avg,
                 pixis_profile_avg_centerx_px, pixis_centerx_px, pixis_centery_px, pixis_profile_alongy,
                 pixis_cts, pixis_cm_x_px, pixis_cm_y_px) = get_pixis_profiles(imageid_loop,
                                                                               use_pixis_avg,
                                                                            pixis_bgfactor,
                                                                            pixis_avg_width,
                                                                            pixis_rotation,
                                                                            pixis_center_autofind,
                                                                            pixis_centerx_px,
                                                                            pixis_centery_px,
                                                                              cropforbp)

            
            
            #pixis_image_norm_dataset.append(pixis_image_norm)  # how to access later. these are sorted by energy ...
            #pixis_profile_avg_dataset.append(pixis_profile_avg)
            
            pixis_image_norm_dataset[imageid_loop] = pixis_image_norm
            pixis_profile_avg_dataset[imageid_loop] = pixis_profile_avg
            
            if use_pixis_profile_avg_centerx == True:
                shiftx_px =  pixis_profile_avg_centerx_px - pixis_profile_avg.size/2                
            else:
                shiftx_px =  pixis_centerx_px - pixis_profile_avg.size/2

            shiftx_um = shiftx_px * 13 + shiftx_um_adjust
            
            pixis_centerx_px_auto_widget.value = pixis_centerx_px
            #pixis_profile_avg_centerx_px_auto_widget.value = pixis_profile_avg_centerx_px

            (pinholes_image_norm, pinholes_profile_alongx,
             pinholes_profile_alongx_avg,
             pinholes_profile_alongy,
             pinholes_profile_alongx_avg,
             pinholes_cts, pinholes_cm_x_px[imageid_loop], pinholes_cm_y_px[imageid_loop]) = get_pinholes_profiles(imageid_loop,
                                                                  pinholes_bgfactor,
                                                                  pinholes_rotation,
                                                                  pinholes_centerx_px,
                                                                  pinholes_centery_px,
                                                                  pinholes_avg_width_alongy,
                                                                  pinholes_avg_width_alongx)
            
            df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid'] == imageid_loop)), 'pinholes_cm_x_px'] = pinholes_cm_x_px[imageid_loop]
            df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid'] == imageid_loop)), 'pinholes_cm_y_px'] = pinholes_cm_y_px[imageid_loop]
            
            

            # make a lower and upper profile along x to determine the proper rotation
            
            n = pixis_profile_avg.size # number of sampling point  # number of pixels
            dx = 13*1e-6 # sampling # pixel size
            xdata = list(range(n))
            ydata = pixis_profile_alongy
           
            if plot_simulation_fit == True:
                pixis_yshift_px = pixis_centery_px
                p0 = (pixis_yshift_px, 400)
                popt_gauss, pcov_gaussian = curve_fit(lambda x, m, w: gaussianbeam(x, 1, m ,w, 0), xdata, ydata, p0)
                pixis_beamwidth_px = popt_gauss[1]  # this is 2 sigma!
                pixis_centery_px = popt_gauss[0]
            else:
                pixis_beamwidth_px = 100
                     
            pixis_beamwidth_px
            pixis_uppery_px = pixis_centery_px + int(pixis_beamwidth_px/2) 
            pixis_lowery_px = pixis_centery_px - int(pixis_beamwidth_px/2)  # smaller pixel coordinate, but in upper part of image

            pixis_profile_avg_lower = get_pixis_profiles(imageid_loop,
                                                         use_pixis_avg,
                                                        pixis_bgfactor,
                                                        30,
                                                        pixis_rotation,
                                                        pixis_center_autofind,
                                                        pixis_centerx_px,
                                                        pixis_lowery_px,
                                                        cropforbp)[4]
            
            pixis_profile_avg_upper = get_pixis_profiles(imageid_loop,
                                                         use_pixis_avg,
                                                        pixis_bgfactor,
                                                        30,
                                                        pixis_rotation,
                                                        pixis_center_autofind,
                                                        pixis_centerx_px,
                                                        pixis_uppery_px,
                                                        cropforbp)[4]
            
            ydata_lower = pixis_profile_avg_lower
            ydata_upper = pixis_profile_avg_upper
            
            
            xdata = dx * np.linspace(-n/2+1, n/2, n)# coordinate
            ydata = pixis_profile_avg
            
            # Fitting of wavelength and shiftx_um
            if plot_simulation_fit == True:
                
                p0 = (shiftx_um,_lambda_nm,gamma)
                p0 = (shiftx_um,_lambda_nm,gamma) # fit for a gamma=1
                #bounds = ([dx*(-n/2+1)/um,_lambda_nm-1], [dx*n/2/um,_lambda_nm+1])
                #bounds = ([dx*(-n/2+1)/um,_lambda_nm-1,0], [dx*n/2/um,_lambda_nm+1,1]) 
                bounds = ([dx*(-n/2+1)/um,_lambda_nm-1,gamma], [dx*n/2/um,_lambda_nm+1,gamma]) # fit for a gamma given
                popt, pcov = curve_fit(lambda x,shiftx_um,_lambda_nm, gamma: simulation(x, shiftx_um, _lambda_nm, z_mm, d_um, w1_um,w2_um,I_w1,I_w2, I_Airy1, I_Airy2, sep_factor,gamma),xdata, ydata, p0)
                
                
                
                popt_lower, pcov_lower = curve_fit(lambda x,shiftx_um,_lambda_nm, gamma: simulation(x, shiftx_um, _lambda_nm, z_mm, d_um, w1_um,w2_um,I_w1,I_w2, I_Airy1, I_Airy2, sep_factor,gamma),xdata, ydata_lower, p0)
                popt_upper, pcov_upper = curve_fit(lambda x,shiftx_um,_lambda_nm, gamma: simulation(x, shiftx_um, _lambda_nm, z_mm, d_um, w1_um,w2_um,I_w1,I_w2, I_Airy1, I_Airy2, sep_factor,gamma),xdata, ydata_upper, p0)

                shiftx_um_fit = popt[0]
                shiftx_um_fit_lower = popt_lower[0]
                shiftx_um_fit_upper = popt_upper[0]
                _lambda_nm_fit[imageid_loop] = popt[1]
                df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid'] == imageid_loop)), '_lambda_nm_fit'] = _lambda_nm_fit[imageid_loop]
                gamma_fit[imageid_loop] = popt[2]
                

                I_D_normalized_fit = simulation(xdata, shiftx_um_fit, _lambda_nm_fit[imageid_loop], z_mm, d_um, w1_um,w2_um,I_w1,I_w2,I_Airy1, I_Airy2, sep_factor, gamma_fit[imageid_loop])
            else:
                shiftx_um_fit = shiftx_um
                shiftx_um_fit_lower = 0
                shiftx_um_fit_upper = 0
                _lambda_nm_fit[imageid_loop] = _lambda_nm
                gamma_fit[imageid_loop] = gamma
                
                
                
           
                
            pixis_profile_centerx_px_fit[imageid_loop] = n/2 + shiftx_um_fit/13
            pixis_profile_centerx_px_fit_lower = n/2 + shiftx_um_fit_lower/13
            pixis_profile_centerx_px_fit_upper = n/2 + shiftx_um_fit_upper/13
            pixis_profile_centerx_px_fit_delta = pixis_profile_centerx_px_fit_lower - pixis_profile_centerx_px_fit_upper
 

            a1 = 1
            a2 = 1
            b1=1
            b2=1
            c1=1
            c2=1
            e1=1
            e2=1
            I_D_normalized = simulation(xdata, shiftx_um, _lambda_nm, z_mm, d_um, w1_um,w2_um,I_w1,I_w2, I_Airy1, I_Airy2, sep_factor,gamma)
            

            do_deconvmethod = True
            if do_deconvmethod == True:
                
                z = 5781 * 1e-3
                z_0 = 1067 * mm
                z_T = z + z_0
                z_eff = z * z_0 / (z_T)
                dX_1 = 13 * 1e-6
                
                partiallycoherent = pixis_image_norm
                partiallycoherent_profile, fullycoherent, fullycoherent_profile, partiallycoherent_rec_profile, partiallycoherent_rec_profile, sigma_F_gamma_um_opt, F_gamma, abs_gamma, xi_um, I_bp, dX_2, cor = deconvmethod(partiallycoherent, z, dX_1, pixis_avg_width, int(pixis_centery_px),_lambda_nm_fit[imageid_loop]*1e-9)
                df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid'] == imageid_loop)), 'sigma_F_gamma_um_opt'] = sigma_F_gamma_um_opt
                df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid'] == imageid_loop)), 'xi_um'] = xi_um
                df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid'] == imageid_loop)), 'pixis_profile_centerx_px_fit'] = pixis_profile_centerx_px_fit[imageid_loop]
                
                if imageid_loop == -1:
                    sigma_F_gamma_um_opt_of_average = sigma_F_gamma_um_opt
                    xi_um_of_average = xi_um


                #A_bp = fftpack.fftshift(fftpack.ifftn(fftpack.ifftshift(np.sqrt(partiallycoherent))))  # amplitude
                #I_bp = np.abs(A_bp)**2  # intensity


                #imagesc(X2_axis*R_1,Y2_axis*R_1,log10(I_bp));


                #pixis_yshift_px = int(pixis_centery_px_widget.value)
                #p0 = (pixis_yshift_px, 400)
                #popt_gauss, pcov_gaussian = curve_fit(lambda x, m, w: gaussianbeam(x, 1, m ,w, 0), xdata, ydata, p0)
                #pixis_beamwidth_px = popt_gauss[1]

                xdata = list(range(n))

                fig=plt.figure(figsize=(48, 12), dpi= 80, facecolor='w', edgecolor='k')
                plt.subplot(1,4,1)
                plt.plot(xdata, partiallycoherent_profile, 'b-', label='measured partially coherent', linewidth=3)
                plt.plot(xdata, fullycoherent_profile, 'r-', label='recovered fully coherent')
                plt.plot(xdata, partiallycoherent_rec_profile, 'g-', label='recovered partially coherent')
                #plt.plot(xdata, gaussianbeam(xdata, 1, popt_gauss[0] ,popt_gauss[1], 0), 'r-', label='fit: m=%5.1f px, w=%5.1f px' % tuple([popt_gauss[0] ,popt_gauss[1]]))
                plt.axhline(0, color='k')
                plt.legend()
                
                plt.title('coherence length $\\xi$ / $\mu$m = ' + str(round(xi_um,2)))

                plt.subplot(1,4,2)
                #plt.contourf(x,y,psf,cmap='jet')
                #plt.xlim(-5,5)
                #plt.ylim(-5,5)
                plt.imshow(F_gamma,cmap='jet', extent=((-n/2)*dX_1, (+n/2-1)*dX_1, -n/2*dX_1, (+n/2-1)*dX_1))
                #plt.xlim(-5*dX_1,5*dX_1)
                #plt.ylim(-5*dX_1,5*dX_1)
                plt.title('$F(\\gamma)$ with $\sigma$ = ' + str(round(sigma_F_gamma_um_opt,2)))


                plt.subplot(1,4,3)
                #plt.xlim(-5,5)
                #plt.ylim(-5,5)
                #plt.imshow(gamma,cmap='jet', extent=((-n/2)*dX_2, (+n/2-1)*dX_2, -n/2*dX_2, (+n/2-1)*dX_2))
                plt.contourf(abs_gamma,cmap='jet', extent=((-n/2)*dX_2, (+n/2-1)*dX_2, -n/2*dX_2, (+n/2-1)*dX_2))

            
                n = abs_gamma.shape[0]
                xdata = list(range(n))
                ydata = abs_gamma[int(n/2),:]
                p0 = (int(n/2), 1)
                popt_gauss, pcov_gaussian = curve_fit(lambda x, m, w: gaussianbeam(x, 1, m ,w, 0), xdata, ydata, p0)
                plt.subplot(1,4,4)
                plt.plot(xdata, ydata, 'b-', label='abs(gamma)', linewidth=1)
                plt.plot(xdata, gaussianbeam(xdata, 1, popt_gauss[0] ,popt_gauss[1], 0), 'r-', label='fit: m=%5.1f px, w=%5.1f px' % tuple([popt_gauss[0] ,popt_gauss[1]]))
                plt.legend()
                
                
                if savefigure == True:
                    #savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
                    savefigure_dir = scratch_dir + str(settings_widget.label)
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value))
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/deconv_a/'
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    plt.savefig(savefigure_dir + '/' + 'deconv_a_' + hdf5_file_name_image \
                            + '_ph_'+str(ph) \
                            + '_d_'+str(separation_um) \
                            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
                            + '_image_'+str(imageid_loop) \
                            + '.png', dpi=None, facecolor='w', edgecolor='w',
                    orientation='portrait', papertype=None, format=None,
                    transparent=False, bbox_inches=None, pad_inches=0.1,
                    frameon=None)
                
                
                #### only the  profiles
                n = partiallycoherent_profile.shape[0]
                
                xdata = np.linspace((-n/2)*dX_1*1e3, (+n/2-1)*dX_1*1e3, n)

                fig=plt.figure(figsize=(11.69,8.27), dpi= 300, facecolor='w', edgecolor='k')  # A4 sheet in landscape
                plt.subplot(1,1,1)
                plt.plot(xdata, partiallycoherent_profile, 'b-', label='measured partially coherent', linewidth=3)
                plt.plot(xdata, fullycoherent_profile, 'r-', label='recovered fully coherent', linewidth=1)
                #plt.plot(xdata, gaussianbeam(xdata, 1, popt_gauss[0] ,popt_gauss[1], 0), 'r-', label='fit: m=%5.1f px, w=%5.1f px' % tuple([popt_gauss[0] ,popt_gauss[1]]))
                plt.axhline(0, color='k')
                plt.xlabel('x / mm', fontsize = 14)
                plt.ylabel('Intensity / a.u.', fontsize = 14)
                plt.legend()
                
                plt.title('d / $\mu$m = '+str(int(separation_um)) + ' coherence length $\\xi$ / $\mu$m = ' + str(round(xi_um,2)), fontsize=16)
             
                if savefigure == True:
                    savefigure_dir = scratch_dir + str(settings_widget.label)
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    #savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
                    savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value))
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/profiles/'
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    plt.savefig(savefigure_dir + '/' + 'profiles_' + hdf5_file_name_image \
                            + '_ph_'+str(ph) \
                            + '_d_'+str(separation_um) \
                            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
                            + '_image_'+str(imageid_loop) \
                            + '.png', dpi=300, facecolor='w', edgecolor='w',
                    orientation='portrait', papertype=None, format=None,
                    transparent=False, bbox_inches=None, pad_inches=0.1,
                    frameon=None)
                    plt.savefig(savefigure_dir + '/' + 'profiles_' + hdf5_file_name_image \
                            + '_ph_'+str(ph) \
                            + '_d_'+str(separation_um) \
                            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
                            + '_image_'+str(imageid_loop) \
                            + '.pdf', dpi=None, facecolor='w', edgecolor='w',
                    orientation='portrait', papertype=None, format=None,
                    transparent=False, bbox_inches=None, pad_inches=0.1,
                    frameon=None)

                #plt.show()
                plt.close(fig)

                                #### only the  profiles
                n = partiallycoherent_profile.shape[0]
                
                xdata = np.linspace((-n/2)*dX_1*1e3, (+n/2-1)*dX_1*1e3, n)

                fig=plt.figure(figsize=(11.69,8.27), dpi= 300, facecolor='w', edgecolor='k')  # A4 sheet in landscape
                ax = plt.subplot(1,1,1)
                plt.plot(xdata, partiallycoherent_profile, 'b-', label='measured partially coherent', linewidth=1)
                plt.plot(xdata, fullycoherent_profile, 'r-', label='recovered fully coherent', linewidth=1)
                plt.plot(xdata, partiallycoherent_rec_profile, 'g-', label='recovered partially coherent', linewidth=1)
                #plt.plot(xdata, gaussianbeam(xdata, 1, popt_gauss[0] ,popt_gauss[1], 0), 'r-', label='fit: m=%5.1f px, w=%5.1f px' % tuple([popt_gauss[0] ,popt_gauss[1]]))
                plt.axhline(0, color='k')
                plt.xlabel('x / mm', fontsize = 14)
                plt.ylabel('Intensity / a.u.', fontsize = 14)
                plt.legend()
                
                plt.title('d / $\mu$m = '+str(int(separation_um)) + ' coherence length $\\xi$ / $\mu$m = ' + str(round(xi_um,2)), fontsize=16)
                
                props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
                textstr = 'corr=' + str(round(cor*100,2)) + '%'
                ax.text(0.01, 0.99, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
             
                if savefigure == True:
                    savefigure_dir = scratch_dir + str(settings_widget.label)
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    #savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
                    savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value))
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/profiles_rec/'
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    plt.savefig(savefigure_dir + '/' + 'profiles_rec_' + hdf5_file_name_image \
                            + '_ph_'+str(ph) \
                            + '_d_'+str(separation_um) \
                            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
                            + '_image_'+str(imageid_loop) \
                            + '.png', dpi=300, facecolor='w', edgecolor='w',
                    orientation='portrait', papertype=None, format=None,
                    transparent=False, bbox_inches=None, pad_inches=0.1,
                    frameon=None)
                    plt.savefig(savefigure_dir + '/' + 'profiles_rec_' + hdf5_file_name_image \
                            + '_ph_'+str(ph) \
                            + '_d_'+str(separation_um) \
                            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
                            + '_image_'+str(imageid_loop) \
                            + '.pdf', dpi=None, facecolor='w', edgecolor='w',
                    orientation='portrait', papertype=None, format=None,
                    transparent=False, bbox_inches=None, pad_inches=0.1,
                    frameon=None)

                plt.show()
                #plt.close(fig)
                
                fig=plt.figure(figsize=(36, 16), dpi= 80, facecolor='w', edgecolor='k')
                plt.subplot(1,3,1)
                plt.imshow(partiallycoherent,origin='lower', interpolation='nearest', aspect=1, cmap='jet', vmin=0, vmax=1)
                plt.subplot(1,3,2)
                plt.imshow(fullycoherent,origin='lower', interpolation='nearest', aspect=1, cmap='jet', vmin=0, vmax=1)

                plt.subplot(1,3,3)
                #plt.xlim(-5,5)
                #plt.ylim(-5,5)
                #plt.imshow(gamma,cmap='jet', extent=((-n/2)*dX_2, (+n/2-1)*dX_2, -n/2*dX_2, (+n/2-1)*dX_2))
                n = partiallycoherent.shape[0]
                #plt.imshow(np.log10(I_bp),cmap='jet', extent=((-n/2)*dX_2, (+n/2-1)*dX_2, -n/2*dX_2, (+n/2-1)*dX_2))
                plt.imshow(np.log10(I_bp),cmap='jet')
                
                if savefigure == True:
                    savefigure_dir = scratch_dir + str(settings_widget.label)
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    #savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
                    savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value))
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/deconv_b/'
                    if os.path.isdir(savefigure_dir) == False:
                        os.mkdir(savefigure_dir)
                    plt.savefig(savefigure_dir + '/' + 'deconv_b_' + hdf5_file_name_image \
                            + '_ph_'+str(ph) \
                            + '_d_'+str(separation_um) \
                            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
                            + '_image_'+str(imageid_loop) \
                            + '.png', dpi=None, facecolor='w', edgecolor='w',
                    orientation='portrait', papertype=None, format=None,
                    transparent=False, bbox_inches=None, pad_inches=0.1,
                    frameon=None)
                plt.close(fig)
                
                
            
            
            # to be fixed:
            Error_min_idx = 0
        #     print(len(Error))
        #     if len(Error) > 0:
        #         Error_min_idx = np.where(Error==np.min(Error))[0][0]
        #     else:
        #         Error_min_idx=0
        #     #print('min. error=' + str(np.min(Error)) + 'at idx=' + str(Error_min_idx)
                  #+ ' for gamma=' + str(gamma_fit[Error_min_idx]))

            # Figure
            #fig = plt.figure(figsize=(48, 36))
            fig = plt.figure(figsize=(30, 18))
            

            gs = gridspec.GridSpec(4, 5, width_ratios=[1,1,1,1,1],height_ratios=[1, 2, 1, 2])
            gs.update(hspace=0.05)

            # Plot Profiles
            ax1 = plt.subplot(gs[0,0])
            if plot_lineout == True:
                ax1.scatter(list(range(pixis_profile.size)),pixis_profile,s=10,marker='+',color='c')
                ax1.scatter(np.where(pixis_profile<0)[0],pixis_profile[np.where(pixis_profile<0)[0]],s=10,marker='x',color='k')
            ax1.plot(list(range(pixis_profile_avg.size)),pixis_profile_avg, color='r')
            ax1.plot(list(range(pixis_profile_avg.size)),pixis_bg_std_avg, color='b')
           


            ax1.scatter(list(range(pixis_profile_avg.size)),pixis_profile_avg,s=15, color='r')

            if plot_simulation == True:
                ax1.plot(list(range(pixis_profile_avg.size)),I_D_normalized, color='k')
            if plot_LightPipes_simulation == True:
                ax1.plot(list(range(pixis_profile_avg.size)),Intlinex_LightPipes, 'g-')
            if plot_simulation_fit == True:
                ax1.plot(list(range(pixis_profile_avg.size)),I_D_normalized_fit, color='g')
            plt.ylim(-0.05,1)

            #plt.title(hdf5_file_name_image + 'cross section at y=' + str(centery) + ' \lambda=' + str(daq_parameter_image_dataset[0][imageid_widget.value]) + ' vs. ' + str(_lambda_nm) )

            ax1.hlines(0,0,pixis_profile_avg.size)

            # plot image

            ax2 = plt.subplot(gs[1,0], sharex=ax1)
            im_ax2 = plt.imshow(pixis_image_norm, origin='lower', interpolation='nearest', aspect='auto', cmap='jet', vmin=0, vmax=1)
            
            if plot_pixis_std_contour == True:
                if levels_auto == True:
                    CS = plt.contour(pixis_bg_std_norm,6,                                
                                 colors=('r', 'green', 'blue', (1, 1, 0), '#afeeee', '0.5'))
                else:
                    levels = np.arange(levels_min,levels_max,levels_max-levels_min/6)
                    CS = plt.contour(pixis_bg_std_norm,6,                                
                                 colors=('r', 'green', 'blue', (1, 1, 0), '#afeeee', '0.5'))
                plt.clabel(CS, fontsize=12, inline=1)
                CB = plt.colorbar(CS, ax=ax2, pad=0.05, fraction=0.1, shrink=1.00, aspect=20, orientation='horizontal')

            plot_center = True
            if plot_center == True:
                plt.scatter(pixis_centerx_px,pixis_centery_px, color='b')
                plt.scatter(pixis_cm_x_px,pixis_cm_y_px, color='r')

            props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

            if plot_simulation_fit == True:
                #xdata = dx * np.linspace(-n/2+1, n/2, n)# coordinate
                n = len(pixis_profile_alongy)
                xdata = list(range(n))
                
                ax52 = plt.subplot(gs[1,4])
                ax52.plot(xdata, pixis_profile_alongy, 'b-', label='data')
                ax52.plot(xdata, gaussianbeam(xdata, 1, pixis_centery_px ,pixis_beamwidth_px, 0), 'r-', label='fit: m=%5.1f px, w=%5.1f px' % tuple([pixis_centery_px ,pixis_beamwidth_px]))
                ax52.legend()
                
            ax54 = plt.subplot(gs[3,4])
            n = partiallycoherent.shape[0]
            #plt.imshow(np.log10(I_bp),cmap='jet', extent=((-n/2)*dX_2, (+n/2-1)*dX_2, -n/2*dX_2, (+n/2-1)*dX_2))
            ax54.imshow(np.log10(I_bp),cmap='jet')
            
            
            # place a text box in upper left in axes coords
            if plot_simulation_fit == True:
                textstr = settings_widget.label + '\n' \
                        + 'imageid =' + str(imageid_loop) + '\n' \
                        + '$\lambda=$' + str(format(_lambda_nm, '.2f')) + '\n' \
                        + '$\lambda fit=$' + str(format(_lambda_nm_fit[imageid_loop], '.2f')) + '\n' \
                        + 'sep = ' + str(separation_um) + 'um' + '\n' \
                        + 'ap5,ap7 (image) = ' + str(aperture5_mm_image) + 'mm, ' + str(aperture7_mm_image) + 'mm' + '\n' \
                        + 'ap5,ap7 (bg) = ' + str(aperture5_mm_bg) + 'mm, ' + str(aperture7_mm_bg) + 'mm' + '\n' \
                        + 'pixis_centerx_px = ' + str(pixis_centerx_px)  + '\n' \
                        + 'pixis_profile_centerx_px_fit = ' + str(pixis_profile_centerx_px_fit[imageid_loop]) + '\n' \
                        + 'pixis_profile_centerx_px_fit_lower = ' + str(pixis_profile_centerx_px_fit_lower) + '\n' \
                        + 'pixis_profile_centerx_px_fit_upper = ' + str(pixis_profile_centerx_px_fit_upper) + '\n' \
                        + 'pixis_profile_centerx_px_fit_delta = ' + str(pixis_profile_centerx_px_fit_delta) + '\n' \
                        + 'w1_um = ' +str(w1_um) + ' w2_um = ' +str(w2_um) + '\n' + ' I_w1 = ' +str(I_w1) + ' I_w2 = ' +str(I_w2) + '\n' \
                        + 'gamma = ' +str(gamma) + '\n' \
                        + 'gamma_fit = ' +str(format(gamma_fit[imageid_loop], '.2f')) + '\n' \
                        + 'energy hall uJ = ' + str(format(energy_hall_uJ, '.2f')) + '\n' \
                        + 'pixis cts = ' + str(format(pixis_cts, '.2f'))
            else:
                textstr = settings_widget.label + '\n' \
                        + 'imageid =' + str(imageid_loop) + '\n' \
                        + '$\lambda=$' + str(format(_lambda_nm, '.2f')) + '\n' \
                        + 'sep = ' + str(separation_um) + 'um' + '\n' \
                        + 'ap5,ap7 (image) = ' + str(aperture5_mm_image) + 'mm, ' + str(aperture7_mm_image) + 'mm' + '\n' \
                        + 'ap5,ap7 (bg) = ' + str(aperture5_mm_bg) + 'mm, ' + str(aperture7_mm_bg) + 'mm' + '\n' \
                        + 'pixis_centerx_px = ' + str(pixis_centerx_px)  + '\n' \
                        + 'pixis_profile_centerx_px_fit = ' + str(pixis_profile_centerx_px_fit[imageid_loop]) + '\n' \
                        + 'w1_um = ' +str(w1_um) + ' w2_um = ' +str(w2_um) + '\n' + ' I_w1 = ' +str(I_w1) + ' I_w2 = ' +str(I_w2) + '\n' \
                        + 'gamma = ' +str(gamma) + '\n' \
                        + 'energy hall uJ = ' + str(format(energy_hall_uJ, '.2f')) + '\n' \
#                         + 'pixis cts = ' + str(format(pixis_cts, '.2f'))
            
            print('_lambda_nm=' + str(_lambda_nm))    
            print('_lambda_nm_fit=' + str(_lambda_nm_fit[imageid_loop]))
            print('energy_hall_uJ=' + str(energy_hall_uJ))
            print('pixis_cts=' + str(pixis_cts))
            
            if PlotTextbox == True: 
                ax1.text(0.01, 0.99, textstr, transform=ax1.transAxes, fontsize=14, verticalalignment='top', bbox=props)

#             if plot_simulation == True or plot_simulation_fit == True:
#                 textstr = textstr + '\n' + '$\gamma=$' + str(format(gamma_fit[Error_min_idx], '.2f')) + ' (min error=' + str(format(np.min(Error), '.2f')) + ')'

                
            ax2.add_patch(
               patches.Rectangle(
                (0, pixis_centery_px-pixis_avg_width/2),
                pixis_profile.size,
                pixis_avg_width,
                color = 'w',
                linestyle = '--',
                fill=False      # remove background
                )
             )

            #to check the rotation:
            ax2.add_patch(
               patches.Rectangle(
                (0, pixis_lowery_px),
                pixis_profile.size,
                pixis_beamwidth_px,
                color = 'y',
                linestyle = '--',
                fill=False      # remove background
                )
             )

            #plt.title('background subtracted and rotation corrected (normalized)')

            if plot_pixis_std_contour == False:
                fig.colorbar(im_ax2, ax=ax2, pad=0.05, fraction=0.1, shrink=1.00, aspect=20, orientation='horizontal')

            #plt.xlim(0,pixis_profile.size)
            plt.xlim(xlim_lower,xlim_upper)

            #plt.tight_layout()

            plt.subplots_adjust(left=0.05, right=0.98, top=0.98, bottom=0.05)

            #fig.tight_layout(pad=0.0, w_pad=0.0, h_pad=0)

            axisx = list(range(pixis_profile_avg.size))
            ### zoomed in plots ###


            ax21 = plt.subplot(gs[2,1])
            plt.setp(ax21, frame_on=False, xticks=(), yticks=())
            ax21.text(0.01, 0.95, textstr, transform=ax21.transAxes, fontsize=12, verticalalignment='top')

            ## column1 left zoom
            left = zoomborderleft
            right = zoomborderleft+zoomwidth

            ax11 = plt.subplot(gs[0,1])
            ax11.plot(axisx[left:right],pixis_profile_avg[left:right], color='r')
            ax11.scatter(axisx[left:right],pixis_profile_avg[left:right],s=15,marker='o', color='r')

            if plot_simulation == True:
                ax11.plot(axisx[left:right],I_D_normalized[left:right], color='k')
            if plot_simulation_fit == True:
                ax11.plot(axisx[left:right],I_D_normalized_fit[left:right], color='g')
            if plot_LightPipes_simulation == True:
                ax11.plot(axisx[left:right],Intlinex_LightPipes[left:right], 'g-')

            #plt.title(hdf5_file_name_image + 'cross section at y=' + str(centery) + ' \lambda=' + str(daq_parameter_image_dataset[0][imageid_widget.value]) + ' vs. ' + str(_lambda_nm) )

            ax11.hlines(0,0,pixis_profile_avg.size)

            # plot image

            ax21 = plt.subplot(gs[1,1], sharex=ax11)
            #im_ax21 = plt.imshow(pixis_image_norm, origin='lower', interpolation='nearest')
            im_ax21 = ax21.imshow(pixis_image_norm, origin='lower', interpolation='nearest', aspect='auto', cmap='jet', vmin=0, vmax=1)
            
            fig.colorbar(im_ax21, ax=ax21, pad=0.05, fraction=0.1, shrink=1.00, aspect=20, orientation='horizontal')

            ax21.add_patch(
               patches.Rectangle(
                (0, pixis_centery_px-pixis_avg_width/2),
                pixis_profile.size,
                pixis_avg_width,
                color = 'w',
                linestyle = '--',
                fill=False      # remove background
                )
            )

            ax11.set_ylim([-0.05,1])
            ax11.set_xlim([left,right])




            ## column2 center zoom
            left = int(pixis_centerx_px)-int(zoomwidth/2)
            right = int(pixis_centerx_px)+int(zoomwidth/2)
            
            #left = int(pixis_profile_centerx_px_fit[imageid_loop])-int(zoomwidth/2)
#             right = int(pixis_profile_centerx_px_fit[imageid_loop])+int(zoomwidth/2)

            ax12 = plt.subplot(gs[0,2])
            ax12.plot(axisx[left:right],pixis_profile_avg[left:right], color='r')
            ax12.scatter(axisx[left:right],pixis_profile_avg[left:right],s=15,marker='o', color='r')

            if plot_simulation == True:
                ax12.plot(axisx[left:right],I_D_normalized[left:right], color='k')
            if plot_simulation_fit == True:
                ax12.plot(axisx[left:right],I_D_normalized_fit[left:right], color='g')
            if plot_LightPipes_simulation == True:
                ax12.plot(axisx[left:right],Intlinex_LightPipes[left:right], 'g-')

            #plt.title(hdf5_file_name_image + 'cross section at y=' + str(centery) + ' \lambda=' + str(daq_parameter_image_dataset[0][imageid_widget.value]) + ' vs. ' + str(_lambda_nm) )

            ax12.hlines(0,0,pixis_profile_avg.size)

            # plot image

            ax22 = plt.subplot(gs[1,2], sharex=ax12)
            im_ax22 = ax22.imshow(pixis_image_norm, origin='lower', interpolation='nearest', aspect='auto', cmap='jet', vmin=0, vmax=1)
            
            fig.colorbar(im_ax22, ax=ax22, pad=0.05, fraction=0.1, shrink=1.00, aspect=20, orientation='horizontal')

            ax22.add_patch(
               patches.Rectangle(
                (0, pixis_centery_px-pixis_avg_width/2),
                pixis_profile.size,
                pixis_avg_width,
                color = 'w',
                linestyle = '--',
                fill=False      # remove background
                )
            )

            ax12.set_ylim([-0.05,1])
            ax12.set_xlim([left,right])

            ## column3 right zoom
            left = pixis_profile_avg.size-zoomborderright-zoomwidth
            right = pixis_profile_avg.size-zoomborderright

            ax13 = plt.subplot(gs[0,3])
            ax13.plot(axisx[left:right],pixis_profile_avg[left:right], color='r')
            ax13.scatter(axisx[left:right],pixis_profile_avg[left:right],s=15,marker='o', color='r')

            if plot_simulation == True:
                ax13.plot(axisx[left:right],I_D_normalized[left:right], color='k')
            if plot_simulation_fit == True:
                ax13.plot(axisx[left:right],I_D_normalized_fit[left:right], color='g')
            if plot_LightPipes_simulation == True:
                ax13.plot(axisx[left:right],Intlinex_LightPipes[left:right], 'g-')

            #plt.title(hdf5_file_name_image + 'cross section at y=' + str(centery) + ' \lambda=' + str(daq_parameter_image_dataset[0][imageid_widget.value]) + ' vs. ' + str(_lambda_nm) )

            ax13.hlines(0,0,pixis_profile_avg.size)

            # plot image

            ax23 = plt.subplot(gs[1,3], sharex=ax13)
            im_ax23 = plt.imshow(pixis_image_norm, origin='lower', interpolation='nearest', aspect='auto', cmap='jet', vmin=0, vmax=1)
            
            fig.colorbar(im_ax23, ax=ax23, pad=0.05, fraction=0.1, shrink=1.00, aspect=20, orientation='horizontal')

            ax23.add_patch(
               patches.Rectangle(
                (0, pixis_centery_px-pixis_avg_width/2),
                pixis_profile.size,
                pixis_avg_width,
                color = 'w',
                linestyle = '--',
                fill=False      # remove background
                )
            )

            ax13.set_ylim([-0.05,1])
            ax13.set_xlim([left,right])

            ### PINHOLES ####

            # Plot Profiles
            ax20 = plt.subplot(gs[2,0])
            #if plot_lineout == True:
            #    ax3.scatter(list(range(pinholes_profile.size)),pinholes_profile,s=10,marker='+',color='c')
            #ax3.scatter(np.where(pinholes_profile<0)[0],pinholes_profile[np.where(pinholes_profile<0)[0]],s=10,marker='x',color='k')
            ax20.plot(list(range(pinholes_profile_alongx_avg.size)),pinholes_profile_alongx_avg, color='r')


            ax20.scatter(list(range(pinholes_profile_alongx_avg.size)),pinholes_profile_alongx_avg,s=15,marker='o', color='r')


            #plt.title(hdf5_file_name_image + 'cross section at y=' + str(centery) + ' \lambda=' + str(daq_parameter_image_dataset[0][imageid_widget.value]) + ' vs. ' + str(_lambda_nm) )

            #ax1.hlines(0,0,pixis_profile_avg.size)

            # plot image

            ax4 = plt.subplot(gs[3,0], sharex=ax20)
            im_ax4 = plt.imshow(pinholes_image_norm, origin='lower', interpolation='nearest', aspect='auto', cmap='nipy_spectral', vmin=0, vmax=1)
            
            plot_center = False
            #if plot_center == True:
            #    plt.scatter(centerx,centery, color='r')
            #ax4.scatter(pinholes_cm_x_px, pinholes_cm_y_px, c=list(df['energy hall']), marker='o', s=5, cmap='jet')
            #ax4.scatter(pinholes_cm_x_px[imageid_loop], pinholes_cm_y_px[imageid_loop], marker='x', s=10, color='k')

#             ax4.add_patch(
#                patches.Rectangle(
#                 (0, pinholes_centery_px-pinholes_avg_width_alongy/2),
#                 pinholes_profile_alongx.size,
#                 pinholes_avg_width_alongy,
#                 color = 'w',
#                 linestyle = '--',
#                 fill=False      # remove background
#                 )
#             )
#             ax4.add_patch(
#                patches.Ellipse(
#                 (ap5_x_px, ap5_y_px), 
#                 ap5_width_px, 
#                 ap5_height_px,
#                 color = 'w',
#                 linestyle = '--',
#                 fill=False      # remove background
#                 )
#             )    
#             ax4.add_patch(
#                patches.Ellipse(
#                 (screenhole_x_px, screenhole_y_px), 
#                 screenhole_width_px, 
#                 screenhole_height_px,
#                 color = 'w',
#                 linestyle = '--',
#                 fill=False      # remove background
#                 )
#             )

            #plt.title('background subtracted and rotation corrected (normalized)')

            fig.colorbar(im_ax4, ax=ax4, pad=0.05, fraction=0.1, shrink=1.00, aspect=20, orientation='horizontal')

            #plt.xlim(0,pixis_profile.size)
            #plt.xlim(xlim_lower,xlim_upper)

            #plt.tight_layout()

            plt.subplots_adjust(left=0.05, right=0.98, top=0.98, bottom=0.05)

            #fig.tight_layout(pad=0.0, w_pad=0.0, h_pad=0)

            
            ### DATASET PLOTS ####
            
            ax31 = plt.subplot(gs[3,1])
        
            df.plot(kind='scatter', x='beam position hall horizontal pulse resolved', y='beam position hall vertical pulse resolved', c='xi_um', s=10, colormap='jet', ax=ax31)
            ax31.scatter(beamposx, beamposy, marker='x', s=60, color='k')

#             ax33.scatter(pinholes_cm_x_px,pinholes_cm_x_px, c=list(df['energy hall']), s=10, cmap='jet')
#             ax33.scatter(pinholes_cm_x_px[imageid_loop],pinholes_cm_y_px[imageid_loop], marker='x', s=60, color='k')

            ax31.set_aspect('equal')
            
            ax32 = plt.subplot(gs[3,2])
            
            

            df.plot(kind='scatter', x='beam position hall horizontal pulse resolved', y='beam position hall vertical pulse resolved', c='energy hall', s=10, colormap='jet', ax=ax32)
            ax32.scatter(beamposx, beamposy, marker='x', s=60, color='k')
            ax32.set_aspect('equal')

#             ax5.scatter(range(imageid_max),pixis_profile_centerx_px_fit, marker='.', color='r', label='pixis profile center / px')
#             ax5.scatter(imageid_loop,pixis_profile_centerx_px_fit[imageid_loop],  marker='.', color='k')
#             ax5.scatter(range(imageid_max),pinholes_cm_x_px, marker='o', color='b', label='pinholes CM x / px')
#             ax5.scatter(imageid_loop,pinholes_cm_x_px[imageid_loop], marker='o', color='k')
#             ax5.legend()
#             ax5.set_xlabel('image id')
            
            ax33 = plt.subplot(gs[3,3])
        
            df.plot(kind='scatter', x='pinholes_cm_x_px', y='pinholes_cm_y_px', c='energy hall', s=10, colormap='jet', ax=ax33)

            #ax33.scatter(pinholes_cm_x_px,pinholes_cm_x_px, c=list(df['energy hall']), s=10, cmap='jet')
            ax33.scatter(pinholes_cm_x_px[imageid_loop],pinholes_cm_y_px[imageid_loop], marker='x', s=60, color='k')
            ax33.set_xlabel('pinholes CM x / px')
            ax33.set_ylabel('pinholes CM y / px')
            ax33.set_aspect('equal')
            

            
            

            if savefigure == True:
                #savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
                
                savefigure_dir = scratch_dir + str(settings_widget.label)
                if os.path.isdir(savefigure_dir) == False:
                    os.mkdir(savefigure_dir)
                savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value))
                if os.path.isdir(savefigure_dir) == False:
                    os.mkdir(savefigure_dir)
                savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/overview/'
                if os.path.isdir(savefigure_dir) == False:
                    os.mkdir(savefigure_dir)
                plt.savefig(savefigure_dir + '/'+ hdf5_file_name_image \
                        + '_ph_'+str(ph) \
                        + '_d_'+str(separation_um) \
                        + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
                        + '_image_'+str(imageid_loop) \
                        + '.png', dpi=None, facecolor='w', edgecolor='w',
                orientation='portrait', papertype=None, format=None,
                transparent=False, bbox_inches=None, pad_inches=0.1,
                frameon=None)
                
                
                
                # save to .mat for deconvolution in MATLAB
                pixis_image_norm_square = pixis_image_norm[0:min(pixis_image_norm.shape),0:min(pixis_image_norm.shape)]
                # Create a dictionary
                adict = {}
                adict['pixis_image_norm'] = pixis_image_norm_square
                adict['pinholes_image_norm'] = pinholes_image_norm
               #adict['wpg_image_norm_square'] = wpg_image_norm_square
                
                if imageid == -1:
                    sio.savemat(savefigure_dir + '/'+ hdf5_file_name_image \
                            + '_ph_'+str(ph) \
                            + '_d_'+str(separation_um) \
                            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
                            + '_average'
                            + '.mat', adict)
                else:
                    sio.savemat(savefigure_dir + '/'+ hdf5_file_name_image \
                            + '_ph_'+str(ph) \
                            + '_d_'+str(separation_um) \
                            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
                            + '_image_'+str(imageid_loop) 
                            + '.mat', adict)
                
                #f = h5py.File(savefigure_dir + '.hdf5', 'w')
                
                

            #if savefigure_all_images == False:
                #plt.show(fig)
                #with plot_data_and_simulation_interactive_output:

                #display(fig)
                
                #plt.show(fig)
            
            clear_output(wait=True)
            display(fig)
            plt.close(fig)

        #pixis_image_avg_norm = np.average(pixis_image_norm_dataset, axis=0)
        #pixis_image_avg_norm = pixis_image_avg_norm / np.max(pixis_image_avg_norm)
        
#         if savefigure == True:

#             # save to .mat for deconvolution in MATLAB
#             import scipy.io as sio
#             # Create a dictionary
#             adict = {}
#             adict['pixis_image_norm'] = pixis_image_norm
#             adict['pixis_image_avg_norm'] = pixis_image_avg_norm
#             #adict['wpg_image_norm_square'] = wpg_image_norm_square
#             sio.savemat(savefigure_dir + '/'+ hdf5_file_name_image \
#                     + '_ph_'+str(ph) \
#                     + '_d_'+str(separation_um) \
#                     + '_E_' + str(format(energy_hall_uJ, '.2f')).zfill(6)  \
#                     + '_average'+ \
#                     + '.mat', adict)


%matplotlib inline

m=1
nm=1e-9*m
um=1e-6*m
mm=1e-3*m
cm=1e-2*m

rescale = 2
N=rescale*1024
#N=1024
pixelsize=13*um / rescale
px=pixelsize
size=N*pixelsize
LP=LightPipes.Init()

#PinholesAtoPinholesB = 7.364*m
#PinholesBtoCCD = 6.02450*m
#PinholesBtoCCD = 5.103*m    # old
PinholesBtoCCD = 5.781*m    # old


PinholesAtoCCD =  12.47671*m

# from KBH to PMMA = 2000*mm
# from PMMA to Aperture7 = 697.5*mm
# from KBH to Aperture7 = 2697.5*mm
# from Aperture7 to PinholesB = 370*mm

#z0to1=0.326*m # Pinholes-A
z0to1=0.371*m # Pinholes-B
#z3to4=PinholesAtoCCD # pinhole before KAOS
z3to4=PinholesBtoCCD # pinhole after KAOS
z=z3to4

def beamFWHM_mm(wavelength_nm):
    return (0.2*wavelength_nm+1.0);

def beam2w_mm(wavelength_nm):
    return beamFWHM_mm(wavelength_nm) * 1.699;

def separation_5px_m(wavelength_nm):
    return wavelength_nm*nm / (5*px) *z;

def magnificationFWHM(wavelength_nm):
    return (separation_5px_m(wavelength_nm)/mm)/beamFWHM_mm(wavelength_nm);

def magnification2w(wavelength_nm):
    m = (separation_5px_m(wavelength_nm)/mm)/beam2w_mm(wavelength_nm)
    return m;

def FresnelNumber(slitwidth_um,wavelength_nm,z_m):
    FN=(slitwidth_um/2*1e-6)**2 / (z_m*wavelength_nm*1e-9)
    return FN;



def doublepinholes_LightPipes(wavelength_nm,
             slitwidth_um,
             separation_um,
             orientation,
             aperturewidth_mm,
             zApertureToPinholes,
             zPinholesToCCD,
             pos,
             overlay,
             zoom,
             savefigure,
             FocusWithKAOS,
             N,
             rescale,
             beammagnification,
             shiftx_um):


    N=rescale*N
    #N=1024
    pixelsize=13*um / rescale
    px=pixelsize
    size=N*pixelsize

    # F0: Gaussian beam hitting an circular aperture at 0m
    # Beam radius w with Intensity(r=w)=1/e
    # --> w = 0.6 FWHM
    wavelength = wavelength_nm*nm
    #beamFWHM = beamFWHM_mm*mm
    beamFWHM = beamFWHM_mm(wavelength_nm)*mm
    #beam2w = beam2w_mm(wavelength_nm)*mm
    beam2w = beamFWHM * 1.699

    if FocusWithKAOS == 'off':
        mag = 1.0
    elif FocusWithKAOS == 'FWHM':
        mag = magnificationFWHM(wavelength_nm)
    elif FocusWithKAOS == '2w':
        mag = magnification2w(wavelength_nm)


    beamFWHM = beamFWHM*mag*beammagnification
    beam2w = beam2w*mag*beammagnification


    slitwidth = slitwidth_um*um
    separation = separation_um*um
    aperturewidth = aperturewidth_mm*mm

    FN=FresnelNumber(slitwidth_um,wavelength_nm,z)

    I={}

    PinholesBtoCCD = 5.781*m    # old


    PinholesAtoCCD =  12.47671*m

    # from source to KBH = ...
    zSourceToKBH = 0
    # from KBH to PMMA = 2000*mm
    zKBHtoAperture7 = 2697.5*mm
    
    focalLength = 2000*mm
    
    # from Aperture7 to PinholesB = 370*mm
    zAperture7toPinholesB = 370*mm
    # from PMMA to Aperture7 = 697.5*mm
    # from KBH to Aperture7 = 2697.5*mm
    # from Aperture7 to PinholesB = 370*mm

    
    
    #z0to1=0.326*m # Pinholes-A
    z0to1=0.371*m # Pinholes-B
    #z3to4=PinholesAtoCCD # pinhole before KAOS
    z3to4=PinholesBtoCCD # pinhole after KAOS
    #z=z3to4

    F0=LP.Begin(size,wavelength,N)
    F0=LP.GaussAperture(0.6*beamFWHM, 0+shiftx_um*um, 0, 1, F0)  # why 0.6?
    I[0]=LP.Intensity(1,F0)

    F1 = LP.LensFresnel(focalLength,zKBHtoAperture7,F0)
    #F2 = LP.Fresnel(zKBHtoAperture7,F1)
    F2 = LP.Convert(F1)
    
    F3=LP.CircAperture(aperturewidth/2, 0+shiftx_um*um, 0, F2)
    I[1]=LP.Intensity(1,F3)
 
    F4=LP.Fresnel(zAperture7toPinholesB,F3)
    I[2]=LP.Intensity(1,F4)

    if orientation=='vertical':
        F5a=LP.CircAperture(slitwidth/2, 0+shiftx_um*um, -separation/2, F4)
        F5b=LP.CircAperture(slitwidth/2, 0+shiftx_um*um, separation/2, F4)
    elif orientation=='horizontal':
        F5a=LP.CircAperture(slitwidth/2, -separation/2+shiftx_um*um, 0, F4)
        F5b=LP.CircAperture(slitwidth/2,  separation/2+shiftx_um*um, 0, F4)

    F5=LP.BeamMix(F5a,F5b)
    I[3]=LP.Intensity(1,F5)

    

    F6=LP.Fresnel(zPinholesToCCD,F5)  # propagate from Pinhole to CCD
    I[4]=LP.Intensity(1,F6)

    Int = I[pos]
    Int_transp=np.transpose(Int)

    Intlinex=Int[int(N/2)]
    Intliney=Int_transp[int(N/2)]

    x_values = list(range(N))

    t={}
    t[0]='Intensity of FEL before Lens/KBH'
    t[1]='Intensity of aperture'
    t[2]='Intensity of aperture on pinhole plate'
    t[3]='Intensity after pinhole plate'
    t[4]='Intensity of double pinholes on CCD z=' + str(z3to4) +'m (KAOS=' + FocusWithKAOS +')'

    # plot it
    fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, ncols=1, sharex=True, sharey=False,
                                   gridspec_kw = {'height_ratios':[1,4,1], 'width_ratios':[1]},
                                  figsize=(8,12))

    fig.suptitle(t[pos],y=1.05, fontsize=24)

    ax0.fill_between(x_values,Intlinex,0)
    ax0.plot(x_values,Intlinex,'.r',MarkerSize=3)
    #ax0.plot(x_values[::2],Intlinex[::2],'.g',MarkerSize=6)
    #ax0.set_title('Intensity along x')
    ax0.set_xlim(N/2+1-(N/2+1)/zoom, N/2+1+(N/2+1)/zoom)
    ax0.set_ylim(0, 1)

    ax1.imshow(Int);
    ax1.axis('off');

    ax2.fill_between(x_values,Intliney,0)
    ax2.plot(x_values,Intliney,'.r',MarkerSize=3)
    #ax2.set_title('Intensity along y')


    if overlay==True:
        if orientation=='vertical':
            pinholecirlce1 = plt.Circle((N/2+1, N/2+1-(separation/2)/px), (slitwidth/2)/px, color='r')
            ax1.add_artist(pinholecirlce1)
            pinholecirlce2 = plt.Circle((N/2+1, N/2+1+(separation/2)/px), (slitwidth/2)/px, color='r')
            ax1.add_artist(pinholecirlce2)
        elif orientation=='horizontal':
            pinholecirlce1 = plt.Circle((N/2+1-(separation/2)/px, N/2+1), (slitwidth/2)/px, color='r')
            ax1.add_artist(pinholecirlce1)
            pinholecirlce2 = plt.Circle((N/2+1+(separation/2)/px, N/2+1), (slitwidth/2)/px, color='r')
            ax1.add_artist(pinholecirlce2)

        aperturecircle = plt.Circle((N/2+1, N/2+1), (aperturewidth/2)/px, color='m', fill=False)
        ax1.add_artist(aperturecircle)

        beamFWHMcircle = plt.Circle((N/2+1, N/2+1), (beamFWHM/2)/px, color='g', fill=False)
        ax1.add_artist(beamFWHMcircle)
        beam2wcircle = plt.Circle((N/2+1, N/2+1), (beam2w/2)/px, color='k', fill=False)
        ax1.add_artist(beam2wcircle)

        separation_5px_circle = plt.Circle((N/2+1, N/2+1), (separation_5px_m(wavelength_nm)/2)/px, color='y', fill=False)
        ax1.add_artist(separation_5px_circle)

    textstr = 'wavelength=' + str(wavelength_nm) + 'nm' + '\n' \
    + 'z=' + str(z3to4) + 'm' + '\n' \
    + 'ApertureWidth=' + str(aperturewidth_mm) + 'mm\n' \
    + 'SlitWidth=' + str(round(slitwidth_um,1)) + 'um\n' \
    + 'Separation=' + str(round(separation_um,1)) + 'um\n' \
    + 'Separation5px=' + str(round(separation_5px_m(wavelength_nm)/um,1)) + 'um\n' \
    + 'FN=' + str(round(FN,5)) + '\n' \
    + 'magnification=' + str(round(mag,2)) + '\n' \
    + 'BeamFWHM=' + str(round(beamFWHM/mm,1)) + 'mm\n' \
    + 'Beam2w=' + str(round(beam2w/mm,1))

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)

    # place a text box in upper left in axes coords
    ax0.text(0.01, 0.95, textstr, transform=ax0.transAxes, fontsize=10,
    verticalalignment='top', bbox=props)

    textstr = 'magenta: aperture\n' \
             + 'yellow: max separation for 5px/fringe\n' \
             + 'green: BeamFWHM ' \
             + 'black: Beam2w'

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)

    # place a text box in upper left in axes coords
    ax1.text(0.01, 0.97, textstr, transform=ax1.transAxes, fontsize=12,
    verticalalignment='top', bbox=props)

    fig.tight_layout(pad=0.0, w_pad=0.0, h_pad=0)

    if savefigure == True:
        plt.savefig('w'+str(wavelength_nm) \
                + 's'+str(slitwidth_um) \
                + 'd'+str(separation_um) \
                + 'ap'+str(aperturewidth_mm) \
                + 'z'+str(z3to4) \
                + 'zoom'+str(zoom) \
                + 'KAOS_'+ FocusWithKAOS\
                + '.png', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)
    plt.show()

    return Int, Intlinex[::rescale], Intliney[::rescale];


def run_doublepinholes_LightPipes(run,
                                  wavelength_nm,
                                 slitwidth_um,
                                 separation_um,
                                 orientation,
                                 aperturewidth_mm,
                                 zApertureToPinholes,
                                 zPinholesToCCD,
                                 pos,
                                 overlay,
                                 zoom,
                                 savefigure,
                                 FocusWithKAOS,
                                 N,
                                 rescale,
                                 beammagnification,
                                 shiftx_um):
    if run == True:
        Int, Intlinex, Intliney = doublepinholes_LightPipes(wavelength_nm,
                                                             slitwidth_um,
                                                             separation_um,
                                                             orientation,
                                                             aperturewidth_mm,
                                                             zApertureToPinholes,
                                                             zPinholesToCCD,
                                                             pos,
                                                             overlay,
                                                             zoom,
                                                             savefigure,
                                                             FocusWithKAOS,
                                                             N,
                                                             rescale,
                                                             beammagnification,
                                                             shiftx_um)
        return Int, Intlinex, Intliney


run_doublepinholes_LightPipes_widget = widgets.Checkbox(value=True,
                                              description='run_doublepinholes_LightPipes (disable when changing multiple parameters!)')

run_doublepinholes_LightPipes_interactive = interactive(run_doublepinholes_LightPipes,
                                    run = run_doublepinholes_LightPipes_widget,
                                    wavelength_nm=widgets.FloatText(min=4.0,max=20,step=0.5,value=13.5),
                                    slitwidth_um=widgets.FloatText(min=1,max=60,step=1,value=10),
                                    #separation_um= list(map(lambda x:1000*x, s)),
                                    separation_um=widgets.FloatText(min=47,max=3800,step=1,value=445),
                                    orientation=['horizontal','vertical'],
                                    aperturewidth_mm=widgets.FloatText(min=1.0,max=15.0,step=0.5,value=3.0),
                                    zApertureToPinholes=widgets.FloatText(min=0.0,max=13.0,value=0.37),
                                    zPinholesToCCD=widgets.FloatText(min=0.0,max=13.0,value=5.781),
                                    pos=widgets.IntSlider(min=0,max=4,value=4),
                                    overlay=True,
                                    savefigure=False,
                                    zoom=widgets.IntSlider(min=1,max=10,value=1),
                                    FocusWithKAOS=['off','FWHM','2w'],
                                    N=widgets.IntText(step=1,value=1047),
                                    rescale=widgets.IntText(min=1,max=10,step=1,value=1),
                                    beammagnification=widgets.FloatText(value=1.0),
                                    shiftx_um = widgets.FloatText(value=0));

#display(run_doublepinholes_LightPipes_interactive)
#Int_LightPipes, Intlinex_LightPipes, Intliney_LightPipes = run_doublepinholes_LightPipes_interactive.result


aperture5_mm_bg_widget = widgets.FloatText(disabled=True, description='aperture5_mm_bg')
aperture7_mm_bg_widget = widgets.FloatText(disabled=True, description='aperture7_mm_bg')
aperture5_mm_image_widget = widgets.FloatText(disabled=True, description='aperture5_mm_image')
aperture7_mm_image_widget = widgets.FloatText(disabled=True, description='aperture7_mm_image')
filter_bg_widget = widgets.Text(disabled=True, description='filter_bg')
filter_image_widget = widgets.Text(disabled=True, description='filter_image')


def select_dataset_bg_args(hdf5_file_name):
    dataset_bg_args_widget.options = dataset_bg_args[hdf5_file_name]

layout = widgets.Layout(width='80%')
hdf5_file_name_bg_widget = widgets.Dropdown(options=dataset_bg_args.keys(),
                                            layout=layout)
dataset_bg_args_widget = widgets.Dropdown(options=dataset_bg_args[hdf5_file_name_bg_widget.value],
                                         layout=layout)



select_dataset_bg_args_interactive = interactive(select_dataset_bg_args,
                                                    hdf5_file_name=hdf5_file_name_bg_widget)

get_bg_interactive = interactive(get_images,
                                    hdf5_file_name=hdf5_file_name_bg_widget,
                                    dataset_args=dataset_bg_args_widget)



#display(select_dataset_bg_args_interactive)
display(get_bg_interactive)

(pixis_bg_dataset, pixis_bg_avg, pixis_bg_std, pinholes_bg_dataset, pinholes_bg_avg, timestamp_bg_dataset, sep_bg, orient_bg, daq_parameter_bg_dataset, aperture5_mm_bg_widget.value, aperture7_mm_bg_widget.value, filter_bg_widget.value) = get_bg_interactive.result

def select_first_bg_args_on_hdf5_change(change):
    dataset_bg_args_widget.value = dataset_bg_args[hdf5_file_name_bg_widget.value][0]

hdf5_file_name_bg_widget.observe(select_first_bg_args_on_hdf5_change, names='value')

def bind_pixis_bg_result_to_global_variables(change):
    global pixis_bg_dataset
    global pixis_bg_avg
    global pixis_bg_std
    global pinholes_bg_dataset
    global pinholes_bg_avg
    global timestamp_bg_dataset
    global sep_bg
    global orient_bg
    global daq_parameter_bg_dataset
    
    #plot_data_and_simulation_interactive_output.clear_output()
    get_bg_interactive.update()

    (pixis_bg_dataset, pixis_bg_avg, pixis_bg_std, pinholes_bg_dataset, pinholes_bg_avg, timestamp_bg_dataset, sep_bg, orient_bg, daq_parameter_bg_dataset, aperture5_mm_bg_widget.value, aperture7_mm_bg_widget.value, filter_bg_widget.value) = get_bg_interactive.result
    #Int_LightPipes, Intlinex_LightPipes, Intliney_LightPipes = run_doublepinholes_LightPipes_interactive.result

    imageid_widget.options = get_imageids_with_bgs(beamposition_horizontal_interval_widget.value)
        
    #imageid_widget.value = get_imageids_with_bgs(energy_interval_widget.value)[0]
    #number_of_images_widget.value = len(df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].index)
    number_of_images_widget.value = len(get_imageids_with_bgs(beamposition_horizontal_interval_widget.value))
#     imageid_widget.options = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].sort_values('energy hall')['imageid'].tolist()
#     imageid_widget.value = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].sort_values('energy hall')['imageid'].tolist()[0]
    #plot_data_and_simulation_interactive.update()
    #plot_data_and_simulation_interactive_output
    #plot_data_and_simulation_interactive_input.update()

dataset_bg_args_widget.observe(bind_pixis_bg_result_to_global_variables, names='value')



# getting the image dataset

def select_dataset_image_args(hdf5_file_name):
    dataset_image_args_widget.options = dataset_image_args[hdf5_file_name]

hdf5_file_name_image_widget = widgets.Dropdown(options=dataset_image_args.keys(),
                                              layout=layout)
dataset_image_args_widget = widgets.Dropdown(options=dataset_image_args[hdf5_file_name_image_widget.value],
                                            layout=layout)


get_images_interactive = interactive(get_images,
                                    hdf5_file_name=hdf5_file_name_image_widget,
                                    dataset_args=dataset_image_args_widget)

select_dataset_image_args_interactive = interactive(select_dataset_image_args,
                                                    hdf5_file_name=hdf5_file_name_image_widget)

#display(select_dataset_image_args_interactive)
display(get_images_interactive)
#get_image_interactive.update()

(pixis_dataset, pixis_avg, pixis_std, pinholes_dataset, pinholes_avg, timestamp_dataset, sep, orient, daq_parameter_image_dataset, aperture5_mm_image_widget.value, aperture7_mm_image_widget.value, filter_image_widget.value) = get_images_interactive.result

def select_first_image_args_on_hdf5_change(change):
    dataset_image_args_widget.value = dataset_image_args[hdf5_file_name_image_widget.value][0]

hdf5_file_name_image_widget.observe(select_first_image_args_on_hdf5_change, names='value')


loadsettings_widget = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Loading settings:',
    bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

# def get_imageids_with_bgs(energy_interval):

#     imageid_sequence = []
#     for imageid in df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].sort_values('energy hall')['imageid']:
#         energy_image = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])]['energy hall'][imageid]
#         matching_bg_indices = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2]) & (df_all['energy hall'] > energy_image - energy_interval/2 ) & (df_all['energy hall'] < energy_image + energy_interval/2 ) ]['energy hall'].index
#         if matching_bg_indices.empty == False:
#             imageid_sequence.append(imageid)
        
#     return imageid_sequence



def bind_pixis_result_to_global_variables(change):
    global pixis_dataset
    global pixis_avg
    global pixis_std
    global pinholes_dataset
    global pinholes_avg
    global timestamp_dataset
    global sep
    global orient
    global daq_parameter_image_dataset
    
    loadsettings_widget.bar_style='info'
    loadsettings_widget.value=5
    
    #plot_data_and_simulation_interactive_output.clear_output()
    get_images_interactive.update()
    
    

    (pixis_dataset, pixis_avg, pixis_std, pinholes_dataset, pinholes_avg, timestamp_dataset, sep, orient, daq_parameter_image_dataset, aperture5_mm_image_widget.value, aperture7_mm_image_widget.value, filter_image_widget.value) = get_images_interactive.result
    #imageid_widget.options=df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].sort_values('energy hall')['imageid'].tolist()
    
    #imageid_widget.value=df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].sort_values('energy hall')['imageid'].tolist()[0]
    #imageid_widget.value = -1
    #imageid_widget.max = len(pixis_dataset)-1
    
    imageid_widget.options = get_imageids_with_bgs(beamposition_horizontal_interval_widget.value)
    
    imageid_widget.value = get_imageids_with_bgs(beamposition_horizontal_interval_widget.value)[0]
    #number_of_images_widget.value = len(df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].index)
    number_of_images_widget.value = len(get_imageids_with_bgs(beamposition_horizontal_interval_widget.value))
    d_um_widget.value = sep
    if daq_parameter_image_dataset[0][imageid_widget.value] > 0:
        _lambda_widget.value=daq_parameter_image_dataset[0][imageid_widget.value]
    else:
        _lambda_widget.value=daq_parameter_image_dataset[14][imageid_widget.value]

    if np.isfinite(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_rotation'].iloc[0]) == True:
        pixis_rotation_widget.value = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_rotation'].iloc[0]
     
    if np.isfinite(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_centerx_px'].iloc[0]) == True:
        pixis_centerx_px_widget.value = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_centerx_px'].iloc[0]
    
    if np.isfinite(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_centery_px'].iloc[0]) == True:
        pixis_centery_px_widget.value = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_centery_px'].iloc[0]
                                                                                                           
    if np.isfinite(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pinholes_centerx_px'].iloc[0]) == True:
        pinholes_centerx_px_widget.value = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pinholes_centerx_px'].iloc[0]
    
    if np.isfinite(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pinholes_centery_px'].iloc[0]) == True:
        pinholes_centery_px_widget.value = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pinholes_centery_px'].iloc[0]
    
        loadsettings_widget.value=10
        #run_plot_data_and_simulation_widget.value = True
    else:
        loadsettings_widget.bar_style='warning'


dataset_image_args_widget.observe(bind_pixis_result_to_global_variables, names='value')


# imageid_widget = widgets.BoundedIntText(
#     value=0,
#     step=1,
#     min=-1,
#     max=len(pixis_dataset)-1,
#     description='Image ID',
#     disabled=False,
#     #continuous_update=True
# )

energy_interval_widget = widgets.FloatText(
    value=0.5,
    step=0.1,
    description='energy_interval',
    disabled=False
)

beamposition_horizontal_interval_widget = widgets.FloatText(
    value=1000,
    step=1,
    description='beamposition_horizontal_interval',
    disabled=False
)

def update_imageid_widget(change):
    imageid_widget.options = get_imageids_with_bgs(beamposition_horizontal_interval_widget.value)
    imageid_widget.value = get_imageids_with_bgs(beamposition_horizontal_interval_widget.value)[0]
    number_of_images_widget.value = len(get_imageids_with_bgs(beamposition_horizontal_interval_widget.value))

beamposition_horizontal_interval_widget.observe(update_imageid_widget, names='value')

# imageid_widget = widgets.Dropdown(
#     options=df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].sort_values('energy hall')['imageid'].tolist(),
#     value=df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])].sort_values('energy hall')['imageid'].tolist()[0],
#     description='imaged (sorted by energy):',
#     disabled=False,
# )

imageid_widget = widgets.Dropdown(
    options=get_imageids_with_bgs(beamposition_horizontal_interval_widget.value),
    value=get_imageids_with_bgs(beamposition_horizontal_interval_widget.value)[0],
    description='imaged (sorted by beamposition_horizontal):',
    disabled=False,
)


use_pixis_avg_widget = widgets.Checkbox(
    value=False,
    description='use_pixis_avg_widget',
    disabled=False
)

pixis_rotation_widget = widgets.FloatText(
    value=-1.3,
    step=0.1,
    description='pixis rotation',
    disabled=False
)




pinholes_rotation_widget = widgets.FloatText(
    value=0,
    step=0.1,
    description='pinholes rotation',
    disabled=False
)

number_of_images_widget = widgets.IntText(
    value=len(pixis_dataset)-1,
    placeholder='Type something',
    description='max. Image ID:',
    disabled=True
)

pixis_bgfactor_widget = widgets.FloatText(
    value=1,
    step=0.01,
    description='pixis bgfactor',
    disabled=False
)

pinholes_bgfactor_widget = widgets.FloatText(
    value=0,
    step=0.01,
    description='pinholes bgfactor',
    disabled=False
)

pixis_avg_width_widget = widgets.FloatText(
    value=10,
    step=1,
    description='pixis avg_width',
    disabled=False
)

pinholes_avg_width_widget = widgets.FloatText(
    value=200,
    step=1,
    description='pinholes avg_width',
    disabled=False
)


#get_pixis_profiles_interactive = interactive(get_pixis_profiles,
                                      # imageid=imageid_widget,
                                      # bgfactor=widget_bgfactor,
                                      # avg_width=widget_avg_width)
#display(get_pixis_profiles_interactive)
#get_pixis_profiles_interactive.update()
#(pixis_image_norm, pixis_profile, pixis_profile_avg) = get_pixis_profiles_interactive.result

#centerx = np.where(pixis_profile_avg==1)[0][0]
#shiftx_px = centerx - pixis_profile_avg.size/2
#shiftx_um = shiftx_px * 13


_lambda_widget = widgets.FloatText(
    value=daq_parameter_image_dataset[0][imageid_widget.value],
    step=0.1,
    description='lambda/nm:',
    disabled=False
)


def update__lambda_widget(change):
    if daq_parameter_image_dataset[0][imageid_widget.value] > 0:  # or daq_parameter_image_dataset[0][imageid_widget.value] == NaN):
        _lambda_widget.value=daq_parameter_image_dataset[0][imageid_widget.value]
    else:
        _lambda_widget.value=daq_parameter_image_dataset[14][imageid_widget.value]

imageid_widget.observe(update__lambda_widget, names='value')

w1_widget = widgets.FloatText(
    value=10,
    step=0.1,
    description='ph1 width/um:',
    disabled=False
)

w2_widget = widgets.FloatText(
    value=10,
    step=0.1,
    description='ph2 width/um:',
    disabled=False
)

I_w1_widget = widgets.FloatText(
    value=1,
    step=0.1,
    description='I_w1:',
    disabled=False
)

I_w2_widget = widgets.FloatText(
    value=1,
    step=0.1,
    description='I_w2:',
    disabled=False
)

z_widget = widgets.FloatText(
    value=5781,
    step=1,
    description='Distance z/mm:',
    disabled=False
)

I_Airy1_widget = widgets.FloatText(
    value=0,
    step=0.1,
    description='I_Airy1:',
    disabled=False
)

I_Airy2_widget = widgets.FloatText(
    value=0,
    step=0.1,
    description='I_Airy2:',
    disabled=False
)

sep_factor_widget = widgets.FloatText(
    value=1,
    step=0.1,
    description='sep_factor:',
    disabled=False
)

gamma_widget = widgets.FloatText(
    value=1,
    step=0.1,
    description='Coherence Degree \gamma:',
    disabled=False
)

d_um_widget = widgets.FloatText(
    value=sep,
    step=0.1,
    description='Slit Separation/um',
    disabled=False
)

shiftx_um_adjust_widget = widgets.FloatText(
    value=0,
    step=1,
    description='shift x adjust / um',
    disabled=False
)

widget_shiftx_um_auto = widgets.FloatText(
    value=0,
    step=1,
    description='shift x auto / um',
    disabled=False
)

pinholes_centerx_px_widget = widgets.IntText(value=500,description='pinholes_centerx_px',disabled=False)
pinholes_centery_px_widget = widgets.IntText(value=500,description='pinholes_centery_px',disabled=False)
pinholes_avg_width_alongy_widget = widgets.IntText(value=100,description='pinholes_avg_width_alongy',disabled=False)
pinholes_avg_width_alongx_widget = widgets.IntText(value=100,description='pinholes_avg_width_alongx',disabled=False)

xlim_lower_widget = widgets.BoundedIntText(
    value=0,
    step=100,
    min=0,
    max=1047,
    description='xlim lower',
    disabled=False
)

xlim_upper_widget = widgets.BoundedIntText(
    value=1047,
    step=100,
    min=0,
    max=1047,
    description='xlim lower',
    disabled=False
)


pixis_centerx_px_widget = widgets.FloatText(
    value=523,
    step=1,
    description='centerx_px',
    disabled=False
)

pixis_centery_px_widget = widgets.IntText(
    value=523,
    step=1,
    description='centery_px',
    disabled=False
)

zoomwidth_widget = widgets.IntText(
    value=100,
    step=2,
    description='zoomwidth/px',
    disabled=False
)

zoomborderleft_widget = widgets.IntText(
    value=0,
    step=1,
    description='zoomborderleft/px',
    disabled=False
)

zoomborderright_widget = widgets.IntText(
    value=0,
    step=1,
    description='zoomborderright/px',
    disabled=False
)

#get_simulation_interactive = interactive(get_simulation,
                                    #     _lambda_nm=_lambda_widget,
                                    #     w_um=widget_w,
                                    #     z_mm=z_widget,
                                    #     gamma=gamma_widget,
                                    #     d_um=d_widget,
                                    #     shiftx_um=widget_shiftx_um)
#display(get_simulation_interactive)
#get_simulation_interactive.update()
#(theta, I_D_0, I_D_normalized) = get_simulation_interactive.result

display(number_of_images_widget)

pixis_centerx_px_auto_widget = widgets.FloatText(
    value=0,
    placeholder='Type something',
    description='pixis_centerx_px_auto',
    disabled=True
)
display(pixis_centerx_px_auto_widget)


pixis_profile_avg_centerx_px_auto_widget = widgets.FloatText(
    value=0,
    placeholder='Type something',
    description='pixis_profile_avg_centerx_px_auto',
    disabled=True
)
display(pixis_profile_avg_centerx_px_auto_widget)

use_pixis_profile_avg_centerx_widget = widgets.Checkbox(value=False, description = 'use_pixis_profile_avg_centerx')
pixis_center_autofind_widget = widgets.Checkbox(value=True, description = 'pixis_center_autofind')

plot_lineout_widget = widgets.Checkbox(value=False, description = 'plot_lineout')
plot_simulation_widget = widgets.Checkbox(value=False, description = 'plot_simulation')
plot_simulation_fit_widget = widgets.Checkbox(value=True, description = 'plot_simulation_fit')
plot_LightPipes_simulation_widget = widgets.Checkbox(value=False, description = 'plot_LightPipes_simulation')
plot_pixis_std_contour_widget = widgets.Checkbox(value=True, description = 'plot_pixis_std_contour')
levels_auto_widget = widgets.Checkbox(value=True, description = 'levels_auto')
levels_min_widget = widgets.FloatText(value=0.0, description = 'levels_min')
levels_max_widget = widgets.FloatText(value=0.1, description = 'levels_max')
savefigure_widget = widgets.Checkbox(value=False, description = 'savefigure')
savefigure_all_images_widget = widgets.Checkbox(value=False, description = 'savefigure_all_images')
PlotTextbox_widget = widgets.Checkbox(value=False, description = 'PlotTextbox')
initialize_fits_list_widget = widgets.Checkbox(value=True, description = 'initialize_fits_list')


df_fits_save_widget = widgets.ToggleButton(
    value=False,
    description='save to df_fits',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='save to df_fits',
    icon='check'
)

df_fits_load_widget = widgets.ToggleButton(
    value=False,
    description='load from df_fits',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='load from df_fits',
    icon='check'
)

#['bgfactor', 'pixis_rotation', 'pixis_centerx_px', 'pixis_centery_px', 'pixis_profile_centerx_px_fit', 'pixis_profile_centery_px_fit', 'pinholes_cm_x_px', 'pinholes_cm_y_px', '_lambda_nm_fit', 'gamma_fit']

def update_df_fits_save_widget(change):
    if df_fits_save_widget.value == True:
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])), 'pixis_rotation'] = pixis_rotation_widget.value
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])), 'pixis_centerx_px'] = pixis_centerx_px_widget.value
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])), 'pixis_centery_px'] = pixis_centery_px_widget.value
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])), 'pinholes_centerx_px'] = pinholes_centerx_px_widget.value
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])), 'pinholes_centery_px'] = pinholes_centery_px_widget.value
        df_fits_save_widget.value = False

df_fits_save_widget.observe(update_df_fits_save_widget, names='value')


def update_df_fits_load_widget(change):
    if df_fits_load_widget.value == True:
        pixis_rotation_widget.value = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_rotation'].iloc[0]
        pixis_centerx_px_widget.value = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_centerx_px'].iloc[0]
        pixis_centery_px_widget.value = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_centery_px'].iloc[0]
        df_fits_load_widget.value = False

df_fits_load_widget.observe(update_df_fits_load_widget, names='value')

df_fits_csv_save_widget = widgets.ToggleButton(
    value=False,
    description='save df_fits to csv',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='save df_fits to csv',
    icon='check'
)

def update_df_fits_csv_save_widget(change):
    if df_fits_csv_save_widget.value == True:
        # save fits to csv
        df_fits = df0[['timestamp_pulse_id'] + fits_header_list]
        save_df_fits = True
        if save_df_fits == True:
            df_fits.to_csv('/home/wodzinsk/PycharmProjects/coherence/'+'df_fits_v2.csv')
        df_fits_csv_save_widget.value = False

df_fits_csv_save_widget.observe(update_df_fits_csv_save_widget, names='value')






run_plot_data_and_simulation_widget = widgets.Checkbox(value=False,
                                              description='run_plot_data_and_simulation (Disable when changing multiple parameters!)')

   
ap5_x_px_widget = widgets.FloatText(value=523, step=1, description='ap5_x_px', disabled=False)
ap5_y_px_widget = widgets.FloatText(value=523, step=1, description='ap5_y_px', disabled=False)
ap5_width_px_widget = widgets.FloatText(value=200, step=1, description='ap5_width_px', disabled=False)
ap5_height_px_widget = widgets.FloatText(value=200, step=1, description='ap5_height_px', disabled=False)
screenhole_x_px_widget = widgets.FloatText(value=523, step=1, description='screenhole_x_px', disabled=False)
screenhole_y_px_widget = widgets.FloatText(value=523, step=1, description='screenhole_y_px', disabled=False)
screenhole_width_px_widget = widgets.FloatText(value=50, step=1, description='screenhole_width_px', disabled=False)
screenhole_height_px_widget = widgets.FloatText(value=50, step=1, description='screenhole_height_px', disabled=False)


cropforbp_widget = widgets.IntText(
    value=20,
    description='cropforbp:',
    disabled=False
)


# plot_data_and_simulation_interactive_input = widgets.VBox([
#                                                    run_plot_data_and_simulation_widget,
#                                                    imageid_widget,
#                                                    number_of_images_widget,
#                                                    pixis_rotation_widget,
#                                                    pixis_bgfactor_widget,
#                                                    pixis_avg_width_widget,
#                                                    pinholes_rotation_widget,
#                                                    pinholes_bgfactor_widget,
#                                                    pinholes_avg_width_widget,
#                                                    _lambda_widget,
#                                                    w1_widget,
#                                                    w2_widget,
#                                                    I_w1_widget,
#                                                    I_w2_widget,
#                                                    z_widget,
#                                                    I_Airy1_widget,
#                                                    I_Airy2_widget,
#                                                    sep_factor_widget,
#                                                    gamma_widget,
#                                                    d_um_widget,
#                                                    aperture5_mm_bg_widget,
#                                                    aperture7_mm_bg_widget,
#                                                    aperture5_mm_image_widget,
#                                                    aperture7_mm_image_widget,
#                                                    filter_bg_widget,
#                                                    filter_image_widget,
#                                                    use_pixis_profile_avg_centerx_widget,
#                                                    pixis_center_autofind_widget,
#                                                    pixis_centerx_px_widget,
#                                                    pixis_centery_px_widget,
#                                                    shiftx_um_adjust_widget,
#                                                    pinholes_centerx_px_widget,
#                                                    pinholes_centery_px_widget,
#                                                    pinholes_avg_width_alongy_widget,
#                                                    pinholes_avg_width_alongx_widget,
#                                                    xlim_lower_widget,
#                                                    xlim_upper_widget,
#                                                    plot_lineout_widget,
#                                                    plot_simulation_widget,
#                                                    plot_simulation_fit_widget,
#                                                    plot_LightPipes_simulation_widget,
#                                                    plot_pixis_std_contour_widget,
#                                                    levels_auto_widget,
#                                                    levels_min_widget,
#                                                    levels_max_widget,
#                                                    zoomwidth_widget,
#                                                    zoomborderleft_widget,
#                                                    zoomborderright_widget,
#                                                    savefigure_widget,
#                                                    savefigure_all_images_widget,
#                                                    PlotTextbox_widget,
#                                                    initialize_fits_list_widget])

plot_data_and_simulation_interactive_input_column1 = VBox([
                run_plot_data_and_simulation_widget,
                #imageid_widget,
                number_of_images_widget,
                #pixis_rotation_widget,
                #pixis_bgfactor_widget,
                pixis_avg_width_widget,
                pinholes_rotation_widget,
                pinholes_bgfactor_widget,
                pinholes_avg_width_widget,
                beamposition_horizontal_interval_widget,
                use_pixis_avg_widget
])

plot_data_and_simulation_interactive_input_column2 = VBox([
                _lambda_widget,
                w1_widget,
                w2_widget,
                I_w1_widget,
                I_w2_widget,
                z_widget,
                I_Airy1_widget,
                I_Airy2_widget,
                sep_factor_widget,
                gamma_widget,
                d_um_widget,
])


plot_data_and_simulation_interactive_input_column3 = VBox([
                aperture5_mm_bg_widget,
                aperture7_mm_bg_widget,
                aperture5_mm_image_widget,
                aperture7_mm_image_widget,
                filter_bg_widget,
                filter_image_widget,
])

plot_data_and_simulation_interactive_input_column4 = VBox([
                use_pixis_profile_avg_centerx_widget,
                pixis_center_autofind_widget,
                pixis_centerx_px_widget,
                pixis_centery_px_widget,
                shiftx_um_adjust_widget,
                pinholes_centerx_px_widget,
                pinholes_centery_px_widget,
                pinholes_avg_width_alongy_widget,
                pinholes_avg_width_alongx_widget,
                xlim_lower_widget,
                xlim_upper_widget
])

plot_data_and_simulation_interactive_input_column5 = VBox([
                plot_lineout_widget,
                plot_simulation_widget,
                plot_simulation_fit_widget,
                plot_LightPipes_simulation_widget,
                plot_pixis_std_contour_widget,
                levels_auto_widget,
                levels_min_widget,
                levels_max_widget,
                zoomwidth_widget,
                zoomborderleft_widget,
                zoomborderright_widget,
])

plot_data_and_simulation_interactive_input_column6 = VBox([
                savefigure_widget,
                savefigure_all_images_widget,
                PlotTextbox_widget,
                initialize_fits_list_widget,
    cropforbp_widget
])

plot_data_and_simulation_interactive_input_column7 = VBox([
    ap5_x_px_widget,
    ap5_y_px_widget,
    ap5_width_px_widget,
    ap5_height_px_widget,
    screenhole_x_px_widget,
    screenhole_y_px_widget,
    screenhole_width_px_widget,
    screenhole_height_px_widget
])

plot_data_and_simulation_interactive_input = VBox([
    HBox([
    plot_data_and_simulation_interactive_input_column1,
    plot_data_and_simulation_interactive_input_column2,
    plot_data_and_simulation_interactive_input_column3,
    plot_data_and_simulation_interactive_input_column4,
    plot_data_and_simulation_interactive_input_column5,
    plot_data_and_simulation_interactive_input_column6,
    plot_data_and_simulation_interactive_input_column7
    ]),
    HBox([imageid_widget,pixis_bgfactor_widget, pixis_rotation_widget, df_fits_save_widget, df_fits_load_widget, loadsettings_widget, df_fits_csv_save_widget])
])


def update_settings(change):
    loadsettings_widget.value = 0
    run_plot_data_and_simulation_widget.value = False
    hdf5_file_name_bg_widget.value = settings_widget.value[0]
    dataset_bg_args_widget.value = settings_widget.value[1]
    hdf5_file_name_image_widget.value = settings_widget.value[2]
    dataset_image_args_widget.value = settings_widget.value[3]

def update_LightPipes(change):
    global Int_LightPipes, Intlinex_LightPipes, Intliney_LightPipes
    Int_LightPipes, Intlinex_LightPipes, Intliney_LightPipes = run_doublepinholes_LightPipes_interactive.result
    #plot_data_and_simulation_interactive.update()

widget_LightPipesUpdate = widgets.Button(value=False, description='Update LightPipes Lineout in plot below')
widget_LightPipesUpdate.on_click(update_LightPipes)

settings_widget_layout = widgets.Layout(width='100%')
settings_widget = widgets.Dropdown(options=settings, layout = settings_widget_layout)
settings_widget.observe(update_settings, names='value')
display(settings_widget)

display(widget_LightPipesUpdate)


plot_data_and_simulation_interactive = interactive(plot_data_and_simulation,
                                                   run_plot_data_and_simulation = run_plot_data_and_simulation_widget,
                                                   imageid=imageid_widget,
                                                   use_pixis_avg = use_pixis_avg_widget,
                                                   imageid_max = number_of_images_widget,
                                                   pixis_rotation=pixis_rotation_widget,
                                                   pixis_bgfactor=pixis_bgfactor_widget,
                                                   pixis_avg_width=pixis_avg_width_widget,
                                                   pinholes_rotation=pinholes_rotation_widget,
                                                   pinholes_bgfactor=pinholes_bgfactor_widget,
                                                   pinholes_avg_width=pinholes_avg_width_widget,
                                                   _lambda_nm=_lambda_widget,
                                                   w1_um=w1_widget,
                                                   w2_um=w2_widget,
                                                   I_w1 = I_w1_widget,
                                                   I_w2 = I_w2_widget,
                                                   z_mm=z_widget,
                                                   I_Airy1 = I_Airy1_widget,
                                                   I_Airy2 = I_Airy2_widget,
                                                   sep_factor = sep_factor_widget,
                                                   gamma=gamma_widget,
                                                   d_um=d_um_widget,
                                                   aperture5_mm_bg=aperture5_mm_bg_widget,
                                                   aperture7_mm_bg=aperture7_mm_bg_widget,
                                                   aperture5_mm_image=aperture5_mm_image_widget,
                                                   aperture7_mm_image=aperture7_mm_image_widget,
                                                   filter_bg=filter_bg_widget,
                                                   filter_image = filter_image_widget,
                                                   use_pixis_profile_avg_centerx = use_pixis_profile_avg_centerx_widget,
                                                   pixis_center_autofind = pixis_center_autofind_widget,
                                                   pixis_centerx_px = pixis_centerx_px_widget,
                                                   pixis_centery_px = pixis_centery_px_widget,
                                                   shiftx_um_adjust=shiftx_um_adjust_widget,
                                                   pinholes_centerx_px = pinholes_centerx_px_widget,
                                                   pinholes_centery_px = pinholes_centery_px_widget,
                                                   pinholes_avg_width_alongy = pinholes_avg_width_alongy_widget,
                                                   pinholes_avg_width_alongx = pinholes_avg_width_alongx_widget,
                                                      ap5_x_px = ap5_x_px_widget,
                                                     ap5_y_px = ap5_y_px_widget,
                                                     ap5_width_px = ap5_width_px_widget,
                                                     ap5_height_px = ap5_height_px_widget,
                                                     screenhole_x_px = screenhole_x_px_widget,
                                                     screenhole_y_px = screenhole_y_px_widget,
                                                     screenhole_width_px = screenhole_width_px_widget,
                                                     screenhole_height_px = screenhole_height_px_widget,
                                                   xlim_lower=xlim_lower_widget,
                                                   xlim_upper=xlim_upper_widget,
                                                   plot_lineout = plot_lineout_widget,
                                                   plot_simulation = plot_simulation_widget,
                                                   plot_simulation_fit = plot_simulation_fit_widget,
                                                   plot_LightPipes_simulation = plot_LightPipes_simulation_widget,
                                                   plot_pixis_std_contour = plot_pixis_std_contour_widget,
                                                   levels_auto = levels_auto_widget,
                                                   levels_min = levels_min_widget,
                                                   levels_max = levels_max_widget,
                                                   zoomwidth = zoomwidth_widget,
                                                   zoomborderleft = zoomborderleft_widget,
                                                   zoomborderright = zoomborderright_widget,
                                                   savefigure= savefigure_widget,
                                                   savefigure_all_images = savefigure_all_images_widget,
                                                   PlotTextbox = PlotTextbox_widget,
                                                   initialize_fits_list = initialize_fits_list_widget,
                                                   cropforbp = cropforbp_widget
                                                  )

plot_data_and_simulation_interactive_output = interactive_output(plot_data_and_simulation,
                                                   {'run_plot_data_and_simulation' : run_plot_data_and_simulation_widget,
                                                   'imageid' : imageid_widget,
                                                    'use_pixis_avg': use_pixis_avg_widget,
                                                   'imageid_max' : number_of_images_widget,
                                                   'pixis_rotation' : pixis_rotation_widget,
#                                                    'pixis_rotation_save' : pixis_rotation_save_widget,
#                                                    'pixis_rotation_load' : pixis_rotation_load_widget,
                                                   'pixis_bgfactor' : pixis_bgfactor_widget,
                                                   'pixis_avg_width' : pixis_avg_width_widget,
                                                   'pinholes_rotation' : pinholes_rotation_widget,
                                                   'pinholes_bgfactor' : pinholes_bgfactor_widget,
                                                   'pinholes_avg_width' : pinholes_avg_width_widget,
                                                   '_lambda_nm' : _lambda_widget,
                                                   'w1_um' : w1_widget,
                                                   'w2_um' : w2_widget,
                                                   'I_w1' : I_w1_widget,
                                                   'I_w2' : I_w2_widget,
                                                   'z_mm' : z_widget,
                                                   'I_Airy1' : I_Airy1_widget,
                                                   'I_Airy2' : I_Airy2_widget,
                                                   'sep_factor' : sep_factor_widget,
                                                   'gamma' : gamma_widget,
                                                   'd_um' : d_um_widget,
                                                   'aperture5_mm_bg' : aperture5_mm_bg_widget,
                                                   'aperture7_mm_bg' : aperture7_mm_bg_widget,
                                                   'aperture5_mm_image' : aperture5_mm_image_widget,
                                                   'aperture7_mm_image' :aperture7_mm_image_widget,
                                                   'filter_bg' : filter_bg_widget,
                                                   'filter_image' : filter_image_widget,
                                                   'use_pixis_profile_avg_centerx' : use_pixis_profile_avg_centerx_widget,
                                                   'pixis_center_autofind' : pixis_center_autofind_widget,
                                                   'pixis_centerx_px' : pixis_centerx_px_widget,
                                                   'pixis_centery_px' : pixis_centery_px_widget,
                                                   'shiftx_um_adjust' : shiftx_um_adjust_widget,
                                                   'pinholes_centerx_px' : pinholes_centerx_px_widget,
                                                   'pinholes_centery_px' : pinholes_centery_px_widget,
                                                   'pinholes_avg_width_alongy' : pinholes_avg_width_alongy_widget,
                                                   'pinholes_avg_width_alongx' : pinholes_avg_width_alongx_widget,
                                                     'ap5_x_px' : ap5_x_px_widget,
                                                     'ap5_y_px' : ap5_y_px_widget,
                                                     'ap5_width_px' : ap5_width_px_widget,
                                                     'ap5_height_px' : ap5_height_px_widget,
                                                     'screenhole_x_px' : screenhole_x_px_widget,
                                                     'screenhole_y_px' : screenhole_y_px_widget,
                                                     'screenhole_width_px' : screenhole_width_px_widget,
                                                     'screenhole_height_px' : screenhole_height_px_widget,
                                                   'xlim_lower' : xlim_lower_widget,
                                                   'xlim_upper' : xlim_upper_widget,
                                                   'plot_lineout' : plot_lineout_widget,
                                                   'plot_simulation' : plot_simulation_widget,
                                                   'plot_simulation_fit' : plot_simulation_fit_widget,
                                                   'plot_LightPipes_simulation' : plot_LightPipes_simulation_widget,
                                                   'plot_pixis_std_contour' : plot_pixis_std_contour_widget,
                                                   'levels_auto' : levels_auto_widget,
                                                   'levels_min' : levels_min_widget,
                                                   'levels_max' : levels_max_widget,
                                                   'zoomwidth' : zoomwidth_widget,
                                                   'zoomborderleft' : zoomborderleft_widget,
                                                   'zoomborderright' : zoomborderright_widget,
                                                   'savefigure' :savefigure_widget,
                                                   'savefigure_all_images' : savefigure_all_images_widget,
                                                   'PlotTextbox' : PlotTextbox_widget,
                                                   'initialize_fits_list' : initialize_fits_list_widget,
                                                     'cropforbp': cropforbp_widget}
                                                  )


#plot_data_and_simulation_interactive_output.layout.width = '100%'
plot_data_and_simulation_interactive_output.layout.height = '2000px'
plot_data_and_simulation_interactive_output.layout.border = 'solid'
display(VBox([plot_data_and_simulation_interactive_input,
              plot_data_and_simulation_interactive_output]))


interactive(children=(Dropdown(description='hdf5_file_name', layout=Layout(width='80%'), options=('FLASH2_USER…

interactive(children=(Dropdown(description='hdf5_file_name', layout=Layout(width='80%'), options=('FLASH2_USER…

IntText(value=14, description='max. Image ID:', disabled=True)

FloatText(value=0.0, description='pixis_centerx_px_auto', disabled=True)

FloatText(value=0.0, description='pixis_profile_avg_centerx_px_auto', disabled=True)

Dropdown(layout=Layout(width='100%'), options=OrderedDict([('2017-11-24T1309 13.5nm 68uJ 7Und. KOAS=PMMA 0000u…

Button(description='Update LightPipes Lineout in plot below', style=ButtonStyle())

IndexError: single positional indexer is out-of-bounds

# Testing profile simulation

In [120]:
def Airy_v1(x,D,_lambda,z,d):
    Z = (1.22 * math.pi/_lambda) * D * (x-d)/z

    f = (math.pi/_lambda)*(D**2/z)* (scipy.special.jv(1,Z)/Z)
    return f

def Airy_v2(x,D,_lambda,z,d):
    Z = (2* math.pi/_lambda) * D * (x-d)/z
    f = 2 * (scipy.special.jv(1,Z)/Z)       
    return f

def Airy_v3(x,D,_lambda,z,d, a, b, c, e):
    Z = (math.pi/_lambda) * D * (x-d)/z
    f = (math.pi/_lambda) * D**2 * 1/z * (scipy.special.jv(1,b*Z)/(c*Z)) + e     
    return f

def Airy(x,D,_lambda,z,d, a, b, c, e):
    Z = (math.pi/_lambda) * c*D * (x-d)/(a*z)
    f = (math.pi/_lambda) * (c*D)**2 * 1/(a*z) * scipy.special.jv(1,b*Z)/(Z) + e  
    return f

def Airy_v5(x,D,_lambda,z,d, a, b, c, e):
    Z = (math.pi/_lambda) * D * (x-d)/(a*np.sqrt(z**2+(x-d)**2))
    f = (math.pi/_lambda) * D**2 * 1/(a*z) * scipy.special.jv(1,b*Z)/Z + e  
    return f

def simulation_test(x, shiftx_um, _lambda_nm, z_mm, d_um, w1_um,w2_um,I_w1,I_w2, I_Airy1, I_Airy2, x1_um, x2_um, gamma, a1, b1, c1, e1, a2, b2, c2, e2, normfactor):

    # Young's double pinholes experiment simulation

    #% Parameters are from Andrej's thesis

    # PIXIS 1024B: 13um pixelsize, 1024px, 13.3mmx13.3mm chip size

    _lambda = _lambda_nm*1e-9# wavelength
    w1 = w1_um*1e-6  # first slit width
    w2 = w2_um*1e-6  # second slit width
    z = z_mm*1e-3  # pinholess to detector #5896
    #gamma# degree of spatial coherence
    d = d_um*1e-6 # slit separation
    
    #R = 1 / dx# rescale factor

    #% Intensity simulation

    shiftx = shiftx_um*1e-6
    x1 = x1_um *1e-6
    x2 = x2_um *1e-6

    k = 2 * math.pi / _lambda
    theta = - (k * (d * (x-shiftx) / z))
    
    #Partial_Coherence_Factor = gamma * np.cos(theta/2)**2
    Partial_Coherence_Factor = gamma * np.cos(theta)

    # The amplitudes of the wave field in the far field:

    #Amp_1 = np.sqrt(I_w1) * Airy((x-shiftx), w1, _lambda, z, -d/2)
    #Amp_2 = np.sqrt(I_w2) * Airy((x-shiftx), w2, _lambda, z, +d/2)
    
    Amp_1 = Airy((x-shiftx), w1, _lambda, z, -d/2, a1, b1, c1, e1)
    Amp_2 = Airy((x-shiftx), w2, _lambda, z, +d/2, a2, b2, c2, e2)

    if I_Airy1 < 0:
        print('I_Airy1 negative')
        
    if I_Airy2 < 0:
        print('I_Airy2 negative')
    
    #I_D = (Amp_1**2 + Amp_2**2) + 2 * abs(Amp_1) * abs(Amp_2) * Partial_Coherence_Factor
    I_D = 0 \
    + 1 * (I_w1 * Amp_1**2 + I_w2 * Amp_2**2) \
    + 2 * np.sqrt(I_w1) * np.sqrt(I_w2) * abs(Amp_1) * abs(Amp_2) * gamma * np.cos(theta) \
    + 2 * np.sqrt(I_Airy1) * Airy((x-shiftx), w1, _lambda, z, x1, a1, b1, c1, e1)  * np.sqrt(I_Airy2) * Airy((x-shiftx), w2, _lambda, z, x2, a2, b2, c2, e2) * gamma * np.cos(theta) \
    + I_Airy1 * Airy((x-shiftx), w1, _lambda, z, x1, a1, b1, c1, e1)**2 \
    + I_Airy2 * Airy((x-shiftx), w2, _lambda, z, x2, a2, b2, c2, e2)**2
    
#      
    #+ np.sqrt(I_Airy1) * Airy((x-shiftx), w1, _lambda, z, -x1)**2 \
    #+ np.sqrt(I_Airy2) * Airy((x-shiftx), w2, _lambda, z, +x2)**2
    
    #I_D_normalized = normfactor * I_D / np.max(+ I_Airy1 * Airy((x-shiftx), w1, _lambda, z, x1, a1, b1, c1, e1)**2 + I_Airy2 * Airy((x-shiftx), w2, _lambda, z, x2, a2, b2, c2, e2)**2)
    I_D_normalized = normfactor * I_D / np.max(I_D)
    

    return I_D_normalized

# pixis_centerx_px = 508.1 + 1* 13.52
# _lambda_nm = 13.53
# gamma = 1

# w1_um = 7.2
# w2_um = 7.2

# I_w1 = 1
# I_w2 = 0.8
# I_Airy1 = 1
# I_Airy2 = 0.8

# x1_loc_px = 150.0
# x2_loc_px = 800.0

from lmfit import Model



def plotprofile(imageid, savefigure, save_to_df, do_textbox,
                shiftx_um, _lambda_nm, z_mm, d_um, gamma, w1_um, w2_um, I_w1, I_w2, I_Airy1, I_Airy2, x1_um, x2_um, a1, b1, c1, e1, a2, b2, c2, e2, normfactor, datafactor,
                shiftx_um_range, _lambda_nm_range, z_mm_range, d_um_range, gamma_range, w1_um_range, w2_um_range, I_w1_range, I_w2_range, I_Airy1_range, I_Airy2_range, x1_um_range, x2_um_range, a1_range, b1_range, c1_range, e1_range, a2_range, b2_range, c2_range, e2_range, normfactor_range,
                shiftx_um_do_fit, _lambda_nm_do_fit, z_mm_do_fit, d_um_do_fit, gamma_do_fit, w1_um_do_fit, w2_um_do_fit, I_w1_do_fit, I_w2_do_fit, I_Airy1_do_fit, I_Airy2_do_fit, x1_um_do_fit, x2_um_do_fit, a1_do_fit, b1_do_fit, c1_do_fit, e1_do_fit, a2_do_fit, b2_do_fit, c2_do_fit, e2_do_fit, normfactor_do_fit):

    imageid_loop = imageid
    
#     hdf5_file_name_image = hdf5_file_name_image_widget.value
#     dataset_image_args = dataset_image_args_widget.value
    imageid_sequence_by_energy_hall = get_imageids_with_bgs(beamposition_horizontal_interval_widget.value)
    df = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & df0['imageid'].isin(imageid_sequence_by_energy_hall)]
    ph = dataset_image_args_widget.value[2]
    separation_um = d_um
    orientation = df[df['imageid']==imageid_widget.value]['orientation'].iloc[0]
    
    
    if imageid == -1:
        beamposx = df['beam position hall horizontal pulse resolved'].mean(axis=0)
        beamposy = df['beam position hall vertical pulse resolved'].mean(axis=0)
        energy_hall_uJ = df['energy hall'].mean(axis=0)
    else:
        beamposx = df[df['imageid']==imageid]['beam position hall horizontal pulse resolved']
        beamposy = df[df['imageid']==imageid]['beam position hall vertical pulse resolved']
        energy_hall_uJ = df[df['imageid']==imageid]['energy hall'].iloc[0]
    
    n = pixis_profile_avg.size # number of sampling point  # number of pixels
    dX_1 = 13e-6
    xdata = np.linspace((-n/2)*dX_1, (+n/2-1)*dX_1, n)
    ydata = pixis_profile_avg_dataset[imageid]*datafactor

    fringeseparation_um = z_mm * 1e-3 * _lambda_nm * 1e-9 / (d_um * 1e-6) * 1e6
    fringeseparation_px = fringeseparation_um / 13
        
    func = simulation_test
    mymodel = Model(func)
    params = mymodel.make_params(shiftx_um=shiftx_um, _lambda_nm=_lambda_nm, z_mm=z_mm, d_um=d_um, w1_um=w1_um, w2_um=w2_um, I_w1=I_w1, I_w2=I_w2, I_Airy1=I_Airy1, I_Airy2=I_Airy2, x1_um=x1_um, x2_um=x2_um, gamma=gamma, a1=a1, b1=b1, c1=c1, e1=e1, a2=a2, b2=b2, c2=c2, e2=e2, normfactor = normfactor)

    params['shiftx_um'].vary = shiftx_um_do_fit
    params['_lambda_nm'].vary = _lambda_nm_do_fit
    params['z_mm'].vary = z_mm_do_fit
    params['d_um'].vary = d_um_do_fit
    params['w1_um'].vary = w1_um_do_fit
    params['w2_um'].vary = w2_um_do_fit
    params['I_w1'].vary = I_w1_do_fit
    params['I_w2'].vary = I_w2_do_fit
    params['I_Airy1'].vary = I_Airy1_do_fit
    params['I_Airy2'].vary = I_Airy2_do_fit
    params['x1_um'].vary = x1_um_do_fit
    params['x2_um'].vary = x2_um_do_fit
    params['gamma'].vary = gamma_do_fit
    params['a1'].vary = a1_do_fit
    params['b1'].vary = b1_do_fit
    params['c1'].vary = c1_do_fit
    params['e1'].vary = e1_do_fit
    params['a2'].vary = a2_do_fit
    params['b2'].vary = b2_do_fit
    params['c2'].vary = c2_do_fit
    params['e2'].vary = e2_do_fit
    params['normfactor'].vary = normfactor_do_fit
    
    params['w1_um'].min = 7
    params['w2_um'].min = 7
    params['I_Airy1'].min = 0
    params['I_Airy2'].min = 0
    params['gamma'].min = 0
    

    
    
    
    mymodel.set_param_hint('shiftx_um', min=shiftx_um_range[0], max=shiftx_um_range[1])
    mymodel.set_param_hint('_lambda_nm', min=_lambda_nm_range[0], max=_lambda_nm_range[1])
    mymodel.set_param_hint('z_mm', min=z_mm_range[0], max=z_mm_range[1])
    mymodel.set_param_hint('d_um', min=d_um_range[0], max=d_um_range[1])
    mymodel.set_param_hint('w1_um', min=w1_um_range[0], max=w1_um_range[1])
    mymodel.set_param_hint('w2_um', min=w2_um_range[0], max=w2_um_range[1])
    mymodel.set_param_hint('I_w1', min=I_w1_range[0], max=I_w1_range[1])
    mymodel.set_param_hint('I_w2', min=I_w2_range[0], max=I_w2_range[1])
    mymodel.set_param_hint('I_Airy1', min=I_Airy1_range[0], max=I_Airy1_range[1])
    mymodel.set_param_hint('I_Airy2', min=I_Airy2_range[0], max=I_Airy2_range[1])
    mymodel.set_param_hint('x1_um', min=x1_um_range[0], max=x2_um_range[1])
    mymodel.set_param_hint('x2_um', min=x2_um_range[0], max=x2_um_range[1])
    mymodel.set_param_hint('gamma', min=gamma_range[0], max=gamma_range[1])
    mymodel.set_param_hint('a1', min=a1_range[0], max=a1_range[1])
    mymodel.set_param_hint('b1', min=b1_range[0], max=b1_range[1])
    mymodel.set_param_hint('c1', min=c1_range[0], max=c1_range[1])
    mymodel.set_param_hint('e1', min=e1_range[0], max=e1_range[1])
    mymodel.set_param_hint('a2', min=a2_range[0], max=a2_range[1])
    mymodel.set_param_hint('b2', min=b2_range[0], max=b2_range[1])
    mymodel.set_param_hint('c2', min=c2_range[0], max=c2_range[1])
    mymodel.set_param_hint('e2', min=e2_range[0], max=e2_range[1])
    mymodel.set_param_hint('normfactor', min=e2_range[0], max=e2_range[1])
    
    
   

    
    result = mymodel.fit(ydata, params, x=xdata)
    
    shiftx_um_fit = result.params['shiftx_um'].value
    _lambda_nm_fit = result.params['_lambda_nm'].value
    z_mm_fit = result.params['z_mm'].value
    d_um_fit = result.params['d_um'].value
    w1_um_fit = result.params['w1_um'].value
    w2_um_fit = result.params['w2_um'].value
    I_w1_fit = result.params['I_w1'].value
    I_w2_fit = result.params['I_w2'].value
    I_Airy1_fit = result.params['I_Airy1'].value
    I_Airy2_fit = result.params['I_Airy2'].value
    x1_um_fit = result.params['x1_um'].value
    x2_um_fit = result.params['x2_um'].value
    gamma_fit = result.params['gamma'].value
    a1_fit = result.params['a1'].value
    b1_fit = result.params['b1'].value
    c1_fit = result.params['c1'].value
    e1_fit = result.params['e1'].value
    a2_fit = result.params['a2'].value
    b2_fit = result.params['b2'].value
    c2_fit = result.params['c2'].value
    e2_fit = result.params['e2'].value
    normfactor_fit = result.params['normfactor'].value
    
    d_um_at_detector = (x2_um_fit - x1_um_fit)
    
    fringeseparation_um = z_mm * 1e-3 * _lambda_nm_fit * 1e-9 / (d_um * 1e-6) * 1e6
    fringeseparation_px = fringeseparation_um / 13
    #print('fringeseparation_px=' + str(round(fringeseparation_px,2)))

    fig=plt.figure(figsize=(11.69,8.27), dpi= 150, facecolor='w', edgecolor='k')  # A4 sheet in landscape
    ax = plt.subplot(2,1,1)
    
#     plt.plot(list(range(pixis_profile_avg.size)),ydata, color='r', linewidth=2)
#     plt.plot(list(range(pixis_profile_avg.size)),result.best_fit, color='b', linewidth=0.5)
    plt.plot(xdata*1e3,ydata, color='r', linewidth=2, label='data')
    plt.plot(xdata*1e3,result.best_fit, color='b', linewidth=0.5, label='fit')
    
    Airy1 = [I_Airy1_fit * Airy((x-shiftx_um_fit*1e-6), w1_um_fit*1e-6, _lambda_nm_fit*1e-9, z_mm_fit*1e-3, x1_um_fit*1e-6, a1_fit, b1_fit, c1_fit, e1_fit)**2 for x in xdata]
    Airy1 = normalize(Airy1) / I_Airy2_fit
    Airy2 = [I_Airy2_fit * Airy((x-shiftx_um_fit*1e-6), w2_um_fit*1e-6, _lambda_nm_fit*1e-9, z_mm_fit*1e-3, x2_um_fit*1e-6, a2_fit, b2_fit, c2_fit, e2_fit)**2 for x in xdata]
    Airy2 = normalize(Airy2)
    
    do_plot_Airys = False
    if do_plot_Airys == True:
        plt.plot(xdata*1e3, Airy1, color='k', label='Airy1', linewidth=1)
        plt.plot(xdata*1e3, Airy2, color='grey', label='Airy2', linewidth=1)
    
    #plt.vlines([x1_loc_px_fit, pixis_centerx_px, x2_loc_px_fit],0,1)
    plt.vlines([(shiftx_um_fit+x1_um_fit)*1e-3, shiftx_um_fit*1e-3, (shiftx_um_fit+x2_um_fit)*1e-3],0,0.1)
    ax.annotate('xshift',
               xy=((shiftx_um_fit)*1e-3,0), xycoords='data',
               xytext=(0,-35), textcoords='offset points',
                bbox=dict(boxstyle="round", fc="w"),
                arrowprops=dict(arrowstyle="->"))
    ax.annotate('x2',
               xy=((shiftx_um_fit+x2_um_fit)*1e-3,0), xycoords='data',
               xytext=(0,-35), textcoords='offset points',
                bbox=dict(boxstyle="round", fc="w"),
                arrowprops=dict(arrowstyle="->"))
    ax.annotate('x1',
               xy=((shiftx_um_fit+x1_um_fit)*1e-3,0), xycoords='data',
               xytext=(0,-35), textcoords='offset points',
                bbox=dict(boxstyle="round", fc="w"),
                arrowprops=dict(arrowstyle="->"))
    ax.annotate('x2',
               xy=((shiftx_um_fit+x2_um_fit)*1e-3,0), xycoords='data',
               xytext=(0,-35), textcoords='offset points',
                bbox=dict(boxstyle="round", fc="w"),
                arrowprops=dict(arrowstyle="->"))
    #plt.hlines(0,0,n)
    
    textstr = '\n'.join((
        r'imageid=%.2f' % (imageid, ),
        r'shiftx_um=%.2f' % (result.params['shiftx_um'].value, ),
        r'$\lambda=%.2f$nm' % (result.params['_lambda_nm'].value, ),
        r'fringesepar_um=%.2f' % (fringeseparation_um, ),
        r'w1_um=%.2f' % (result.params['w1_um'].value, ),
        r'w2_um=%.2f' % (result.params['w2_um'].value, ),
        r'I_Airy1=%.2f' % (result.params['I_Airy1'].value, ),
        r'I_Airy2=%.2f' % (result.params['I_Airy2'].value, ),
        r'x1_um=%.2f' % (x1_um_fit, ),
        r'x2_um=%.2f' % (x2_um_fit, ),
        r'$\gamma=%.2f$' % (result.params['gamma'].value, ),
        r'a1=%.2f' % (result.params['a1'].value, ),
        r'a2=%.2f' % (result.params['a2'].value, ),
        r'b1=%.2f' % (result.params['b1'].value, ),
        r'b2=%.2f' % (result.params['b2'].value, ),
        r'c1=%.2f' % (result.params['c1'].value, ),
        r'c2=%.2f' % (result.params['c2'].value, ),
        r'e1=%.2f*1e-6' % (result.params['e1'].value*1e6, ),
        r'e2=%.2f*1e-6' % (result.params['e2'].value*1e6, ),
        r'normfactor=%.2f' % (result.params['normfactor'].value, ),
        r'd_um_at_detector=%.2f' % (d_um_at_detector, )))
    
    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    # place a text box in upper left in axes coords
    
    if do_textbox == True:
        ax.text(1, 0.95, textstr, transform=ax.transAxes, fontsize=6,
                verticalalignment='top', bbox=props)
    
    ax.set_xlim([(-n/2)*dX_1*1e3, (+n/2-1)*dX_1*1e3])
    ax.set_ylim([0, 1])
    
    plt.ylabel('Intensity / a.u.', fontsize = 14)
    plt.legend()

    textstr = ' '.join((
        'ph-'+ph+'.id'+str(int(imageid)),
        r'$\lambda=%.2f$nm' % (result.params['_lambda_nm'].value, ),
        orientation,
        '$d$='+str(int(separation_um))+'um',
        r'$d_{det}$=%.2fum' % (d_um_at_detector, ),
        r'$w_1$=%.2fum' % (result.params['w1_um'].value, ),
        r'$w_2$=%.2fum' % (result.params['w2_um'].value, ),
        r'$I_1$=%.2f' % (result.params['I_Airy1'].value, ),
        r'$I_2=$%.2f' % (result.params['I_Airy2'].value, ),
        r'$\gamma=%.2f$' % (result.params['gamma'].value, ))
        )
    plt.title(textstr, fontsize=12)
    
    

    ax2 = plt.subplot(2,1,2)
    
    im_ax2 = ax2.imshow(pixis_image_norm_dataset[imageid], origin='lower', interpolation='nearest', aspect='auto', cmap='jet', vmin=0, vmax=1, extent=((-n/2)*dX_1*1e3, (+n/2-1)*dX_1*1e3, -n/2*dX_1*1e3, (+n/2-1)*dX_1*1e3))
    
    #fig.colorbar(im_ax2, ax=ax2, pad=0.05, fraction=0.1, shrink=1.00, aspect=20, orientation='horizontal')

    ax2.add_patch(
       patches.Rectangle(
        ((-n/2)*dX_1*1e3, (pixis_centery_px_widget.value-n/2-pixis_avg_width_widget.value/2)*dX_1*1e3),
        n*dX_1*1e3,
        pixis_avg_width_widget.value*dX_1*1e3,
        color = 'w',
        linestyle = '-',
        alpha = 0.8,
        fill=False      # remove background
        )
    )
        
    ax2.set_xlabel('x / mm', fontsize = 14)
    ax2.set_ylabel('y / mm', fontsize = 14)
    ax2.grid(color='w', linewidth=1, alpha=0.5, linestyle='--', which='major')

    if savefigure == True:
        savefigure_dir = scratch_dir + str(settings_widget.label)
        if os.path.isdir(savefigure_dir) == False:
            os.mkdir(savefigure_dir)
        #savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
        savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value))
        if os.path.isdir(savefigure_dir) == False:
            os.mkdir(savefigure_dir)
        savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/profiles_fit/'
        if os.path.isdir(savefigure_dir) == False:
            os.mkdir(savefigure_dir)
        plt.savefig(savefigure_dir + '/' + 'profiles_fit_' + hdf5_file_name_image_widget.value \
            + '_ph_'+str(ph) \
            + '_d_'+str(separation_um) \
            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
            + '_image_'+str(imageid_loop) \
            + '.png', dpi=300, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)
        plt.savefig(savefigure_dir + '/' + 'profiles_fit_' + hdf5_file_name_image_widget.value \
            + '_ph_'+str(ph) \
            + '_d_'+str(separation_um) \
            + '_E_' + str(format(energy_hall_uJ, '.4f')).zfill(6)  \
            + '_image_'+str(imageid_loop) \
            + '.pdf', dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None)
        
    if save_to_df == True:
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid']==imageid)), 'gamma_fit'] = gamma_fit
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid']==imageid)), 'd_um_at_detector'] = d_um_at_detector
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid']==imageid)), 'I_Airy1'] = I_Airy1
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid']==imageid)), 'I_Airy2'] = I_Airy2
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid']==imageid)), 'w1_um'] = w1_um
        df0.loc[((df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid']==imageid)), 'w2_um'] = w2_um
    
    
    
    plt.show()

     
    #print(gamma_fit)
    

n = pixis_profile_avg.size # number of sampling point  # number of pixels    

imageid_profile_fit_widget = widgets.Dropdown(
    options=imageid_widget.options,
    description='imageid:',
    disabled=False,
)

savefigure_profile_fit_widget = widgets.Checkbox(
    value=False,
    description='savefigure',
    disabled=False
)

save_to_df_widget = widgets.Checkbox(
    value=False,
    description='save_to_df',
    disabled=False
)

do_textbox_widget = widgets.Checkbox(
    value=False,
    description='do_textbox',
    disabled=False
)

shiftx_um_widget = widgets.FloatSlider(min=-n/2*13, max=n/2*13, value=477, step=1, description='shiftx_um')
_lambda_nm_widget = widgets.FloatSlider(value=_lambda_widget.value, description='_lambda_nm')
z_mm_widget = widgets.FloatSlider(min=5000.0, max=6000.0, value=5781.0, description='z_mm')
d_um_widget = widgets.FloatSlider(min=107, max= 1337, value=707, description='d_um')
gamma_widget = widgets.FloatSlider(min=0, max=2.0, value=0.8, description='gamma') 
w1_um_widget = widgets.FloatSlider(min=5, max=20, value=10.00, description='w1_um') 
w2_um_widget = widgets.FloatSlider(min=5, max=20, value=10.00, description='w2_um')
I_w1_widget = widgets.FloatSlider(min=0, max=10, value=0, description='I_w1')
I_w2_widget = widgets.FloatSlider(min=0, max=10, value=0, description='I_w2')
I_Airy1_widget = widgets.FloatSlider(min=0, max=10, value=1.0, description='I_Airy1') 
I_Airy2_widget = widgets.FloatSlider(min=0, max=10, value=0.8, description='I_Airy2') 
x1_um_widget = widgets.FloatSlider(min=-n*13/2, max=0, value=-n*13/2 /2, step=0.1, description='x1_um') 
x2_um_widget = widgets.FloatSlider(min=0, max=2*n*13, value=n*13/2 / 2, step=0.1, description='x2_um')
a1_widget = widgets.FloatSlider(min=0, max=5, value=1.0, step=0.01, description='a1')
b1_widget = widgets.FloatSlider(min=0, max=5, value=1.0, step=0.01, description='b1') 
c1_widget = widgets.FloatSlider(min=0, max=5, value=1.0, step=0.01, description='c1') 
e1_widget = widgets.FloatSlider(min=0.0000, max=0.001, value=0.0, step=0.0001, description='e1', readout_format='.2e') 
a2_widget = widgets.FloatSlider(min=0, max=5, value=1.0, step=0.01, description='a2') 
b2_widget = widgets.FloatSlider(min=0, max=5, value=1.0, step=0.01, description='b2') 
c2_widget = widgets.FloatSlider(min=0, max=5, value=1.0, step=0.01, description='c2') 
e2_widget = widgets.FloatSlider(min=0.0000, max=0.0010, value=0.0, step=0.0001, description='e2', readout_format='.2e')
normfactor_widget = widgets.FloatSlider(min=0.00, max=10, value=1.0, step=0.1, description='normfactor', readout_format='.2f')
datafactor_widget = widgets.FloatSlider(min=0.00, max=10, value=1.0, step=0.1, description='datafactor', readout_format='.2f')

shiftx_um_range_widget = widgets.FloatRangeSlider(min=-n/2*13, max=n/2*13, value=[-600, 600], step=1, description='shiftx_um')
_lambda_nm_range_widget = widgets.FloatRangeSlider(min=7, max=19, value=[_lambda_nm_widget.value-1, _lambda_nm_widget.value+1], description='_lambda_nm')
z_mm_range_widget = widgets.FloatRangeSlider(min=5000.0, max=6000.0, value=[5770.0, 5790], description='z_mm')
d_um_range_widget = widgets.FloatRangeSlider(min=107, max= 1337, value=[440.0, 450.0], description='d_um')
gamma_range_widget = widgets.FloatRangeSlider(min=0, max=2.0, value=[0.5, 1.8], description='gamma') 
w1_um_range_widget = widgets.FloatRangeSlider(min=5, max=20, value=[8, 13], description='w1_um') 
w2_um_range_widget = widgets.FloatRangeSlider(min=5, max=20, value=[8, 13], description='w2_um')
I_w1_range_widget = widgets.FloatRangeSlider(min=0, max=10, value=[0.2, 1.5], description='I_w1')
I_w2_range_widget = widgets.FloatRangeSlider(min=0, max=10, value=[0.2, 1.5], description='I_w2')
I_Airy1_range_widget = widgets.FloatRangeSlider(min=0, max=10, value=[0.2, 1.5], description='I_Airy1') 
I_Airy2_range_widget = widgets.FloatRangeSlider(min=0, max=10, value=[0.2, 1.5], description='I_Airy2') 
x1_um_range_widget = widgets.FloatRangeSlider(min=-n*13, max=0, value=[-n*13, 0], step=0.1, description='x1_um') 
x2_um_range_widget = widgets.FloatRangeSlider(min=0, max=n*13, value=[0, n*13], step=0.1, description='x2_um')
a1_range_widget = widgets.FloatRangeSlider(min=0, max=5, value=[0, 5], step=0.01, description='a1')
b1_range_widget = widgets.FloatRangeSlider(min=0, max=5, value=[0, 5], step=0.01, description='b1') 
c1_range_widget = widgets.FloatRangeSlider(min=0, max=5, value=[0, 5], step=0.01, description='c1') 
e1_range_widget = widgets.FloatRangeSlider(min=0.0000, max=0.001, value=[0.0000, 0.0010], step=0.0001, description='e1', readout_format='.2e') 
a2_range_widget = widgets.FloatRangeSlider(min=0, max=5, value=[0, 5], step=0.01, description='a2') 
b2_range_widget = widgets.FloatRangeSlider(min=0, max=5, value=[0, 5], step=0.01, description='b2') 
c2_range_widget = widgets.FloatRangeSlider(min=0, max=5, value=[0, 5], step=0.01, description='c2') 
e2_range_widget = widgets.FloatRangeSlider(min=0.0000, max=0.0010, value=[0.0000, 0.0010], step=0.0001, description='e2', readout_format='.2e')
normfactor_range_widget = widgets.FloatRangeSlider(min=0, max=10, value=[0.5, 1.5], step=0.01, description='normfactor', readout_format='.2f')

shiftx_um_do_fit_widget = widgets.Checkbox(value=True, description='fit')
_lambda_nm_do_fit_widget = widgets.Checkbox(value=True, description='fit')
z_mm_do_fit_widget = widgets.Checkbox(value=False, description='fit')
d_um_do_fit_widget = widgets.Checkbox(value=False, description='fit')
gamma_do_fit_widget = widgets.Checkbox(value=True, description='fit')
w1_um_do_fit_widget = widgets.Checkbox(value=True, description='fit') 
w2_um_do_fit_widget = widgets.Checkbox(value=True, description='fit')
I_w1_do_fit_widget = widgets.Checkbox(value=False, description='fit')
I_w2_do_fit_widget = widgets.Checkbox(value=False, description='fit')
I_Airy1_do_fit_widget = widgets.Checkbox(value=False, description='fit')
I_Airy2_do_fit_widget = widgets.Checkbox(value=True, description='fit') 
x1_um_do_fit_widget = widgets.Checkbox(value=True, description='fit')
x2_um_do_fit_widget = widgets.Checkbox(value=True, description='fit')
a1_do_fit_widget = widgets.Checkbox(value=False, description='fit')
b1_do_fit_widget = widgets.Checkbox(value=False, description='fit')
c1_do_fit_widget = widgets.Checkbox(value=False, description='fit')
e1_do_fit_widget = widgets.Checkbox(value=False, description='fit')
a2_do_fit_widget = widgets.Checkbox(value=False, description='fit')
b2_do_fit_widget = widgets.Checkbox(value=False, description='fit')
c2_do_fit_widget = widgets.Checkbox(value=False, description='fit')
e2_do_fit_widget = widgets.Checkbox(value=False, description='fit')
normfactor_do_fit_widget = widgets.Checkbox(value=False, description='fit')


column0 = widgets.VBox([imageid_profile_fit_widget,
                       savefigure_profile_fit_widget,
                       save_to_df_widget,
                       do_textbox_widget])

column1 = widgets.VBox([shiftx_um_widget,
                        _lambda_nm_widget,
                        z_mm_widget,
                        d_um_widget,
                        gamma_widget,
                        w1_um_widget,
                        w2_um_widget,
                        I_w1_widget,
                        I_w2_widget,
                        I_Airy1_widget,
                        I_Airy2_widget,
                        x1_um_widget,
                        x2_um_widget,
                        a1_widget,
                        b1_widget,
                        c1_widget,
                        e1_widget,
                        a2_widget,
                        b2_widget,
                        c2_widget,
                        e2_widget,
                       normfactor_widget,
                       datafactor_widget])

column2 = widgets.VBox([shiftx_um_do_fit_widget,
                        _lambda_nm_do_fit_widget,
                        z_mm_do_fit_widget,
                        d_um_do_fit_widget,
                        gamma_do_fit_widget,
                        w1_um_do_fit_widget,
                        w2_um_do_fit_widget,
                        I_w1_do_fit_widget,
                        I_w2_do_fit_widget,
                        I_Airy1_do_fit_widget,
                        I_Airy2_do_fit_widget,
                        x1_um_do_fit_widget,
                        x2_um_do_fit_widget,
                        a1_do_fit_widget,
                        b1_do_fit_widget,
                        c1_do_fit_widget,
                        e1_do_fit_widget,
                        a2_do_fit_widget,
                        b2_do_fit_widget,
                        c2_do_fit_widget,
                        e2_do_fit_widget,
                       normfactor_do_fit_widget])

column3 = widgets.VBox([shiftx_um_range_widget,
                        _lambda_nm_range_widget,
                        z_mm_range_widget,
                        d_um_range_widget,
                        gamma_range_widget,
                        w1_um_range_widget,
                        w2_um_range_widget,
                        I_w1_range_widget,
                        I_w2_range_widget,
                        I_Airy1_range_widget,
                        I_Airy2_range_widget,
                        x1_um_range_widget,
                        x2_um_range_widget,
                        a1_range_widget,
                        b1_range_widget,
                        c1_range_widget,
                        e1_range_widget,
                        a2_range_widget,
                        b2_range_widget,
                        c2_range_widget,
                        e2_range_widget,
                       normfactor_range_widget])

plotprofile_interactive_input = widgets.HBox([column0, column1, column2, column3])

plotprofile_interactive_output = interactive_output(plotprofile,
                                                   {
                                                        'imageid' : imageid_profile_fit_widget,
                                                        'savefigure' : savefigure_profile_fit_widget,
                                                        'save_to_df' : save_to_df_widget,
                                                       'do_textbox' : do_textbox_widget,
                                                        'shiftx_um' : shiftx_um_widget,
                                                        '_lambda_nm' : _lambda_nm_widget,
                                                        'z_mm' : z_mm_widget,
                                                        'd_um' : d_um_widget,
                                                        'gamma' : gamma_widget,
                                                        'w1_um' : w1_um_widget,
                                                        'w2_um' : w2_um_widget,
                                                        'I_w1' : I_w1_widget,
                                                        'I_w2' : I_w2_widget,
                                                        'I_Airy1' : I_Airy1_widget,
                                                        'I_Airy2' : I_Airy2_widget,
                                                        'x1_um' : x1_um_widget,
                                                        'x2_um' : x2_um_widget,
                                                        'a1' : a1_widget,
                                                        'b1' : b1_widget,
                                                        'c1' : c1_widget,
                                                        'e1' : e1_widget,
                                                        'a2' : a2_widget,
                                                        'b2' : b2_widget,
                                                        'c2' : c2_widget,
                                                        'e2' : e2_widget,
                                                       'normfactor' : normfactor_widget,
                                                       'datafactor' : datafactor_widget,
                                                        'shiftx_um_range' : shiftx_um_range_widget,
                                                        '_lambda_nm_range' : _lambda_nm_range_widget,
                                                        'z_mm_range' : z_mm_range_widget,
                                                        'd_um_range' : d_um_range_widget,
                                                        'gamma_range' : gamma_range_widget,
                                                        'w1_um_range' : w1_um_range_widget,
                                                        'w2_um_range' : w2_um_range_widget,
                                                        'I_w1_range' : I_w1_range_widget,
                                                        'I_w2_range' : I_w2_range_widget,
                                                        'I_Airy1_range' : I_Airy1_range_widget,
                                                        'I_Airy2_range' : I_Airy2_range_widget,
                                                        'x1_um_range' : x1_um_range_widget,
                                                        'x2_um_range' : x2_um_range_widget,
                                                        'a1_range' : a1_range_widget,
                                                        'b1_range' : b1_range_widget,
                                                        'c1_range' : c1_range_widget,
                                                        'e1_range' : e1_range_widget,
                                                        'a2_range' : a2_range_widget,
                                                        'b2_range' : b2_range_widget,
                                                        'c2_range' : c2_range_widget,
                                                        'e2_range' : e2_range_widget,
                                                       'normfactor_range' : normfactor_range_widget,
                                                        'shiftx_um_do_fit' : shiftx_um_do_fit_widget,
                                                        '_lambda_nm_do_fit' : _lambda_nm_do_fit_widget,
                                                        'z_mm_do_fit' : z_mm_do_fit_widget,
                                                        'd_um_do_fit' : d_um_do_fit_widget,
                                                        'gamma_do_fit' : gamma_do_fit_widget,
                                                        'w1_um_do_fit' : w1_um_do_fit_widget,
                                                        'w2_um_do_fit' : w2_um_do_fit_widget,
                                                        'I_w1_do_fit' : I_w1_do_fit_widget,
                                                        'I_w2_do_fit' : I_w2_do_fit_widget,
                                                        'I_Airy1_do_fit' : I_Airy1_do_fit_widget,
                                                        'I_Airy2_do_fit' : I_Airy2_do_fit_widget,
                                                        'x1_um_do_fit' : x1_um_do_fit_widget,
                                                        'x2_um_do_fit' : x2_um_do_fit_widget,
                                                        'a1_do_fit' : a1_do_fit_widget,
                                                        'b1_do_fit' : b1_do_fit_widget,
                                                        'c1_do_fit' : c1_do_fit_widget,
                                                        'e1_do_fit' : e1_do_fit_widget,
                                                        'a2_do_fit' : a2_do_fit_widget,
                                                        'b2_do_fit' : b2_do_fit_widget,
                                                        'c2_do_fit' : c2_do_fit_widget,
                                                        'e2_do_fit' : e2_do_fit_widget,
                                                       'normfactor_do_fit' : normfactor_do_fit_widget
                                                       
                                                   })

In [121]:
display(VBox([plotprofile_interactive_input, plotprofile_interactive_output]))

In [57]:
ph = dataset_image_args_widget.value[2]

imageid_sequence_by_energy_hall = get_imageids_with_bgs(beamposition_horizontal_interval_widget.value)
df = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & df0['imageid'].isin(imageid_sequence_by_energy_hall)]
energy_hall_uJ = df[df['imageid']==imageid_widget.value]['energy hall'].iloc[0]

In [85]:
df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & (df0['imageid']==imageid_widget.value)]['gamma_fit'].iloc[0]

0.9410213408218133

In [100]:
df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value)]['gamma_fit'].unique()

array([       nan, 0.94102134, 0.84303043, 0.06881228, 0.38789747,
       0.30428828])

In [117]:
df0[df0['gamma_fit']==df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value)]['gamma_fit'].unique()[3]]['separation_um']

4240    707.0
Name: separation_um, dtype: float64

In [ ]:
dX_1 = 13e-6
xdata = np.linspace((-n/2)*dX_1, (+n/2-1)*dX_1, n)
#ydata = pixis_profile_avg_dataset[imageid]

shiftx_um_fit = 477
_lambda_nm_fit = 7.89
z_mm_fit = 5781
d_um_fit = 707
w1_um_fit = 10
w2_um_fit = 10
I_w1_fit = 0
I_w2_fit = 0
I_Airy1_fit = 1
I_Airy2_fit = 1
x1_um_fit = -3000
x2_um_fit = 3000
gamma_fit = 0.37
a1_fit = 1
b1_fit = 1
c1_fit = 1
e1_fit = 0
a2_fit = 1
b2_fit = 1
c2_fit = 1
e2_fit = 0

Airy1 = [I_Airy1_fit * Airy((x-shiftx_um_fit*1e-6), w1_um_fit*1e-6, _lambda_nm_fit*1e-9, z_mm_fit*1e-3, x1_um_fit*1e-6, a1_fit, b1_fit, c1_fit, e1_fit)**2 for x in xdata]
plt.plot(xdata*1e3, Airy1, color='k', label='Airy1', linewidth=2)

In [ ]:
plt.figure(figsize=(11.69,8.27), dpi=300)

xdata = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['imageid']]
ydata = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['xi_um']]
ydata_mean = ydata.mean(axis=0)[0]
ydata_std = ydata.std(axis=0)[0]

plt.subplot(1,1,1)
plt.plot(xdata, ydata, 'o')
plt.axhline(ydata_mean, label='mean')
plt.axhline(ydata_mean-ydata_std, linestyle='--', label='std')
plt.axhline(ydata_mean+ydata_std, linestyle='--')
plt.axhline(xi_um_of_average, color='r', label='averaged image: $\\xi$ / $\\mu$m = ' + str(round(xi_um_of_average,2)))
plt.title('coherence length $\\xi$ / $\mu$m = ' + str(round(ydata_mean,2)) + ' $\\pm$ ' + str(round(ydata_std,2)))
plt.xlabel('image id')
plt.ylabel('coherence length $\\xi$ / $\mu$m')
plt.legend()



#xdata_name = 'energy hall'
xdata_name = 'beam position hall horizontal pulse resolved'
ydata_name = 'beam position hall vertical pulse resolved'
#xdata_name = '_lambda_nm_fit'
#xdata_name = 'mean wavelength'
# xdata = list(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & df0['xi_um'].notnull()][[xdata_name]])
# ydata = list(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & df0['xi_um'].notnull()][[ydata_name]])
# colordata = list(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) & df0['xi_um'].notnull()][['xi_um']])

savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'coherencelength_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'coherencelength_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()


plt.figure(figsize=(11.69,8.27), dpi=300)

xdata = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['imageid']]
ydata = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['pixis_profile_centerx_px_fit']]
ydata_mean = ydata.mean(axis=0)[0]
ydata_std = ydata.std(axis=0)[0]

plt.subplot(1,1,1)
plt.plot(xdata, ydata, 'o')
plt.axhline(ydata_mean, label='mean')
plt.axhline(ydata_mean-ydata_std, linestyle='--', label='std')
plt.axhline(ydata_mean+ydata_std, linestyle='--')
#plt.title('coherence length $\\xi$ / $\mu$m = ' + str(round(ydata_mean,2)) + ' $\\pm$ ' + str(round(ydata_std,2)))
plt.xlabel('image id')
plt.ylabel('pixis_profile_centerx_px_fit')
plt.legend()



savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'pixis_profile_centerx_px_fit_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'pixis_profile_centerx_px_fit_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()


plt.figure(figsize=(11.69,8.27), dpi=300)

xdata = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['imageid']]
ydata1 = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['_lambda_nm_fit']]
ydata2 = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['mean wavelength']]
ydata3 = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['setting_wavelength_nm']]
ax = plt.subplot(1,1,1)
ax.plot(xdata, ydata1, 'o', label='fitted from profile')
if ydata2.mean(axis=0)[0] > 0:
    plt.plot(xdata, ydata2, 'x', label='mean wavelength from GMD')
ax.plot(xdata, ydata3, '-', label='machine setting')
plt.xlabel('image id')
plt.ylabel('wavelength / nm')

ax.legend()

if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'wavelength_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'wavelength_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()



fig = plt.figure(figsize=(11.69,8.27), dpi=300)

ax = plt.subplot(1,1,1)
# ax.plot(xdata, ydata, c=colordata)
# plt.xlabel(xdata_name)
# plt.ylabel('coherence length')
ax = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot(kind='scatter', x='beam position hall horizontal pulse resolved', y='beam position hall vertical pulse resolved', c='xi_um', s=10, colormap='jet', ax=ax)
ax.axis('equal')

#if savefigure == True:
#savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'coherencelength_vs_position_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'coherencelength_vs_position_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()


fig = plt.figure(figsize=(11.69,8.27), dpi=300)

ax = plt.subplot(1,1,1)
# ax.plot(xdata, ydata, c=colordata)
# plt.xlabel(xdata_name)
# plt.ylabel('coherence length')
ax = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot(kind='scatter', x='pinholes_cm_x_px', y='pinholes_cm_y_px', c='xi_um', s=10, colormap='jet', ax=ax)
ax.axis('equal')

#if savefigure == True:
#savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'xi_vs_yag_cm_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'xi_vs_yag_cm_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()


fig = plt.figure(figsize=(11.69,8.27), dpi=300)

ax = plt.subplot(1,1,1)
# ax.plot(xdata, ydata, c=colordata)
# plt.xlabel(xdata_name)
# plt.ylabel('coherence length')
ax = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot(kind='scatter', x='pinholes_cm_x_px', y='pinholes_cm_y_px', c='energy hall', s=10, colormap='jet', ax=ax)
ax.axis('equal')

#if savefigure == True:
#savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'energy_vs_yag_cm_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'energy_vs_yag_cm_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()


fig = plt.figure(figsize=(11.69,8.27), dpi=300)

ax = plt.subplot(1,1,1)
# ax.plot(xdata, ydata, c=colordata)
# plt.xlabel(xdata_name)
# plt.ylabel('coherence length')
ax = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot(kind='scatter', x='beam position hall horizontal pulse resolved', y='beam position hall vertical pulse resolved', c='energy hall', s=10, colormap='jet', ax=ax)
ax.axis('equal')

#if savefigure == True:
#savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'energy_vs_position_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'energy_vs_position_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()


fig = plt.figure(figsize=(11.69,8.27), dpi=300)

ax = plt.subplot(1,1,1)
# ax.plot(xdata, ydata, c=colordata)
# plt.xlabel(xdata_name)
# plt.ylabel('coherence length')
ax = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot(kind='scatter', x='pinholes_cm_x_px', y='pinholes_cm_y_px', c='pixis_profile_centerx_px_fit', s=10, colormap='jet', ax=ax)
ax.axis('equal')

#if savefigure == True:
#savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'pixis_profile_centerx_px_fit_vs_yag_cm_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'pixis_profile_centerx_px_fit_vs_yag_cm_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()


fig = plt.figure(figsize=(11.69,8.27), dpi=300)

ax = plt.subplot(1,1,1)
# ax.plot(xdata, ydata, c=colordata)
# plt.xlabel(xdata_name)
# plt.ylabel('coherence length')
ax = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot(kind='scatter', x='beam position hall horizontal pulse resolved', y='beam position hall vertical pulse resolved', c='pixis_profile_centerx_px_fit', s=10, colormap='jet', ax=ax)
ax.axis('equal')

#if savefigure == True:
#savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'pixis_profile_centerx_px_fit_vs_beamposition' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'pixis_profile_centerx_px_fit_vs_beamposition' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()

fig = plt.figure(figsize=(11.69,8.27), dpi=300)

ax = plt.subplot(1,1,1)
# ax.plot(xdata, ydata, c=colordata)
# plt.xlabel(xdata_name)
# plt.ylabel('coherence length')
ax = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot(kind='scatter', x='energy hall', y='xi_um', c='_lambda_nm_fit', s=20, colormap='jet', ax=ax)
#ax.axis('equal')

#if savefigure == True:
#savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
savefigure_dir = scratch_dir + str(settings_widget.label)
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'xi_um_vs_energy_hall_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'xi_um_vs_energy_hall_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()


fig = plt.figure(figsize=(11.69,8.27), dpi=300)

ax = plt.subplot(1,1,1)
# ax.plot(xdata, ydata, c=colordata)
# plt.xlabel(xdata_name)
# plt.ylabel('coherence length')
ax = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot(kind='scatter', x='_lambda_nm_fit', y='xi_um', s=20, ax=ax)
#ax.axis('equal')

#if savefigure == True:
#savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
savefigure_dir = scratch_dir + str(settings_widget.label)
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'xi_um_vs_lambda_fit_' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'xi_um_vs_lambda_fit_' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()


fig = plt.figure(figsize=(11.69,8.27), dpi=300)

ax = plt.subplot(1,1,1)
# ax.plot(xdata, ydata, c=colordata)
# plt.xlabel(xdata_name)
# plt.ylabel('coherence length')
ax = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot(kind='scatter', x='energy hall', y='_lambda_nm_fit', s=20, ax=ax)
#ax.axis('equal')

#if savefigure == True:
#savefigure_dir = scratch_dir + hdf5_file_name_image + '_ph_'+str(ph) + '_d_'+str(separation_um)
savefigure_dir = scratch_dir + str(settings_widget.label)
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
savefigure_dir = scratch_dir + str(settings_widget.label) + '/' + 'profilewidth_px_' + str(int(pixis_avg_width_widget.value)) + '_' + 'bg_intervall_um_' + str(int(beamposition_horizontal_interval_widget.value)) + '/plots/'
if os.path.isdir(savefigure_dir) == False:
    os.mkdir(savefigure_dir)
plt.savefig(savefigure_dir + '/' + 'lambda_fit_vs_energy_hall' + hdf5_file_name_image_widget.value \
        + '.png', dpi=300, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.savefig(savefigure_dir + '/' + 'lambda_fit_vs_energy_hall' + hdf5_file_name_image_widget.value \
        + '.pdf', dpi=None, facecolor='w', edgecolor='w',
orientation='portrait', papertype=None, format=None,
transparent=False, bbox_inches=None, pad_inches=0.1,
frameon=None)

plt.show()

In [ ]:
df = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2]) &
                     (df0['background'] == dataset_bg_args_widget.value[2]) &
                     (df0['aperture5_mm'] == dataset_image_args_widget.value[5]) &
                     (df0['aperture7_mm'] == dataset_image_args_widget.value[6])]
beamposx = df.iloc[imageid_widget.value]['beam position hall horizontal pulse resolved']
beamposy = df.iloc[imageid_widget.value]['beam position hall vertical pulse resolved']


fig, ax = plt.subplots()
df.plot(kind='scatter', x='beam position hall horizontal pulse resolved', y='beam position hall vertical pulse resolved', c='energy hall', s=10, colormap='jet', ax=ax)
plt.scatter(beamposx, beamposy, marker='x', s=30, color='k')
ax.set_aspect('equal')
plt.show()

In [ ]:
fig, ax = plt.subplots()
#plt.scatter('beam position hall horizontal pulse resolved', '_lambda_nm_fit', c='energy hall', s=10, cmap='jet', ax=ax)
#plt.scatter(pinholes_cm_x_px,pinholes_cm_y_px, c=list(df['energy hall']), s=10, cmap='jet')
img = ax.scatter(df['beam position hall horizontal pulse resolved'], df['beam position hall vertical pulse resolved'], c=_lambda_nm_fit, s=10, cmap='jet')
fig.colorbar(img, ax=ax)
# ax6.scatter(pinholes_cm_x_px[imageid_loop],pinholes_cm_y_px[imageid_loop], marker='x', s=60, color='k')
ax.set_xlabel('beam position hall horizontal pulse resolved')
ax.set_ylabel('beam position hall vertical pulse resolved')
ax.set_aspect('equal')
# plt.scatter(beamposx, beamposy, marker='x', s=30, color='k')
plt.show()

In [ ]:
fig, ax = plt.subplots()
#plt.scatter('beam position hall horizontal pulse resolved', '_lambda_nm_fit', c='energy hall', s=10, cmap='jet', ax=ax)
#plt.scatter(pinholes_cm_x_px,pinholes_cm_y_px, c=list(df['energy hall']), s=10, cmap='jet')
img = ax.scatter(df['beam position hall horizontal pulse resolved'], df['beam position hall vertical pulse resolved'], c=pixis_profile_centerx_px_fit, s=10, cmap='jet')
fig.colorbar(img, ax=ax)
# ax6.scatter(pinholes_cm_x_px[imageid_loop],pinholes_cm_y_px[imageid_loop], marker='x', s=60, color='k')
ax.set_xlabel('beam position hall horizontal pulse resolved')
ax.set_ylabel('beam position hall vertical pulse resolved')
ax.set_aspect('equal')
# plt.scatter(beamposx, beamposy, marker='x', s=30, color='k')
plt.show()

In [ ]:
fig, ax = plt.subplots()
#plt.scatter('beam position hall horizontal pulse resolved', '_lambda_nm_fit', c='energy hall', s=10, cmap='jet', ax=ax)
#plt.scatter(pinholes_cm_x_px,pinholes_cm_y_px, c=list(df['energy hall']), s=10, cmap='jet')
img = ax.scatter(pinholes_cm_x_px,pinholes_cm_y_px, c=_lambda_nm_fit, s=10, cmap='jet')
fig.colorbar(img, ax=ax)
# ax6.scatter(pinholes_cm_x_px[imageid_loop],pinholes_cm_y_px[imageid_loop], marker='x', s=60, color='k')
# ax6.set_xlabel('pinholes CM x / px')
# ax6.set_ylabel('pixis profile center / px')
# ax6.set_aspect('equal')
# plt.scatter(beamposx, beamposy, marker='x', s=30, color='k')
plt.show()

In [ ]:
fig, ax = plt.subplots()
#plt.scatter('beam position hall horizontal pulse resolved', '_lambda_nm_fit', c='energy hall', s=10, cmap='jet', ax=ax)
#plt.scatter(pinholes_cm_x_px,pinholes_cm_y_px, c=list(df['energy hall']), s=10, cmap='jet')
img = ax.scatter(pinholes_cm_x_px,pinholes_cm_y_px, c=pixis_profile_centerx_px_fit, s=10, cmap='jet')
fig.colorbar(img, ax=ax)
# ax6.scatter(pinholes_cm_x_px[imageid_loop],pinholes_cm_y_px[imageid_loop], marker='x', s=60, color='k')
# ax6.set_xlabel('pinholes CM x / px')
# ax6.set_ylabel('pixis profile center / px')
# ax6.set_aspect('equal')
# plt.scatter(beamposx, beamposy, marker='x', s=30, color='k')
plt.show()

In [ ]:
# beam position colored by beam of the images:
df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot.scatter(x='beam position hall horizontal pulse resolved',
       y='beam position hall vertical pulse resolved', c='energy hall', colormap='jet')

In [ ]:
# beam position colored by beam of the bgs:
df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2])].plot.scatter(x='beam position hall horizontal pulse resolved',
       y='beam position hall vertical pulse resolved', c='energy hall', colormap='jet')

In [ ]:
df_all.columns

In [ ]:
df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])]['energy hall']

In [ ]:
df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2])]['energy hall']

In [ ]:
imageid = imageid_widget.value
energy_interval = 1
energy_image = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df_all['pinholes'] == dataset_image_args_widget.value[2])]['energy hall'][imageid]
matching_bg_indices = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2]) & (df_all['energy hall'] > energy_image - energy_interval/2 ) & (df_all['energy hall'] < energy_image + energy_interval/2 ) ]['energy hall'].index
if matching_bg_indices.empty:
    print('empty!')
matching_bg_indices
image = pixis_dataset[imageid]
background = np.mean(pixis_bg_dataset[matching_bg_indices], axis=0)
energy_bg_mean = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2])]['energy hall'][matching_bg_indices].mean(axis=0)
energy_bg_std = df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2])]['energy hall'][matching_bg_indices].std(axis=0)

plt.figure(figsize=(30,10))

plt.subplot(1,3,1)
plt.imshow(image)
plt.subplot(1,3,2)
plt.imshow(background)

plt.title('E=' + str(round(energy_image,2)) + ' E_bg=' + str(round(energy_bg_mean,2)) + '+/-' + str(round(energy_bg_std,2)))

In [ ]:
df_all[(df_all['hdf5_file_name'] == hdf5_file_name_bg_widget.value) & (df_all['pinholes'] == dataset_bg_args_widget.value[2])]['energy hall'][matching_bg_indices].mean(axis=0)

In [ ]:
df0_energy = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['imageid','energy hall', 'setting_energy_uJ']]
plt.figure()
plt.plot(df0_energy.index,df0_energy['energy hall'])
plt.plot(df0_energy.index[imageid_widget.value],df0_energy.iloc[imageid_widget.value]['energy hall'], marker='o')



In [ ]:
plt.imshow(pixis_image_norm_dataset[0]) # but this is already subtracted with the average bg!

In [ ]:
plt.imshow(pixis_bg_dataset[0])

In [ ]:
df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['energy hall', 'setting_energy_uJ']]

In [ ]:
df_fits.columns

In [ ]:
plt.scatter(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['imageid'].tolist(), _lambda_nm_fit[0:-1])  # why is there one more fit than imageids?

In [ ]:
len(_lambda_nm_fit[:-1])

In [ ]:
[df_fits['timestamp_pulse_id'].isin(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['timestamp_pulse_id'])]['_lambda_nm_fit'].plot()

In [ ]:
#df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['average energy hall']

df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['average energy tunnel','average energy hall', 'energy aux tunnel', 'energy aux hall', 'energy hall', 'energy tunnel', 'setting_energy_uJ']].plot()

df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['_lambda_nm_fit']].plot()

df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['mean wavelength', '_lambda_nm_fit']].plot()

df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['beam position hall horizontal average',
       'beam position hall vertical average',
       'beam position tunnel horizontal average',
       'beam position tunnel vertical average',
       'beam position hall horizontal pulse resolved',
       'beam position hall vertical pulse resolved',
       'beam position tunnel horizontal pulse resolved',
       'beam position tunnel vertical pulse resolved']].plot()

df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value)][['beam position hall horizontal average',
       'beam position hall vertical average',
       'beam position tunnel horizontal average',
       'beam position tunnel vertical average',
       'beam position hall horizontal pulse resolved',
       'beam position hall vertical pulse resolved',
       'beam position tunnel horizontal pulse resolved',
       'beam position tunnel vertical pulse resolved']].plot()

df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['energy hall', 'setting_energy_uJ']].plot()
df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['energy hall', 'setting_energy_uJ']].plot(marker='o')

df0_energy = df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['energy hall', 'setting_energy_uJ']]
plt.figure()
plt.plot(df0_energy.index,df0_energy['energy hall'])
plt.plot(df0_energy.index[5],df0_energy.iloc[5]['energy hall'], marker='o')


df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])][['beam position hall horizontal pulse resolved',
       'beam position hall vertical pulse resolved']].plot()

df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])].plot.scatter(x='beam position hall horizontal pulse resolved',
       y='beam position hall vertical pulse resolved', c='energy hall', colormap='jet')


df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value)].plot.scatter(x='beam position hall horizontal pulse resolved',
       y='beam position hall vertical pulse resolved', c='energy hall', colormap='jet')

# deconvolution (without wpg)

In [ ]:
plt.figure(1, figsize=(20,20))

plt.subplot(1,1,1)
plt.imshow(pixis_image_norm, origin='lower', interpolation='nearest', aspect=1, cmap='nipy_spectral', vmin=0, vmax=1)
plt.colorbar()
plt.title('cropped measurement')

In [ ]:
pixis_centery_px = int(df0[(df0['hdf5_file_name'] == hdf5_file_name_image_widget.value) & (df0['pinholes'] == dataset_image_args_widget.value[2])]['pixis_centery_px'].iloc[0])
n = pixis_image_norm.shape[0]

xdata = list(range(n))
ydata = pixis_image_norm[pixis_centery_px,:]  # carefull, this was determined for the non-cropped pixis_image_norm
ydata = np.mean(pixis_image_norm[pixis_centery_px-50:pixis_centery_px+50,:],axis=0)  # carefull, this was determined for the non-cropped pixis_image_norm
ydata = ydata / np.max(ydata)

pixis_yshift_px = int(pixis_centery_px_widget.value)
p0 = (pixis_yshift_px, 400)
popt_gauss, pcov_gaussian = curve_fit(lambda x, m, w: gaussianbeam(x, 1, m ,w, 0), xdata, ydata, p0)
pixis_beamwidth_px = popt_gauss[1]

fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
plt.plot(xdata, ydata, 'b-', label='data')
plt.plot(xdata, gaussianbeam(xdata, 1, popt_gauss[0] ,popt_gauss[1], 0), 'r-', label='fit: m=%5.1f px, w=%5.1f px' % tuple([popt_gauss[0] ,popt_gauss[1]]))
plt.axhline(0, color='k')
plt.legend()
plt.show()

In [ ]:
z = 5781 * 1e-3
z_0 = 1067 * mm
z_T = z + z_0
z_eff = z * z_0 / (z_T)
dX_1 = 13 * 1e-6



partiallycoherent = pixis_image_norm
partiallycoherent_profile, fullycoherent, fullycoherent_profile,  partiallycoherent_rec, partiallycoherent_rec_profile, sigma_F_gamma_um_opt, F_gamma, abs_gamma, xi_um, I_bp, dX_2 = deconvmethod(partiallycoherent, z, dX_1, 20, pixis_centery_px_widget.value, 13.5*1e-9)
#partiallycoherent_rec = partiallycoherent_rec / partiallycoherent_rec[int(pixis_centery_px_widget.value), int(pixis_centerx_px_widget.value)]

partiallycoherent_rec = normalize(partiallycoherent_rec)
partiallycoherent_rec_profile = normalize(partiallycoherent_rec_profile)




cor = corr2(partiallycoherent, partiallycoherent_rec)
print('correlation=' + str(cor))


#imagesc(X2_axis*R_1,Y2_axis*R_1,log10(I_bp));


#pixis_yshift_px = int(pixis_centery_px_widget.value)
#p0 = (pixis_yshift_px, 400)
#popt_gauss, pcov_gaussian = curve_fit(lambda x, m, w: gaussianbeam(x, 1, m ,w, 0), xdata, ydata, p0)
#pixis_beamwidth_px = popt_gauss[1]
n = partiallycoherent.shape[0]
xdata = list(range(n))

fig=plt.figure(figsize=(48, 12), dpi= 80, facecolor='w', edgecolor='k')
plt.subplot(1,4,1)
plt.plot(xdata, partiallycoherent_profile, 'b-', label='partially coherent', linewidth=3)
plt.plot(xdata, fullycoherent_profile, 'r-', label='recovered fully coherent')
plt.plot(xdata, partiallycoherent_rec_profile, 'g-', label='recovered partially coherent')
#plt.plot(xdata, gaussianbeam(xdata, 1, popt_gauss[0] ,popt_gauss[1], 0), 'r-', label='fit: m=%5.1f px, w=%5.1f px' % tuple([popt_gauss[0] ,popt_gauss[1]]))
plt.axhline(0, color='k')
plt.legend()

plt.subplot(1,4,2)
#plt.contourf(x,y,psf,cmap='jet')
#plt.xlim(-5,5)
#plt.ylim(-5,5)
plt.imshow(F_gamma,cmap='jet', extent=((-n/2)*dX_1, (+n/2-1)*dX_1, -n/2*dX_1, (+n/2-1)*dX_1))
#plt.xlim(-5*dX_1,5*dX_1)
#plt.ylim(-5*dX_1,5*dX_1)


plt.subplot(1,4,3)
#plt.xlim(-5,5)
#plt.ylim(-5,5)
#plt.imshow(gamma,cmap='jet', extent=((-n/2)*dX_2, (+n/2-1)*dX_2, -n/2*dX_2, (+n/2-1)*dX_2))
plt.contourf(abs_gamma,cmap='jet', extent=((-n/2)*dX_2, (+n/2-1)*dX_2, -n/2*dX_2, (+n/2-1)*dX_2))

n = abs_gamma.shape[0]
xdata = list(range(n))
ydata = abs_gamma[int(n/2),:]
p0 = (int(n/2), 1)
popt_gauss, pcov_gaussian = curve_fit(lambda x, m, w: gaussianbeam(x, 1, m ,w, 0), xdata, ydata, p0)
plt.subplot(1,4,4)
plt.plot(xdata, ydata, 'b-', label='abs(gamma)', linewidth=1)
plt.plot(xdata, gaussianbeam(xdata, 1, popt_gauss[0] ,popt_gauss[1], 0), 'r-', label='fit: m=%5.1f px, w=%5.1f px' % tuple([popt_gauss[0] ,popt_gauss[1]]))
plt.legend()

plt.show()

fig=plt.figure(figsize=(36, 16), dpi= 80, facecolor='w', edgecolor='k')
plt.subplot(1,3,1)
plt.imshow(partiallycoherent,origin='lower', interpolation='nearest', aspect=1, cmap='jet', vmin=0, vmax=1)
plt.subplot(1,3,2)
plt.imshow(fullycoherent,origin='lower', interpolation='nearest', aspect=1, cmap='jet', vmin=0, vmax=1)

plt.subplot(1,3,3)
#plt.xlim(-5,5)
#plt.ylim(-5,5)
#plt.imshow(gamma,cmap='jet', extent=((-n/2)*dX_2, (+n/2-1)*dX_2, -n/2*dX_2, (+n/2-1)*dX_2))
#plt.imshow(np.log10(I_bp),cmap='jet', extent=((-n/2)*dX_2, (+n/2-1)*dX_2, -n/2*dX_2, (+n/2-1)*dX_2))
plt.imshow(partiallycoherent_rec,origin='lower', interpolation='nearest', aspect=1, cmap='jet', vmin=0, vmax=1)

In [ ]:
crop = 20
A_bp = fftpack.fftshift(fftpack.ifftn(fftpack.ifftshift(np.sqrt(partiallycoherent[crop:-crop,crop:-crop]))))  # amplitude
I_bp = np.abs(A_bp)**2  # intensity
fig = plt.figure(figsize=(15,15))
plt.imshow(np.log10(I_bp), cmap='jet')

## save average measurement image as .mat for MATLAB

In [ ]:
pixis_image_avg_norm = np.average(pixis_image_norm_dataset, axis=0)
pixis_image_avg_norm = pixis_image_avg_norm / np.max(pixis_image_avg_norm)

import scipy.io as sio
# Create a dictionary
adict = {}
adict['pixis_image_norm_square'] = pixis_image_norm_square
adict['pixis_image_avg_norm_square'] = pixis_image_avg_norm_square
#adict['wpg_image_norm_square'] = wpg_image_norm_square
sio.savemat('/asap3/flash/gpfs/fl24/2017/data/11004861/scratch_cc/pixis_image_ph29T1007-2c#2.mat', adict)